<a href="https://colab.research.google.com/github/PondKann/CXR_classification_model/blob/main/Inception_Kfold_3class.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1) Pretrained รันครบ กรอกแล้ว 

2) K-fold 0-100 รันครบ กรอกแล้ว แต่ไม่ได้กรอก binary

#### import

In [ ]:
import os
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
import cv2
import numpy as np
import pandas as pd
from torchsummary import summary
from skimage.io import imread, imsave
from tqdm import tqdm

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix

import PIL
from PIL import Image


In [ ]:
# set the matplotlib backend so figures can be saved in the background
import matplotlib
matplotlib.use("Agg")

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from tensorflow.keras.optimizers import SGD, Adam, RMSprop
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model
from imutils import paths
import argparse

from sklearn.model_selection import KFold
from sklearn import metrics
from scipy.stats import zscore


import matplotlib.image as mpimg
from matplotlib.pyplot import savefig

import itertools
from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels

#### Load data // KKU

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data_dir = '/content/drive/My Drive/Senior Project/CXR Image3class/Image'

train 3 class

In [ ]:
Normal151_dir = os.path.join(data_dir,'Normal151')
TB150_dir = os.path.join(data_dir,'TB150')
CA150_dir = os.path.join(data_dir,'CA150')

All_3class_dir = os.path.join(data_dir,'All_3class')

#### Load data // Gmail

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data_dir = '/content/drive/My Drive/Senior Project/CXR Image3class'

train 3 class

In [ ]:
Normal151_dir = os.path.join(data_dir,'Normal151')
TB150_dir = os.path.join(data_dir,'TB150')
CA150_dir = os.path.join(data_dir,'CA150')

All_3class_dir = os.path.join(data_dir,'All_3class')

#### train 3 class  
ข้อมูลใน All_3class_dir รวมภาพทั้ง 3 คลาส รวม 450 ภาพ TB150, Normal151 (เอามาแค่150) และ CA150 (CA คือ lungcancer)

In [ ]:
Normal151_dir = os.path.join(data_dir,'Normal151')
TB150_dir = os.path.join(data_dir,'TB150')
CA150_dir = os.path.join(data_dir,'CA150')

All_3class_dir = os.path.join(data_dir,'All_3class')

In [ ]:
normal = []
for i in range(len(os.listdir(Normal151_dir))):
    img = os.listdir(Normal151_dir)[i]
    normal.append(img)
    
tnormal = pd.DataFrame({'ImageName':[normal][0],
                                       'NameType': "Normal" ,
                                       'NumberType': 0                      })

In [ ]:
tuberculosis = []
for i in range(len(os.listdir(TB150_dir))):
    img = os.listdir(TB150_dir)[i]
    tuberculosis.append(img)

ttuberculosis = pd.DataFrame({'ImageName':[tuberculosis][0],
                                       'NameType': "Tuberculosis" ,
                                       'NumberType': 1                        })

In [ ]:
lungcancer = []
for i in range(len(os.listdir(CA150_dir))):
    img = os.listdir(CA150_dir)[i]
    lungcancer.append(img)

tlungcancer = pd.DataFrame({'ImageName':[lungcancer][0],
                                       'NameType': "Lungcancer" ,
                                       'NumberType': 2                        })

In [ ]:
data_train3 = pd.concat([tnormal[:-1], ttuberculosis, tlungcancer])
data_train3[-3:]

,ImageName,NameType,NumberType
147,ca2.jpg,Lungcancer,2
148,ca7.jpg,Lungcancer,2
149,ca26.jpg,Lungcancer,2


# K-fold Pretrained ครบ 
- 311 layer
-   filepath มี .h5 ทุกอัน
- bi cate RMSprop Adam SGD

### 1  


#### 1  soft pre bi


In [ ]:
BS = 1
filepath = "Kmodel1_soft_pre_bi.h5"                  ##

train_img_Incep = []
for ImageName in tqdm(data_train3['ImageName']):              
    image_path = All_3class_dir +'/' + ImageName                  
    img = cv2.imread(image_path)                                          
    img = cv2.resize(img, (299, 299))                                      
    train_img_Incep.append(img)
data_Incep = np.array(train_img_Incep, dtype="float32") / 255.0

labels_Incep = np.array(data_train3[['NameType']] )        
labels_Incep.shape = (len(labels_Incep),) 
le_Incep = LabelEncoder()                                                                     
labels_Incep = le_Incep.fit_transform(labels_Incep)                                         
labels_Incep = to_categorical(labels_Incep, num_classes=3)                   

aug = ImageDataGenerator( rotation_range=15,	width_shift_range=0.1, 	height_shift_range=0.1,
		shear_range=0.15, 	horizontal_flip=True, 	fill_mode="nearest")
                                                                                                           ## 
callback = EarlyStopping( monitor= "val_accuracy",    patience=15,       mode="max")

# checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1,   save_best_only=True, mode='max')
                                                                                                            ##
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5,   patience=6,     verbose=1, mode='max', min_lr=0.00001)

100%|██████████| 450/450 [02:08<00:00,  3.51it/s]


In [ ]:
base_model_Incep = tf.keras.applications.InceptionV3(input_shape=(299, 299, 3), include_top=False, weights="imagenet")
base_model_Incep.trainable = False

#Add custom head
average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model_Incep.output)
prediction_layer = tf.keras.layers.Dense(units=3, activation="softmax")(average_pooling_layer)   # unit 3
model_Incep = tf.keras.models.Model(inputs=base_model_Incep.input, outputs=prediction_layer)

model_Incep.compile(optimizer=RMSprop(learning_rate=0.0001), loss="binary_crossentropy", metrics=["accuracy"])


### K-fold
kf_Incep = KFold(5, shuffle=True, random_state=40) 

oos_y = []
oos_pred = []
fold = 0

for train, test in kf_Incep.split(data_Incep):
    fold+=1   
    print(f"Fold #{fold}")

    filename = f" Kmodel1_soft_pre_bi{fold}.h5 "
    checkpoint = ModelCheckpoint(filename, monitor='val_accuracy', verbose=1,   save_best_only=True, mode='max')
        
    x_train_Incep = data_Incep[train]
    y_train_Incep = labels_Incep[train]
    x_test_Incep = data_Incep[test]
    y_test_Incep = labels_Incep[test]

    H_Incep = model_Incep.fit( x=aug.flow(x_train_Incep, y_train_Incep, batch_size=BS),
	                                            validation_data=(x_test_Incep, y_test_Incep),
	                                            steps_per_epoch=len(x_train_Incep) // BS,
	                                            epochs=100,  callbacks = [callback,checkpoint,reduce_lr])

    pred_Incep = model_Incep.predict(x_test_Incep)
    
    oos_y.append(y_test_Incep)
    oos_pred.append(pred_Incep)  
    print(classification_report(y_test_Incep.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        pred_Incep.argmax(axis=1),  
                                        target_names=le_Incep.classes_))
    
oos_y = np.concatenate(oos_y)
oos_pred = np.concatenate(oos_pred)
print(classification_report(oos_y.argmax(axis=1), oos_pred.argmax(axis=1),  target_names=le_Incep.classes_))

87924736/87910968 [==============================] - 1s 0us/step
Fold #1
Epoch 1/100
360/360 [==============================] - ETA: 0s - loss: 0.6350 - accuracy: 0.4028
Epoch 1: val_accuracy improved from -inf to 0.43333, saving model to  Kmodel1_soft_pre_bi1.h5 
INFO:tensorflow:Assets written to:  Kmodel1_soft_pre_bi1.h5 /assets
360/360 [==============================] - 77s 173ms/step - loss: 0.6350 - accuracy: 0.4028 - val_loss: 0.6075 - val_accuracy: 0.4333 - lr: 1.0000e-04
Epoch 2/100
360/360 [==============================] - ETA: 0s - loss: 0.5957 - accuracy: 0.4833
Epoch 2: val_accuracy improved from 0.43333 to 0.52222, saving model to  Kmodel1_soft_pre_bi1.h5 
INFO:tensorflow:Assets written to:  Kmodel1_soft_pre_bi1.h5 /assets
360/360 [==============================] - 53s 148ms/step - loss: 0.5957 - accuracy: 0.4833 - val_loss: 0.5742 - val_accuracy: 0.5222 - lr: 1.0000e-04
Epoch 3/100
360/360 [==============================] - ETA: 0s - loss: 0.5607 - accuracy: 0.5694
Epoch

#### 1 soft pre bi adam  


In [ ]:
BS = 1
filepath = "Kmodel1_soft_pre_bi_adam.h5"                  ##

train_img_Incep = []
for ImageName in tqdm(data_train3['ImageName']):              
    image_path = All_3class_dir +'/' + ImageName                  
    img = cv2.imread(image_path)                                          
    img = cv2.resize(img, (299, 299))                                      
    train_img_Incep.append(img)
data_Incep = np.array(train_img_Incep, dtype="float32") / 255.0

labels_Incep = np.array(data_train3[['NameType']] )        
labels_Incep.shape = (len(labels_Incep),) 
le_Incep = LabelEncoder()                                                                     
labels_Incep = le_Incep.fit_transform(labels_Incep)                                         
labels_Incep = to_categorical(labels_Incep, num_classes=3)                   

aug = ImageDataGenerator( rotation_range=15,	width_shift_range=0.1, 	height_shift_range=0.1,
		shear_range=0.15, 	horizontal_flip=True, 	fill_mode="nearest")
                                                                                                           ## 
callback = EarlyStopping( monitor= "val_accuracy",    patience=15,       mode="max")

# checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1,   save_best_only=True, mode='max')
                                                                                                            ##
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5,   patience=6,     verbose=1, mode='max', min_lr=0.00001)

100%|██████████| 450/450 [01:59<00:00,  3.75it/s]


In [ ]:
base_model_Incep = tf.keras.applications.InceptionV3(input_shape=(299, 299, 3), include_top=False, weights="imagenet")
base_model_Incep.trainable = False


#Add custom head
average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model_Incep.output)
prediction_layer = tf.keras.layers.Dense(units=3, activation="softmax")(average_pooling_layer)   # unit 3
model_Incep = tf.keras.models.Model(inputs=base_model_Incep.input, outputs=prediction_layer)

model_Incep.compile(optimizer=Adam(learning_rate=0.0001), loss="binary_crossentropy", metrics=["accuracy"])


### K-fold
kf_Incep = KFold(5, shuffle=True, random_state=40) 

oos_y = []
oos_pred = []
fold = 0

for train, test in kf_Incep.split(data_Incep):
    fold+=1   
    print(f"Fold #{fold}")

    filename = f" Kmodel1_soft_pre_bi_adam{fold}.h5 "
    checkpoint = ModelCheckpoint(filename, monitor='val_accuracy', verbose=1,   save_best_only=True, mode='max')
        
    x_train_Incep = data_Incep[train]
    y_train_Incep = labels_Incep[train]
    x_test_Incep = data_Incep[test]
    y_test_Incep = labels_Incep[test]

    H_Incep = model_Incep.fit( x=aug.flow(x_train_Incep, y_train_Incep, batch_size=BS),
	                                            validation_data=(x_test_Incep, y_test_Incep),
	                                            steps_per_epoch=len(x_train_Incep) // BS,
	                                            epochs=100,  callbacks = [callback,checkpoint,reduce_lr])

    pred_Incep = model_Incep.predict(x_test_Incep)
    
    oos_y.append(y_test_Incep)
    oos_pred.append(pred_Incep)  
    print(classification_report(y_test_Incep.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        pred_Incep.argmax(axis=1),  
                                        target_names=le_Incep.classes_))
    
oos_y = np.concatenate(oos_y)
oos_pred = np.concatenate(oos_pred)
print(classification_report(oos_y.argmax(axis=1), oos_pred.argmax(axis=1),  target_names=le_Incep.classes_))

87924736/87910968 [==============================] - 1s 0us/step
Fold #1
Epoch 1/100
359/360 [============================>.] - ETA: 0s - loss: 0.6371 - accuracy: 0.4178
Epoch 1: val_accuracy improved from -inf to 0.52222, saving model to  Kmodel1_soft_pre_bi_adam1.h5 
INFO:tensorflow:Assets written to:  Kmodel1_soft_pre_bi_adam1.h5 /assets
360/360 [==============================] - 69s 152ms/step - loss: 0.6366 - accuracy: 0.4194 - val_loss: 0.6047 - val_accuracy: 0.5222 - lr: 1.0000e-04
Epoch 2/100
360/360 [==============================] - ETA: 0s - loss: 0.5824 - accuracy: 0.5250
Epoch 2: val_accuracy improved from 0.52222 to 0.58889, saving model to  Kmodel1_soft_pre_bi_adam1.h5 
INFO:tensorflow:Assets written to:  Kmodel1_soft_pre_bi_adam1.h5 /assets
360/360 [==============================] - 48s 133ms/step - loss: 0.5824 - accuracy: 0.5250 - val_loss: 0.5744 - val_accuracy: 0.5889 - lr: 1.0000e-04
Epoch 3/100
360/360 [==============================] - ETA: 0s - loss: 0.5544 - ac

#### 1 soft pre bi sgd  

In [ ]:
BS = 1
filepath = "Kmodel1_soft_pre_bi_sgd.h5"                  ##

train_img_Incep = []
for ImageName in tqdm(data_train3['ImageName']):              
    image_path = All_3class_dir +'/' + ImageName                  
    img = cv2.imread(image_path)                                          
    img = cv2.resize(img, (299, 299))                                      
    train_img_Incep.append(img)
data_Incep = np.array(train_img_Incep, dtype="float32") / 255.0

labels_Incep = np.array(data_train3[['NameType']] )        
labels_Incep.shape = (len(labels_Incep),) 
le_Incep = LabelEncoder()                                                                     
labels_Incep = le_Incep.fit_transform(labels_Incep)                                         
labels_Incep = to_categorical(labels_Incep, num_classes=3)                   

aug = ImageDataGenerator( rotation_range=15,	width_shift_range=0.1, 	height_shift_range=0.1,
		shear_range=0.15, 	horizontal_flip=True, 	fill_mode="nearest")
                                                                                                           ## 
callback = EarlyStopping( monitor= "val_accuracy",    patience=15,       mode="max")

# checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1,   save_best_only=True, mode='max')
                                                                                                            ##
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5,   patience=6,     verbose=1, mode='max', min_lr=0.00001)

100%|██████████| 450/450 [00:05<00:00, 88.74it/s] 


In [ ]:
base_model_Incep = tf.keras.applications.InceptionV3(input_shape=(299, 299, 3), include_top=False, weights="imagenet")
base_model_Incep.trainable = False

#Add custom head
average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model_Incep.output)
prediction_layer = tf.keras.layers.Dense(units=3, activation="softmax")(average_pooling_layer)   # unit 3
model_Incep = tf.keras.models.Model(inputs=base_model_Incep.input, outputs=prediction_layer)

model_Incep.compile(optimizer=SGD(learning_rate=0.0001), loss="binary_crossentropy", metrics=["accuracy"])


### K-fold
kf_Incep = KFold(5, shuffle=True, random_state=40) 

oos_y = []
oos_pred = []
fold = 0

for train, test in kf_Incep.split(data_Incep):
    fold+=1   
    print(f"Fold #{fold}")

    filename = f" Kmodel1_soft_pre_bi_sgd{fold}.h5 "
    checkpoint = ModelCheckpoint(filename, monitor='val_accuracy', verbose=1,   save_best_only=True, mode='max')
        
    x_train_Incep = data_Incep[train]
    y_train_Incep = labels_Incep[train]
    x_test_Incep = data_Incep[test]
    y_test_Incep = labels_Incep[test]

    H_Incep = model_Incep.fit( x=aug.flow(x_train_Incep, y_train_Incep, batch_size=BS),
	                                            validation_data=(x_test_Incep, y_test_Incep),
	                                            steps_per_epoch=len(x_train_Incep) // BS,
	                                            epochs=100,  callbacks = [callback,checkpoint,reduce_lr])

    pred_Incep = model_Incep.predict(x_test_Incep)
    
    oos_y.append(y_test_Incep)
    oos_pred.append(pred_Incep)  
    print(classification_report(y_test_Incep.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        pred_Incep.argmax(axis=1),  
                                        target_names=le_Incep.classes_))
    
oos_y = np.concatenate(oos_y)
oos_pred = np.concatenate(oos_pred)
print(classification_report(oos_y.argmax(axis=1), oos_pred.argmax(axis=1),  target_names=le_Incep.classes_))

Fold #1
Epoch 1/100
359/360 [============================>.] - ETA: 0s - loss: 0.6920 - accuracy: 0.3844
Epoch 1: val_accuracy improved from -inf to 0.43333, saving model to  Kmodel1_soft_pre_bi_sgd1.h5 
INFO:tensorflow:Assets written to:  Kmodel1_soft_pre_bi_sgd1.h5 /assets
360/360 [==============================] - 58s 148ms/step - loss: 0.6919 - accuracy: 0.3833 - val_loss: 0.6280 - val_accuracy: 0.4333 - lr: 1.0000e-04
Epoch 2/100
360/360 [==============================] - ETA: 0s - loss: 0.6424 - accuracy: 0.3694
Epoch 2: val_accuracy improved from 0.43333 to 0.45556, saving model to  Kmodel1_soft_pre_bi_sgd1.h5 
INFO:tensorflow:Assets written to:  Kmodel1_soft_pre_bi_sgd1.h5 /assets
360/360 [==============================] - 52s 145ms/step - loss: 0.6424 - accuracy: 0.3694 - val_loss: 0.6152 - val_accuracy: 0.4556 - lr: 1.0000e-04
Epoch 3/100
360/360 [==============================] - ETA: 0s - loss: 0.6312 - accuracy: 0.4250
Epoch 3: val_accuracy improved from 0.45556 to 0.46667

#### 1 soft pre cate 


In [ ]:
BS = 1
filepath = "Kmodel1_soft_pre_cate.h5"                  ##

train_img_Incep = []
for ImageName in tqdm(data_train3['ImageName']):              
    image_path = All_3class_dir +'/' + ImageName                  
    img = cv2.imread(image_path)                                          
    img = cv2.resize(img, (299, 299))                                      
    train_img_Incep.append(img)
data_Incep = np.array(train_img_Incep, dtype="float32") / 255.0

labels_Incep = np.array(data_train3[['NameType']] )        
labels_Incep.shape = (len(labels_Incep),) 
le_Incep = LabelEncoder()                                                                     
labels_Incep = le_Incep.fit_transform(labels_Incep)                                         
labels_Incep = to_categorical(labels_Incep, num_classes=3)                   

aug = ImageDataGenerator( rotation_range=15,	width_shift_range=0.1, 	height_shift_range=0.1,
		shear_range=0.15, 	horizontal_flip=True, 	fill_mode="nearest")
                                                                                                           ## 
callback = EarlyStopping( monitor= "val_accuracy",    patience=15,       mode="max")

# checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1,   save_best_only=True, mode='max')
                                                                                                            ##
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5,   patience=6,     verbose=1, mode='max', min_lr=0.00001)

100%|██████████| 450/450 [00:04<00:00, 91.95it/s] 


In [ ]:
base_model_Incep = tf.keras.applications.InceptionV3(input_shape=(299, 299, 3), include_top=False, weights="imagenet")
base_model_Incep.trainable = False


#Add custom head
average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model_Incep.output)
prediction_layer = tf.keras.layers.Dense(units=3, activation="softmax")(average_pooling_layer)   # unit 3
model_Incep = tf.keras.models.Model(inputs=base_model_Incep.input, outputs=prediction_layer)

model_Incep.compile(optimizer=RMSprop(learning_rate=0.0001), loss="categorical_crossentropy", metrics=["accuracy"])


### K-fold
kf_Incep = KFold(5, shuffle=True, random_state=40) 

oos_y = []
oos_pred = []
fold = 0

for train, test in kf_Incep.split(data_Incep):
    fold+=1   
    print(f"Fold #{fold}")

    filename = f" Kmodel1_soft_pre_cate{fold}.h5 "
    checkpoint = ModelCheckpoint(filename, monitor='val_accuracy', verbose=1,   save_best_only=True, mode='max')
        
    x_train_Incep = data_Incep[train]
    y_train_Incep = labels_Incep[train]
    x_test_Incep = data_Incep[test]
    y_test_Incep = labels_Incep[test]

    H_Incep = model_Incep.fit( x=aug.flow(x_train_Incep, y_train_Incep, batch_size=BS),
	                                            validation_data=(x_test_Incep, y_test_Incep),
	                                            steps_per_epoch=len(x_train_Incep) // BS,
	                                            epochs=100,  callbacks = [callback,checkpoint,reduce_lr])

    pred_Incep = model_Incep.predict(x_test_Incep)
    
    oos_y.append(y_test_Incep)
    oos_pred.append(pred_Incep)  
    print(classification_report(y_test_Incep.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        pred_Incep.argmax(axis=1),  
                                        target_names=le_Incep.classes_))
    
oos_y = np.concatenate(oos_y)
oos_pred = np.concatenate(oos_pred)
print(classification_report(oos_y.argmax(axis=1), oos_pred.argmax(axis=1),  target_names=le_Incep.classes_))

Fold #1
Epoch 1/100
359/360 [============================>.] - ETA: 0s - loss: 1.0967 - accuracy: 0.4234
Epoch 1: val_accuracy improved from -inf to 0.40000, saving model to  Kmodel1_soft_pre_cate1.h5 
INFO:tensorflow:Assets written to:  Kmodel1_soft_pre_cate1.h5 /assets
360/360 [==============================] - 54s 136ms/step - loss: 1.0948 - accuracy: 0.4250 - val_loss: 1.0515 - val_accuracy: 0.4000 - lr: 1.0000e-04
Epoch 2/100
359/360 [============================>.] - ETA: 0s - loss: 1.0079 - accuracy: 0.4986
Epoch 2: val_accuracy improved from 0.40000 to 0.62222, saving model to  Kmodel1_soft_pre_cate1.h5 
INFO:tensorflow:Assets written to:  Kmodel1_soft_pre_cate1.h5 /assets
360/360 [==============================] - 48s 133ms/step - loss: 1.0071 - accuracy: 0.5000 - val_loss: 0.9051 - val_accuracy: 0.6222 - lr: 1.0000e-04
Epoch 3/100
359/360 [============================>.] - ETA: 0s - loss: 0.9229 - accuracy: 0.6128
Epoch 3: val_accuracy did not improve from 0.62222
360/360 [==

#### 1 soft pre cate adam


In [ ]:
BS = 1
filepath = "Kmodel1_soft_pre_cate_adam.h5"                  ##

train_img_Incep = []
for ImageName in tqdm(data_train3['ImageName']):              
    image_path = All_3class_dir +'/' + ImageName                  
    img = cv2.imread(image_path)                                          
    img = cv2.resize(img, (299, 299))                                      
    train_img_Incep.append(img)
data_Incep = np.array(train_img_Incep, dtype="float32") / 255.0

labels_Incep = np.array(data_train3[['NameType']] )        
labels_Incep.shape = (len(labels_Incep),) 
le_Incep = LabelEncoder()                                                                     
labels_Incep = le_Incep.fit_transform(labels_Incep)                                         
labels_Incep = to_categorical(labels_Incep, num_classes=3)                   

aug = ImageDataGenerator( rotation_range=15,	width_shift_range=0.1, 	height_shift_range=0.1,
		shear_range=0.15, 	horizontal_flip=True, 	fill_mode="nearest")
                                                                                                           ## 
callback = EarlyStopping( monitor= "val_accuracy",    patience=15,       mode="max")

# checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1,   save_best_only=True, mode='max')
                                                                                                            ##
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5,   patience=6,     verbose=1, mode='max', min_lr=0.00001)

100%|██████████| 450/450 [00:05<00:00, 83.51it/s] 


In [ ]:
base_model_Incep = tf.keras.applications.InceptionV3(input_shape=(299, 299, 3), include_top=False, weights="imagenet")
base_model_Incep.trainable = False


#Add custom head
average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model_Incep.output)
prediction_layer = tf.keras.layers.Dense(units=3, activation="softmax")(average_pooling_layer)   # unit 3
model_Incep = tf.keras.models.Model(inputs=base_model_Incep.input, outputs=prediction_layer)

model_Incep.compile(optimizer=Adam(learning_rate=0.0001), loss="categorical_crossentropy", metrics=["accuracy"])

### K-fold
kf_Incep = KFold(5, shuffle=True, random_state=40) 

oos_y = []
oos_pred = []
fold = 0

for train, test in kf_Incep.split(data_Incep):
    fold+=1   
    print(f"Fold #{fold}")

    filename = f" Kmodel1_soft_pre_cate_adam{fold}.h5 "
    checkpoint = ModelCheckpoint(filename, monitor='val_accuracy', verbose=1,   save_best_only=True, mode='max')
        
    x_train_Incep = data_Incep[train]
    y_train_Incep = labels_Incep[train]
    x_test_Incep = data_Incep[test]
    y_test_Incep = labels_Incep[test]

    H_Incep = model_Incep.fit( x=aug.flow(x_train_Incep, y_train_Incep, batch_size=BS),
	                                            validation_data=(x_test_Incep, y_test_Incep),
	                                            steps_per_epoch=len(x_train_Incep) // BS,
	                                            epochs=100,  callbacks = [callback,checkpoint,reduce_lr])

    pred_Incep = model_Incep.predict(x_test_Incep)
    
    oos_y.append(y_test_Incep)
    oos_pred.append(pred_Incep)  
    print(classification_report(y_test_Incep.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        pred_Incep.argmax(axis=1),  
                                        target_names=le_Incep.classes_))
    
oos_y = np.concatenate(oos_y)
oos_pred = np.concatenate(oos_pred)
print(classification_report(oos_y.argmax(axis=1), oos_pred.argmax(axis=1),  target_names=le_Incep.classes_))

Fold #1
Epoch 1/100
360/360 [==============================] - ETA: 0s - loss: 1.0828 - accuracy: 0.4194
Epoch 1: val_accuracy improved from -inf to 0.41111, saving model to  Kmodel1_soft_pre_cate_adam1.h5 
INFO:tensorflow:Assets written to:  Kmodel1_soft_pre_cate_adam1.h5 /assets
360/360 [==============================] - 57s 145ms/step - loss: 1.0828 - accuracy: 0.4194 - val_loss: 1.0432 - val_accuracy: 0.4111 - lr: 1.0000e-04
Epoch 2/100
359/360 [============================>.] - ETA: 0s - loss: 0.9801 - accuracy: 0.5097
Epoch 2: val_accuracy improved from 0.41111 to 0.51111, saving model to  Kmodel1_soft_pre_cate_adam1.h5 
INFO:tensorflow:Assets written to:  Kmodel1_soft_pre_cate_adam1.h5 /assets
360/360 [==============================] - 51s 141ms/step - loss: 0.9801 - accuracy: 0.5083 - val_loss: 0.9496 - val_accuracy: 0.5111 - lr: 1.0000e-04
Epoch 3/100
360/360 [==============================] - ETA: 0s - loss: 0.9014 - accuracy: 0.6083
Epoch 3: val_accuracy improved from 0.5111

#### 1 soft pre cate sgd


In [ ]:
BS = 1
filepath = "Kmodel1_soft_pre_cate_sgd.h5"                  ##

train_img_Incep = []
for ImageName in tqdm(data_train3['ImageName']):              
    image_path = All_3class_dir +'/' + ImageName                  
    img = cv2.imread(image_path)                                          
    img = cv2.resize(img, (299, 299))                                      
    train_img_Incep.append(img)
data_Incep = np.array(train_img_Incep, dtype="float32") / 255.0

labels_Incep = np.array(data_train3[['NameType']] )        
labels_Incep.shape = (len(labels_Incep),) 
le_Incep = LabelEncoder()                                                                     
labels_Incep = le_Incep.fit_transform(labels_Incep)                                         
labels_Incep = to_categorical(labels_Incep, num_classes=3)                   

aug = ImageDataGenerator( rotation_range=15,	width_shift_range=0.1, 	height_shift_range=0.1,
		shear_range=0.15, 	horizontal_flip=True, 	fill_mode="nearest")
                                                                                                           ## 
callback = EarlyStopping( monitor= "val_accuracy",    patience=15,       mode="max")

# checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1,   save_best_only=True, mode='max')
                                                                                                            ##
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5,   patience=6,     verbose=1, mode='max', min_lr=0.00001)

100%|██████████| 450/450 [00:12<00:00, 36.17it/s]


In [ ]:
base_model_Incep = tf.keras.applications.InceptionV3(input_shape=(299, 299, 3), include_top=False, weights="imagenet")
base_model_Incep.trainable = False


#Add custom head
average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model_Incep.output)
prediction_layer = tf.keras.layers.Dense(units=3, activation="softmax")(average_pooling_layer)   # unit 3
model_Incep = tf.keras.models.Model(inputs=base_model_Incep.input, outputs=prediction_layer)

model_Incep.compile(optimizer=SGD(learning_rate=0.0001), loss="categorical_crossentropy", metrics=["accuracy"])

### K-fold
kf_Incep = KFold(5, shuffle=True, random_state=40) 

oos_y = []
oos_pred = []
fold = 0

for train, test in kf_Incep.split(data_Incep):
    fold+=1   
    print(f"Fold #{fold}")

    filename = f" Kmodel1_soft_pre_cate_sgd{fold}.h5 "
    checkpoint = ModelCheckpoint(filename, monitor='val_accuracy', verbose=1,   save_best_only=True, mode='max')
        
    x_train_Incep = data_Incep[train]
    y_train_Incep = labels_Incep[train]
    x_test_Incep = data_Incep[test]
    y_test_Incep = labels_Incep[test]

    H_Incep = model_Incep.fit( x=aug.flow(x_train_Incep, y_train_Incep, batch_size=BS),
	                                            validation_data=(x_test_Incep, y_test_Incep),
	                                            steps_per_epoch=len(x_train_Incep) // BS,
	                                            epochs=100,  callbacks = [callback,checkpoint,reduce_lr])

    pred_Incep = model_Incep.predict(x_test_Incep)
    
    oos_y.append(y_test_Incep)
    oos_pred.append(pred_Incep)  
    print(classification_report(y_test_Incep.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        pred_Incep.argmax(axis=1),  
                                        target_names=le_Incep.classes_))
    
oos_y = np.concatenate(oos_y)
oos_pred = np.concatenate(oos_pred)
print(classification_report(oos_y.argmax(axis=1), oos_pred.argmax(axis=1),  target_names=le_Incep.classes_))

Fold #1
Epoch 1/100
360/360 [==============================] - ETA: 0s - loss: 1.1365 - accuracy: 0.3861
Epoch 1: val_accuracy improved from -inf to 0.42222, saving model to  Kmodel1_soft_pre_cate_sgd1.h5 
INFO:tensorflow:Assets written to:  Kmodel1_soft_pre_cate_sgd1.h5 /assets
360/360 [==============================] - 84s 213ms/step - loss: 1.1365 - accuracy: 0.3861 - val_loss: 1.0322 - val_accuracy: 0.4222 - lr: 1.0000e-04
Epoch 2/100
360/360 [==============================] - ETA: 0s - loss: 1.0871 - accuracy: 0.3972
Epoch 2: val_accuracy improved from 0.42222 to 0.48889, saving model to  Kmodel1_soft_pre_cate_sgd1.h5 
INFO:tensorflow:Assets written to:  Kmodel1_soft_pre_cate_sgd1.h5 /assets
360/360 [==============================] - 73s 204ms/step - loss: 1.0871 - accuracy: 0.3972 - val_loss: 1.0108 - val_accuracy: 0.4889 - lr: 1.0000e-04
Epoch 3/100
359/360 [============================>.] - ETA: 0s - loss: 1.0644 - accuracy: 0.4457
Epoch 3: val_accuracy improved from 0.48889 to

### 8 


#### 8  soft pre bi


In [ ]:
BS = 8
filepath = "Kmodel8_soft_pre_bi.h5"                  ##

train_img_Incep = []
for ImageName in tqdm(data_train3['ImageName']):              
    image_path = All_3class_dir +'/' + ImageName                  
    img = cv2.imread(image_path)                                          
    img = cv2.resize(img, (299, 299))                                      
    train_img_Incep.append(img)
data_Incep = np.array(train_img_Incep, dtype="float32") / 255.0

labels_Incep = np.array(data_train3[['NameType']] )        
labels_Incep.shape = (len(labels_Incep),) 
le_Incep = LabelEncoder()                                                                     
labels_Incep = le_Incep.fit_transform(labels_Incep)                                         
labels_Incep = to_categorical(labels_Incep, num_classes=3)                   

aug = ImageDataGenerator( rotation_range=15,	width_shift_range=0.1, 	height_shift_range=0.1,
		shear_range=0.15, 	horizontal_flip=True, 	fill_mode="nearest")
                                                                                                           ## 
callback = EarlyStopping( monitor= "val_accuracy",    patience=15,       mode="max")

# checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1,   save_best_only=True, mode='max')
                                                                                                            ##
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5,   patience=6,     verbose=1, mode='max', min_lr=0.00001)

100%|██████████| 450/450 [01:44<00:00,  4.31it/s]


In [ ]:
base_model_Incep = tf.keras.applications.InceptionV3(input_shape=(299, 299, 3), include_top=False, weights="imagenet")
base_model_Incep.trainable = False


#Add custom head
average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model_Incep.output)
prediction_layer = tf.keras.layers.Dense(units=3, activation="softmax")(average_pooling_layer)   # unit 3
model_Incep = tf.keras.models.Model(inputs=base_model_Incep.input, outputs=prediction_layer)

model_Incep.compile(optimizer=RMSprop(learning_rate=0.0001), loss="binary_crossentropy", metrics=["accuracy"])


### K-fold
kf_Incep = KFold(5, shuffle=True, random_state=40) 

oos_y = []
oos_pred = []
fold = 0

for train, test in kf_Incep.split(data_Incep):
    fold+=1   
    print(f"Fold #{fold}")

    filename = f" Kmodel8_soft_pre_bi{fold}.h5 "
    checkpoint = ModelCheckpoint(filename, monitor='val_accuracy', verbose=1,   save_best_only=True, mode='max')
        
    x_train_Incep = data_Incep[train]
    y_train_Incep = labels_Incep[train]
    x_test_Incep = data_Incep[test]
    y_test_Incep = labels_Incep[test]

    H_Incep = model_Incep.fit( x=aug.flow(x_train_Incep, y_train_Incep, batch_size=BS),
	                                            validation_data=(x_test_Incep, y_test_Incep),
	                                            steps_per_epoch=len(x_train_Incep) // BS,
	                                            epochs=100,  callbacks = [callback,checkpoint,reduce_lr])

    pred_Incep = model_Incep.predict(x_test_Incep)
    
    oos_y.append(y_test_Incep)
    oos_pred.append(pred_Incep)  
    print(classification_report(y_test_Incep.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        pred_Incep.argmax(axis=1),  
                                        target_names=le_Incep.classes_))
    
oos_y = np.concatenate(oos_y)
oos_pred = np.concatenate(oos_pred)
print(classification_report(oos_y.argmax(axis=1), oos_pred.argmax(axis=1),  target_names=le_Incep.classes_))

87924736/87910968 [==============================] - 1s 0us/step
Fold #1
Epoch 1/100
45/45 [==============================] - ETA: 0s - loss: 0.6274 - accuracy: 0.4111
Epoch 1: val_accuracy improved from -inf to 0.50000, saving model to  Kmodel8_soft_pre_bi1.h5 
INFO:tensorflow:Assets written to:  Kmodel8_soft_pre_bi1.h5 /assets
45/45 [==============================] - 77s 1s/step - loss: 0.6274 - accuracy: 0.4111 - val_loss: 0.6159 - val_accuracy: 0.5000 - lr: 1.0000e-04
Epoch 2/100
45/45 [==============================] - ETA: 0s - loss: 0.6060 - accuracy: 0.4750
Epoch 2: val_accuracy did not improve from 0.50000
45/45 [==============================] - 10s 215ms/step - loss: 0.6060 - accuracy: 0.4750 - val_loss: 0.5993 - val_accuracy: 0.5000 - lr: 1.0000e-04
Epoch 3/100
45/45 [==============================] - ETA: 0s - loss: 0.5914 - accuracy: 0.5333
Epoch 3: val_accuracy improved from 0.50000 to 0.53333, saving model to  Kmodel8_soft_pre_bi1.h5 
INFO:tensorflow:Assets written to: 

In [ ]:
H_Incep.history.keys()

dict_keys(['loss', 'accuracy', 'val_loss', 'val_accuracy', 'lr'])

In [ ]:
lr = H_Incep.history['lr']

plt.figure(figsize=(20,15))

plt.subplot(2, 2, 1)
plt.plot(lr, label = " Learning rate")

#### 8 soft pre bi adam  


In [ ]:
BS = 8
filepath = "Kmodel8_soft_pre_bi_adam.h5"                  ##

train_img_Incep = []
for ImageName in tqdm(data_train3['ImageName']):              
    image_path = All_3class_dir +'/' + ImageName                  
    img = cv2.imread(image_path)                                          
    img = cv2.resize(img, (299, 299))                                      
    train_img_Incep.append(img)
data_Incep = np.array(train_img_Incep, dtype="float32") / 255.0

labels_Incep = np.array(data_train3[['NameType']] )        
labels_Incep.shape = (len(labels_Incep),) 
le_Incep = LabelEncoder()                                                                     
labels_Incep = le_Incep.fit_transform(labels_Incep)                                         
labels_Incep = to_categorical(labels_Incep, num_classes=3)                   

aug = ImageDataGenerator( rotation_range=15,	width_shift_range=0.1, 	height_shift_range=0.1,
		shear_range=0.15, 	horizontal_flip=True, 	fill_mode="nearest")
                                                                                                           ## 
callback = EarlyStopping( monitor= "val_accuracy",    patience=15,       mode="max")

# checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1,   save_best_only=True, mode='max')
                                                                                                            ##
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5,   patience=6,     verbose=1, mode='max', min_lr=0.00001)

100%|██████████| 450/450 [00:10<00:00, 43.01it/s]


In [ ]:
base_model_Incep = tf.keras.applications.InceptionV3(input_shape=(299, 299, 3), include_top=False, weights="imagenet")
base_model_Incep.trainable = False

#Add custom head
average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model_Incep.output)
prediction_layer = tf.keras.layers.Dense(units=3, activation="softmax")(average_pooling_layer)   # unit 3
model_Incep = tf.keras.models.Model(inputs=base_model_Incep.input, outputs=prediction_layer)

model_Incep.compile(optimizer=Adam(learning_rate=0.0001), loss="binary_crossentropy", metrics=["accuracy"])


### K-fold
kf_Incep = KFold(5, shuffle=True, random_state=40) 

oos_y = []
oos_pred = []
fold = 0

for train, test in kf_Incep.split(data_Incep):
    fold+=1   
    print(f"Fold #{fold}")

    filename = f" Kmodel8_soft_pre_bi_adam{fold}.h5 "
    checkpoint = ModelCheckpoint(filename, monitor='val_accuracy', verbose=1,   save_best_only=True, mode='max')
        
    x_train_Incep = data_Incep[train]
    y_train_Incep = labels_Incep[train]
    x_test_Incep = data_Incep[test]
    y_test_Incep = labels_Incep[test]

    H_Incep = model_Incep.fit( x=aug.flow(x_train_Incep, y_train_Incep, batch_size=BS),
	                                            validation_data=(x_test_Incep, y_test_Incep),
	                                            steps_per_epoch=len(x_train_Incep) // BS,
	                                            epochs=100,  callbacks = [callback,checkpoint,reduce_lr])

    pred_Incep = model_Incep.predict(x_test_Incep)
    
    oos_y.append(y_test_Incep)
    oos_pred.append(pred_Incep)  
    print(classification_report(y_test_Incep.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        pred_Incep.argmax(axis=1),  
                                        target_names=le_Incep.classes_))
    
oos_y = np.concatenate(oos_y)
oos_pred = np.concatenate(oos_pred)
print(classification_report(oos_y.argmax(axis=1), oos_pred.argmax(axis=1),  target_names=le_Incep.classes_))

Fold #1
Epoch 1/100
45/45 [==============================] - ETA: 0s - loss: 0.6737 - accuracy: 0.3556
Epoch 1: val_accuracy improved from -inf to 0.38889, saving model to  Kmodel8_soft_pre_bi_adam1.h5 
INFO:tensorflow:Assets written to:  Kmodel8_soft_pre_bi_adam1.h5 /assets
45/45 [==============================] - 50s 992ms/step - loss: 0.6737 - accuracy: 0.3556 - val_loss: 0.6233 - val_accuracy: 0.3889 - lr: 1.0000e-04
Epoch 2/100
45/45 [==============================] - ETA: 0s - loss: 0.6211 - accuracy: 0.4250
Epoch 2: val_accuracy improved from 0.38889 to 0.44444, saving model to  Kmodel8_soft_pre_bi_adam1.h5 
INFO:tensorflow:Assets written to:  Kmodel8_soft_pre_bi_adam1.h5 /assets
45/45 [==============================] - 44s 997ms/step - loss: 0.6211 - accuracy: 0.4250 - val_loss: 0.6118 - val_accuracy: 0.4444 - lr: 1.0000e-04
Epoch 3/100
45/45 [==============================] - ETA: 0s - loss: 0.6150 - accuracy: 0.4667
Epoch 3: val_accuracy improved from 0.44444 to 0.47778, savi

#### 8 soft pre bi sgd  


In [ ]:
BS = 8
filepath = "Kmodel8_soft_pre_bi_sgd.h5"                  ##

train_img_Incep = []
for ImageName in tqdm(data_train3['ImageName']):              
    image_path = All_3class_dir +'/' + ImageName                  
    img = cv2.imread(image_path)                                          
    img = cv2.resize(img, (299, 299))                                      
    train_img_Incep.append(img)
data_Incep = np.array(train_img_Incep, dtype="float32") / 255.0

labels_Incep = np.array(data_train3[['NameType']] )        
labels_Incep.shape = (len(labels_Incep),) 
le_Incep = LabelEncoder()                                                                     
labels_Incep = le_Incep.fit_transform(labels_Incep)                                         
labels_Incep = to_categorical(labels_Incep, num_classes=3)                   

aug = ImageDataGenerator( rotation_range=15,	width_shift_range=0.1, 	height_shift_range=0.1,
		shear_range=0.15, 	horizontal_flip=True, 	fill_mode="nearest")
                                                                                                           ## 
callback = EarlyStopping( monitor= "val_accuracy",    patience=15,       mode="max")

# checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1,   save_best_only=True, mode='max')
                                                                                                            ##
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5,   patience=6,     verbose=1, mode='max', min_lr=0.00001)

100%|██████████| 450/450 [01:37<00:00,  4.62it/s]


In [ ]:
base_model_Incep = tf.keras.applications.InceptionV3(input_shape=(299, 299, 3), include_top=False, weights="imagenet")
base_model_Incep.trainable = False

#Add custom head
average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model_Incep.output)
prediction_layer = tf.keras.layers.Dense(units=3, activation="softmax")(average_pooling_layer)   # unit 3
model_Incep = tf.keras.models.Model(inputs=base_model_Incep.input, outputs=prediction_layer)

model_Incep.compile(optimizer=SGD(learning_rate=0.0001), loss="binary_crossentropy", metrics=["accuracy"])


### K-fold
kf_Incep = KFold(5, shuffle=True, random_state=40) 

oos_y = []
oos_pred = []
fold = 0

for train, test in kf_Incep.split(data_Incep):
    fold+=1   
    print(f"Fold #{fold}")

    filename = f" Kmodel8_soft_pre_bi_sgd{fold}.h5 "
    checkpoint = ModelCheckpoint(filename, monitor='val_accuracy', verbose=1,   save_best_only=True, mode='max')
        
    x_train_Incep = data_Incep[train]
    y_train_Incep = labels_Incep[train]
    x_test_Incep = data_Incep[test]
    y_test_Incep = labels_Incep[test]

    H_Incep = model_Incep.fit( x=aug.flow(x_train_Incep, y_train_Incep, batch_size=BS),
	                                            validation_data=(x_test_Incep, y_test_Incep),
	                                            steps_per_epoch=len(x_train_Incep) // BS,
	                                            epochs=100,  callbacks = [callback,checkpoint,reduce_lr])

    pred_Incep = model_Incep.predict(x_test_Incep)
    
    oos_y.append(y_test_Incep)
    oos_pred.append(pred_Incep)  
    print(classification_report(y_test_Incep.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        pred_Incep.argmax(axis=1),  
                                        target_names=le_Incep.classes_))
    
oos_y = np.concatenate(oos_y)
oos_pred = np.concatenate(oos_pred)
print(classification_report(oos_y.argmax(axis=1), oos_pred.argmax(axis=1),  target_names=le_Incep.classes_))

87924736/87910968 [==============================] - 1s 0us/step
Fold #1
Epoch 1/100
45/45 [==============================] - ETA: 0s - loss: 0.7931 - accuracy: 0.3139
Epoch 1: val_accuracy improved from -inf to 0.33333, saving model to  Kmodel8_soft_pre_bi_sgd1.h5 
INFO:tensorflow:Assets written to:  Kmodel8_soft_pre_bi_sgd1.h5 /assets
45/45 [==============================] - 65s 1s/step - loss: 0.7931 - accuracy: 0.3139 - val_loss: 0.8084 - val_accuracy: 0.3333 - lr: 1.0000e-04
Epoch 2/100
45/45 [==============================] - ETA: 0s - loss: 0.7607 - accuracy: 0.3083
Epoch 2: val_accuracy improved from 0.33333 to 0.34444, saving model to  Kmodel8_soft_pre_bi_sgd1.h5 
INFO:tensorflow:Assets written to:  Kmodel8_soft_pre_bi_sgd1.h5 /assets
45/45 [==============================] - 43s 960ms/step - loss: 0.7607 - accuracy: 0.3083 - val_loss: 0.7777 - val_accuracy: 0.3444 - lr: 1.0000e-04
Epoch 3/100
45/45 [==============================] - ETA: 0s - loss: 0.7344 - accuracy: 0.3278
Ep

#### 8 soft pre cate 


In [ ]:
BS = 8
filepath = "Kmodel8_soft_pre_cate.h5"                  ##

train_img_Incep = []
for ImageName in tqdm(data_train3['ImageName']):              
    image_path = All_3class_dir +'/' + ImageName                  
    img = cv2.imread(image_path)                                          
    img = cv2.resize(img, (299, 299))                                      
    train_img_Incep.append(img)
data_Incep = np.array(train_img_Incep, dtype="float32") / 255.0

labels_Incep = np.array(data_train3[['NameType']] )        
labels_Incep.shape = (len(labels_Incep),) 
le_Incep = LabelEncoder()                                                                     
labels_Incep = le_Incep.fit_transform(labels_Incep)                                         
labels_Incep = to_categorical(labels_Incep, num_classes=3)                   

aug = ImageDataGenerator( rotation_range=15,	width_shift_range=0.1, 	height_shift_range=0.1,
		shear_range=0.15, 	horizontal_flip=True, 	fill_mode="nearest")
                                                                                                           ## 
callback = EarlyStopping( monitor= "val_accuracy",    patience=15,       mode="max")

# checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1,   save_best_only=True, mode='max')
                                                                                                            ##
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5,   patience=6,     verbose=1, mode='max', min_lr=0.00001)

100%|██████████| 450/450 [00:04<00:00, 90.93it/s] 


In [ ]:
base_model_Incep = tf.keras.applications.InceptionV3(input_shape=(299, 299, 3), include_top=False, weights="imagenet")
base_model_Incep.trainable = False


#Add custom head
average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model_Incep.output)
prediction_layer = tf.keras.layers.Dense(units=3, activation="softmax")(average_pooling_layer)   # unit 3
model_Incep = tf.keras.models.Model(inputs=base_model_Incep.input, outputs=prediction_layer)

model_Incep.compile(optimizer=RMSprop(learning_rate=0.0001), loss="categorical_crossentropy", metrics=["accuracy"])


### K-fold
kf_Incep = KFold(5, shuffle=True, random_state=40) 

oos_y = []
oos_pred = []
fold = 0

for train, test in kf_Incep.split(data_Incep):
    fold+=1   
    print(f"Fold #{fold}")

    filename = f" Kmodel8_soft_pre_cate{fold}.h5 "
    checkpoint = ModelCheckpoint(filename, monitor='val_accuracy', verbose=1,   save_best_only=True, mode='max')
        
    x_train_Incep = data_Incep[train]
    y_train_Incep = labels_Incep[train]
    x_test_Incep = data_Incep[test]
    y_test_Incep = labels_Incep[test]

    H_Incep = model_Incep.fit( x=aug.flow(x_train_Incep, y_train_Incep, batch_size=BS),
	                                            validation_data=(x_test_Incep, y_test_Incep),
	                                            steps_per_epoch=len(x_train_Incep) // BS,
	                                            epochs=100,  callbacks = [callback,checkpoint,reduce_lr])

    pred_Incep = model_Incep.predict(x_test_Incep)
    
    oos_y.append(y_test_Incep)
    oos_pred.append(pred_Incep)  
    print(classification_report(y_test_Incep.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        pred_Incep.argmax(axis=1),  
                                        target_names=le_Incep.classes_))
    
oos_y = np.concatenate(oos_y)
oos_pred = np.concatenate(oos_pred)
print(classification_report(oos_y.argmax(axis=1), oos_pred.argmax(axis=1),  target_names=le_Incep.classes_))

Fold #1
Epoch 1/100
45/45 [==============================] - ETA: 0s - loss: 1.0992 - accuracy: 0.3639
Epoch 1: val_accuracy improved from -inf to 0.51111, saving model to  Kmodel8_soft_pre_cate1.h5 
INFO:tensorflow:Assets written to:  Kmodel8_soft_pre_cate1.h5 /assets
45/45 [==============================] - 50s 1s/step - loss: 1.0992 - accuracy: 0.3639 - val_loss: 1.0315 - val_accuracy: 0.5111 - lr: 1.0000e-04
Epoch 2/100
45/45 [==============================] - ETA: 0s - loss: 1.0501 - accuracy: 0.4278
Epoch 2: val_accuracy improved from 0.51111 to 0.53333, saving model to  Kmodel8_soft_pre_cate1.h5 
INFO:tensorflow:Assets written to:  Kmodel8_soft_pre_cate1.h5 /assets
45/45 [==============================] - 44s 990ms/step - loss: 1.0501 - accuracy: 0.4278 - val_loss: 1.0016 - val_accuracy: 0.5333 - lr: 1.0000e-04
Epoch 3/100
45/45 [==============================] - ETA: 0s - loss: 1.0124 - accuracy: 0.4972
Epoch 3: val_accuracy improved from 0.53333 to 0.55556, saving model to  Km

#### 8 soft pre cate adam


In [ ]:
BS = 8
filepath = "Kmodel8_soft_pre_cate_adam.h5"                  ##

train_img_Incep = []
for ImageName in tqdm(data_train3['ImageName']):              
    image_path = All_3class_dir +'/' + ImageName                  
    img = cv2.imread(image_path)                                          
    img = cv2.resize(img, (299, 299))                                      
    train_img_Incep.append(img)
data_Incep = np.array(train_img_Incep, dtype="float32") / 255.0

labels_Incep = np.array(data_train3[['NameType']] )        
labels_Incep.shape = (len(labels_Incep),) 
le_Incep = LabelEncoder()                                                                     
labels_Incep = le_Incep.fit_transform(labels_Incep)                                         
labels_Incep = to_categorical(labels_Incep, num_classes=3)                   

aug = ImageDataGenerator( rotation_range=15,	width_shift_range=0.1, 	height_shift_range=0.1,
		shear_range=0.15, 	horizontal_flip=True, 	fill_mode="nearest")
                                                                                                           ## 
callback = EarlyStopping( monitor= "val_accuracy",    patience=15,       mode="max")

# checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1,   save_best_only=True, mode='max')
                                                                                                            ##
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5,   patience=6,     verbose=1, mode='max', min_lr=0.00001)

100%|██████████| 450/450 [01:53<00:00,  3.98it/s]


In [ ]:
base_model_Incep = tf.keras.applications.InceptionV3(input_shape=(299, 299, 3), include_top=False, weights="imagenet")
base_model_Incep.trainable = False


#Add custom head
average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model_Incep.output)
prediction_layer = tf.keras.layers.Dense(units=3, activation="softmax")(average_pooling_layer)   # unit 3
model_Incep = tf.keras.models.Model(inputs=base_model_Incep.input, outputs=prediction_layer)

model_Incep.compile(optimizer=Adam(learning_rate=0.0001), loss="categorical_crossentropy", metrics=["accuracy"])

### K-fold
kf_Incep = KFold(5, shuffle=True, random_state=40) 

oos_y = []
oos_pred = []
fold = 0

for train, test in kf_Incep.split(data_Incep):
    fold+=1   
    print(f"Fold #{fold}")

    filename = f" Kmodel8_soft_pre_cate_adam{fold}.h5 "
    checkpoint = ModelCheckpoint(filename, monitor='val_accuracy', verbose=1,   save_best_only=True, mode='max')
        
    x_train_Incep = data_Incep[train]
    y_train_Incep = labels_Incep[train]
    x_test_Incep = data_Incep[test]
    y_test_Incep = labels_Incep[test]

    H_Incep = model_Incep.fit( x=aug.flow(x_train_Incep, y_train_Incep, batch_size=BS),
	                                            validation_data=(x_test_Incep, y_test_Incep),
	                                            steps_per_epoch=len(x_train_Incep) // BS,
	                                            epochs=100,  callbacks = [callback,checkpoint,reduce_lr])

    pred_Incep = model_Incep.predict(x_test_Incep)
    
    oos_y.append(y_test_Incep)
    oos_pred.append(pred_Incep)  
    print(classification_report(y_test_Incep.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        pred_Incep.argmax(axis=1),  
                                        target_names=le_Incep.classes_))
    
oos_y = np.concatenate(oos_y)
oos_pred = np.concatenate(oos_pred)
print(classification_report(oos_y.argmax(axis=1), oos_pred.argmax(axis=1),  target_names=le_Incep.classes_))

87924736/87910968 [==============================] - 0s 0us/step
Fold #1
Epoch 1/100
45/45 [==============================] - ETA: 0s - loss: 1.1706 - accuracy: 0.2722
Epoch 1: val_accuracy improved from -inf to 0.33333, saving model to  Kmodel8_soft_pre_cate_adam1.h5 
INFO:tensorflow:Assets written to:  Kmodel8_soft_pre_cate_adam1.h5 /assets
45/45 [==============================] - 51s 808ms/step - loss: 1.1706 - accuracy: 0.2722 - val_loss: 1.1290 - val_accuracy: 0.3333 - lr: 1.0000e-04
Epoch 2/100
45/45 [==============================] - ETA: 0s - loss: 1.1212 - accuracy: 0.3250
Epoch 2: val_accuracy improved from 0.33333 to 0.41111, saving model to  Kmodel8_soft_pre_cate_adam1.h5 
INFO:tensorflow:Assets written to:  Kmodel8_soft_pre_cate_adam1.h5 /assets
45/45 [==============================] - 33s 737ms/step - loss: 1.1212 - accuracy: 0.3250 - val_loss: 1.0726 - val_accuracy: 0.4111 - lr: 1.0000e-04
Epoch 3/100
45/45 [==============================] - ETA: 0s - loss: 1.0692 - accu

#### 8 soft pre cate sgd


In [ ]:
BS = 8
filepath = "Kmodel8_soft_pre_cate_sgd.h5"                  ##

train_img_Incep = []
for ImageName in tqdm(data_train3['ImageName']):              
    image_path = All_3class_dir +'/' + ImageName                  
    img = cv2.imread(image_path)                                          
    img = cv2.resize(img, (299, 299))                                      
    train_img_Incep.append(img)
data_Incep = np.array(train_img_Incep, dtype="float32") / 255.0

labels_Incep = np.array(data_train3[['NameType']] )        
labels_Incep.shape = (len(labels_Incep),) 
le_Incep = LabelEncoder()                                                                     
labels_Incep = le_Incep.fit_transform(labels_Incep)                                         
labels_Incep = to_categorical(labels_Incep, num_classes=3)                   

aug = ImageDataGenerator( rotation_range=15,	width_shift_range=0.1, 	height_shift_range=0.1,
		shear_range=0.15, 	horizontal_flip=True, 	fill_mode="nearest")
                                                                                                           ## 
callback = EarlyStopping( monitor= "val_accuracy",    patience=15,       mode="max")

# checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1,   save_best_only=True, mode='max')
                                                                                                            ##
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5,   patience=6,     verbose=1, mode='max', min_lr=0.00001)

100%|██████████| 450/450 [04:15<00:00,  1.76it/s]


In [ ]:
base_model_Incep = tf.keras.applications.InceptionV3(input_shape=(299, 299, 3), include_top=False, weights="imagenet")
base_model_Incep.trainable = False

#Add custom head
average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model_Incep.output)
prediction_layer = tf.keras.layers.Dense(units=3, activation="softmax")(average_pooling_layer)   # unit 3
model_Incep = tf.keras.models.Model(inputs=base_model_Incep.input, outputs=prediction_layer)

model_Incep.compile(optimizer=SGD(learning_rate=0.0001), loss="categorical_crossentropy", metrics=["accuracy"])

### K-fold
kf_Incep = KFold(5, shuffle=True, random_state=40) 

oos_y = []
oos_pred = []
fold = 0

for train, test in kf_Incep.split(data_Incep):
    fold+=1   
    print(f"Fold #{fold}")

    filename = f" Kmodel8_soft_pre_cate_sgd{fold}.h5 "
    checkpoint = ModelCheckpoint(filename, monitor='val_accuracy', verbose=1,   save_best_only=True, mode='max')
        
    x_train_Incep = data_Incep[train]
    y_train_Incep = labels_Incep[train]
    x_test_Incep = data_Incep[test]
    y_test_Incep = labels_Incep[test]

    H_Incep = model_Incep.fit( x=aug.flow(x_train_Incep, y_train_Incep, batch_size=BS),
	                                            validation_data=(x_test_Incep, y_test_Incep),
	                                            steps_per_epoch=len(x_train_Incep) // BS,
	                                            epochs=100,  callbacks = [callback,checkpoint,reduce_lr])

    pred_Incep = model_Incep.predict(x_test_Incep)
    
    oos_y.append(y_test_Incep)
    oos_pred.append(pred_Incep)  
    print(classification_report(y_test_Incep.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        pred_Incep.argmax(axis=1),  
                                        target_names=le_Incep.classes_))
    
oos_y = np.concatenate(oos_y)
oos_pred = np.concatenate(oos_pred)
print(classification_report(oos_y.argmax(axis=1), oos_pred.argmax(axis=1),  target_names=le_Incep.classes_))

87924736/87910968 [==============================] - 3s 0us/step
Fold #1
Epoch 1/100
45/45 [==============================] - ETA: 0s - loss: 1.2973 - accuracy: 0.3306
Epoch 1: val_accuracy improved from -inf to 0.28889, saving model to  Kmodel8_soft_pre_cate_sgd1.h5 
INFO:tensorflow:Assets written to:  Kmodel8_soft_pre_cate_sgd1.h5 /assets
45/45 [==============================] - 65s 1s/step - loss: 1.2973 - accuracy: 0.3306 - val_loss: 1.2078 - val_accuracy: 0.2889 - lr: 1.0000e-04
Epoch 2/100
45/45 [==============================] - ETA: 0s - loss: 1.1963 - accuracy: 0.3444
Epoch 2: val_accuracy did not improve from 0.28889
45/45 [==============================] - 10s 230ms/step - loss: 1.1963 - accuracy: 0.3444 - val_loss: 1.1779 - val_accuracy: 0.2556 - lr: 1.0000e-04
Epoch 3/100
45/45 [==============================] - ETA: 0s - loss: 1.1638 - accuracy: 0.2972
Epoch 3: val_accuracy did not improve from 0.28889
45/45 [==============================] - 10s 229ms/step - loss: 1.1638

### 16 


#### 16  soft pre bi


In [ ]:
BS = 16
filepath = "Kmodel16_soft_pre_bi.h5"                  ##

train_img_Incep = []
for ImageName in tqdm(data_train3['ImageName']):              
    image_path = All_3class_dir +'/' + ImageName                  
    img = cv2.imread(image_path)                                          
    img = cv2.resize(img, (299, 299))                                      
    train_img_Incep.append(img)
data_Incep = np.array(train_img_Incep, dtype="float32") / 255.0

labels_Incep = np.array(data_train3[['NameType']] )        
labels_Incep.shape = (len(labels_Incep),) 
le_Incep = LabelEncoder()                                                                     
labels_Incep = le_Incep.fit_transform(labels_Incep)                                         
labels_Incep = to_categorical(labels_Incep, num_classes=3)                   

aug = ImageDataGenerator( rotation_range=15,	width_shift_range=0.1, 	height_shift_range=0.1,
		shear_range=0.15, 	horizontal_flip=True, 	fill_mode="nearest")
                                                                                                           ## 
callback = EarlyStopping( monitor= "val_accuracy",    patience=15,       mode="max")

# checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1,   save_best_only=True, mode='max')
                                                                                                            ##
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5,   patience=6,     verbose=1, mode='max', min_lr=0.00001)

100%|██████████| 450/450 [00:07<00:00, 63.37it/s] 


In [ ]:
base_model_Incep = tf.keras.applications.InceptionV3(input_shape=(299, 299, 3), include_top=False, weights="imagenet")
base_model_Incep.trainable = False


#Add custom head
average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model_Incep.output)
prediction_layer = tf.keras.layers.Dense(units=3, activation="softmax")(average_pooling_layer)   # unit 3
model_Incep = tf.keras.models.Model(inputs=base_model_Incep.input, outputs=prediction_layer)

model_Incep.compile(optimizer=RMSprop(learning_rate=0.0001), loss="binary_crossentropy", metrics=["accuracy"])


### K-fold
kf_Incep = KFold(5, shuffle=True, random_state=40) 

oos_y = []
oos_pred = []
fold = 0

for train, test in kf_Incep.split(data_Incep):
    fold+=1   
    print(f"Fold #{fold}")

    filename = f" Kmodel16_soft_pre_bi{fold}.h5 "
    checkpoint = ModelCheckpoint(filename, monitor='val_accuracy', verbose=1,   save_best_only=True, mode='max')
        
    x_train_Incep = data_Incep[train]
    y_train_Incep = labels_Incep[train]
    x_test_Incep = data_Incep[test]
    y_test_Incep = labels_Incep[test]

    H_Incep = model_Incep.fit( x=aug.flow(x_train_Incep, y_train_Incep, batch_size=BS),
	                                            validation_data=(x_test_Incep, y_test_Incep),
	                                            steps_per_epoch=len(x_train_Incep) // BS,
	                                            epochs=100,  callbacks = [callback,checkpoint,reduce_lr])

    pred_Incep = model_Incep.predict(x_test_Incep)
    
    oos_y.append(y_test_Incep)
    oos_pred.append(pred_Incep)  
    print(classification_report(y_test_Incep.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        pred_Incep.argmax(axis=1),  
                                        target_names=le_Incep.classes_))
    
oos_y = np.concatenate(oos_y)
oos_pred = np.concatenate(oos_pred)
print(classification_report(oos_y.argmax(axis=1), oos_pred.argmax(axis=1),  target_names=le_Incep.classes_))

Fold #1
Epoch 1/100
22/22 [==============================] - ETA: 0s - loss: 0.6686 - accuracy: 0.3488
Epoch 1: val_accuracy improved from -inf to 0.25556, saving model to  Kmodel16_soft_pre_bi1.h5 
INFO:tensorflow:Assets written to:  Kmodel16_soft_pre_bi1.h5 /assets
22/22 [==============================] - 72s 3s/step - loss: 0.6686 - accuracy: 0.3488 - val_loss: 0.6456 - val_accuracy: 0.2556 - lr: 1.0000e-04
Epoch 2/100
22/22 [==============================] - ETA: 0s - loss: 0.6365 - accuracy: 0.3866
Epoch 2: val_accuracy improved from 0.25556 to 0.31111, saving model to  Kmodel16_soft_pre_bi1.h5 
INFO:tensorflow:Assets written to:  Kmodel16_soft_pre_bi1.h5 /assets
22/22 [==============================] - 42s 2s/step - loss: 0.6365 - accuracy: 0.3866 - val_loss: 0.6350 - val_accuracy: 0.3111 - lr: 1.0000e-04
Epoch 3/100
22/22 [==============================] - ETA: 0s - loss: 0.6354 - accuracy: 0.3551
Epoch 3: val_accuracy improved from 0.31111 to 0.33333, saving model to  Kmodel16_

#### 16 soft pre bi adam  


In [ ]:
BS = 16
filepath = "Kmodel16_soft_pre_bi_adam.h5"                  ##

train_img_Incep = []
for ImageName in tqdm(data_train3['ImageName']):              
    image_path = All_3class_dir +'/' + ImageName                  
    img = cv2.imread(image_path)                                          
    img = cv2.resize(img, (299, 299))                                      
    train_img_Incep.append(img)
data_Incep = np.array(train_img_Incep, dtype="float32") / 255.0

labels_Incep = np.array(data_train3[['NameType']] )        
labels_Incep.shape = (len(labels_Incep),) 
le_Incep = LabelEncoder()                                                                     
labels_Incep = le_Incep.fit_transform(labels_Incep)                                         
labels_Incep = to_categorical(labels_Incep, num_classes=3)                   

aug = ImageDataGenerator( rotation_range=15,	width_shift_range=0.1, 	height_shift_range=0.1,
		shear_range=0.15, 	horizontal_flip=True, 	fill_mode="nearest")
                                                                                                           ## 
callback = EarlyStopping( monitor= "val_accuracy",    patience=15,       mode="max")

# checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1,   save_best_only=True, mode='max')
                                                                                                            ##
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5,   patience=6,     verbose=1, mode='max', min_lr=0.00001)

100%|██████████| 450/450 [01:50<00:00,  4.06it/s]


In [ ]:
base_model_Incep = tf.keras.applications.InceptionV3(input_shape=(299, 299, 3), include_top=False, weights="imagenet")
base_model_Incep.trainable = False

#Add custom head
average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model_Incep.output)
prediction_layer = tf.keras.layers.Dense(units=3, activation="softmax")(average_pooling_layer)   # unit 3
model_Incep = tf.keras.models.Model(inputs=base_model_Incep.input, outputs=prediction_layer)

model_Incep.compile(optimizer=Adam(learning_rate=0.0001), loss="binary_crossentropy", metrics=["accuracy"])


### K-fold
kf_Incep = KFold(5, shuffle=True, random_state=40) 

oos_y = []
oos_pred = []
fold = 0

for train, test in kf_Incep.split(data_Incep):
    fold+=1   
    print(f"Fold #{fold}")

    filename = f" Kmodel16_soft_pre_bi_adam{fold}.h5 "
    checkpoint = ModelCheckpoint(filename, monitor='val_accuracy', verbose=1,   save_best_only=True, mode='max')
        
    x_train_Incep = data_Incep[train]
    y_train_Incep = labels_Incep[train]
    x_test_Incep = data_Incep[test]
    y_test_Incep = labels_Incep[test]

    H_Incep = model_Incep.fit( x=aug.flow(x_train_Incep, y_train_Incep, batch_size=BS),
	                                            validation_data=(x_test_Incep, y_test_Incep),
	                                            steps_per_epoch=len(x_train_Incep) // BS,
	                                            epochs=100,  callbacks = [callback,checkpoint,reduce_lr])

    pred_Incep = model_Incep.predict(x_test_Incep)
    
    oos_y.append(y_test_Incep)
    oos_pred.append(pred_Incep)  
    print(classification_report(y_test_Incep.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        pred_Incep.argmax(axis=1),  
                                        target_names=le_Incep.classes_))
    
oos_y = np.concatenate(oos_y)
oos_pred = np.concatenate(oos_pred)
print(classification_report(oos_y.argmax(axis=1), oos_pred.argmax(axis=1),  target_names=le_Incep.classes_))

87924736/87910968 [==============================] - 1s 0us/step
Fold #1
Epoch 1/100
22/22 [==============================] - ETA: 0s - loss: 0.6886 - accuracy: 0.3576
Epoch 1: val_accuracy improved from -inf to 0.43333, saving model to  Kmodel16_soft_pre_bi_adam1.h5 
INFO:tensorflow:Assets written to:  Kmodel16_soft_pre_bi_adam1.h5 /assets
22/22 [==============================] - 68s 2s/step - loss: 0.6886 - accuracy: 0.3576 - val_loss: 0.6375 - val_accuracy: 0.4333 - lr: 1.0000e-04
Epoch 2/100
22/22 [==============================] - ETA: 0s - loss: 0.6324 - accuracy: 0.4070
Epoch 2: val_accuracy did not improve from 0.43333
22/22 [==============================] - 9s 396ms/step - loss: 0.6324 - accuracy: 0.4070 - val_loss: 0.6322 - val_accuracy: 0.3778 - lr: 1.0000e-04
Epoch 3/100
22/22 [==============================] - ETA: 0s - loss: 0.6199 - accuracy: 0.4448
Epoch 3: val_accuracy did not improve from 0.43333
22/22 [==============================] - 9s 394ms/step - loss: 0.6199 -

#### 16 soft pre bi sgd  


In [ ]:
BS = 16
filepath = "Kmodel16_soft_pre_bi_sgd.h5"                  ##

train_img_Incep = []
for ImageName in tqdm(data_train3['ImageName']):              
    image_path = All_3class_dir +'/' + ImageName                  
    img = cv2.imread(image_path)                                          
    img = cv2.resize(img, (299, 299))                                      
    train_img_Incep.append(img)
data_Incep = np.array(train_img_Incep, dtype="float32") / 255.0

labels_Incep = np.array(data_train3[['NameType']] )        
labels_Incep.shape = (len(labels_Incep),) 
le_Incep = LabelEncoder()                                                                     
labels_Incep = le_Incep.fit_transform(labels_Incep)                                         
labels_Incep = to_categorical(labels_Incep, num_classes=3)                   

aug = ImageDataGenerator( rotation_range=15,	width_shift_range=0.1, 	height_shift_range=0.1,
		shear_range=0.15, 	horizontal_flip=True, 	fill_mode="nearest")
                                                                                                           ## 
callback = EarlyStopping( monitor= "val_accuracy",    patience=15,       mode="max")

# checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1,   save_best_only=True, mode='max')
                                                                                                            ##
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5,   patience=6,     verbose=1, mode='max', min_lr=0.00001)

100%|██████████| 450/450 [00:06<00:00, 72.48it/s]


In [ ]:
base_model_Incep = tf.keras.applications.InceptionV3(input_shape=(299, 299, 3), include_top=False, weights="imagenet")
base_model_Incep.trainable = False

#Add custom head
average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model_Incep.output)
prediction_layer = tf.keras.layers.Dense(units=3, activation="softmax")(average_pooling_layer)   # unit 3
model_Incep = tf.keras.models.Model(inputs=base_model_Incep.input, outputs=prediction_layer)

model_Incep.compile(optimizer=SGD(learning_rate=0.0001), loss="binary_crossentropy", metrics=["accuracy"])


### K-fold
kf_Incep = KFold(5, shuffle=True, random_state=40) 

oos_y = []
oos_pred = []
fold = 0

for train, test in kf_Incep.split(data_Incep):
    fold+=1   
    print(f"Fold #{fold}")

    filename = f" Kmodel16_soft_pre_bi_sgd{fold}.h5 "
    checkpoint = ModelCheckpoint(filename, monitor='val_accuracy', verbose=1,   save_best_only=True, mode='max')
        
    x_train_Incep = data_Incep[train]
    y_train_Incep = labels_Incep[train]
    x_test_Incep = data_Incep[test]
    y_test_Incep = labels_Incep[test]

    H_Incep = model_Incep.fit( x=aug.flow(x_train_Incep, y_train_Incep, batch_size=BS),
	                                            validation_data=(x_test_Incep, y_test_Incep),
	                                            steps_per_epoch=len(x_train_Incep) // BS,
	                                            epochs=100,  callbacks = [callback,checkpoint,reduce_lr])

    pred_Incep = model_Incep.predict(x_test_Incep)
    
    oos_y.append(y_test_Incep)
    oos_pred.append(pred_Incep)  
    print(classification_report(y_test_Incep.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        pred_Incep.argmax(axis=1),  
                                        target_names=le_Incep.classes_))
    
oos_y = np.concatenate(oos_y)
oos_pred = np.concatenate(oos_pred)
print(classification_report(oos_y.argmax(axis=1), oos_pred.argmax(axis=1),  target_names=le_Incep.classes_))

Fold #1
Epoch 1/100
22/22 [==============================] - ETA: 0s - loss: 0.8749 - accuracy: 0.3401
Epoch 1: val_accuracy improved from -inf to 0.34444, saving model to  Kmodel16_soft_pre_bi_sgd1.h5 
INFO:tensorflow:Assets written to:  Kmodel16_soft_pre_bi_sgd1.h5 /assets
22/22 [==============================] - 52s 2s/step - loss: 0.8749 - accuracy: 0.3401 - val_loss: 0.8570 - val_accuracy: 0.3444 - lr: 1.0000e-04
Epoch 2/100
22/22 [==============================] - ETA: 0s - loss: 0.8562 - accuracy: 0.3314
Epoch 2: val_accuracy did not improve from 0.34444
22/22 [==============================] - 9s 405ms/step - loss: 0.8562 - accuracy: 0.3314 - val_loss: 0.8305 - val_accuracy: 0.3444 - lr: 1.0000e-04
Epoch 3/100
22/22 [==============================] - ETA: 0s - loss: 0.8303 - accuracy: 0.3285
Epoch 3: val_accuracy did not improve from 0.34444
22/22 [==============================] - 9s 400ms/step - loss: 0.8303 - accuracy: 0.3285 - val_loss: 0.8069 - val_accuracy: 0.3444 - lr: 1

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

  Lungcancer       0.00      0.00      0.00        28
      Normal       0.35      1.00      0.52        31
Tuberculosis       0.00      0.00      0.00        31

    accuracy                           0.34        90
   macro avg       0.12      0.33      0.17        90
weighted avg       0.12      0.34      0.18        90

Fold #2
Epoch 1/100
22/22 [==============================] - ETA: 0s - loss: 0.6942 - accuracy: 0.3580
Epoch 1: val_accuracy improved from -inf to 0.31111, saving model to  Kmodel16_soft_pre_bi_sgd2.h5 
INFO:tensorflow:Assets written to:  Kmodel16_soft_pre_bi_sgd2.h5 /assets
22/22 [==============================] - 45s 2s/step - loss: 0.6942 - accuracy: 0.3580 - val_loss: 0.7111 - val_accuracy: 0.3111 - lr: 2.5000e-05
Epoch 2/100
22/22 [==============================] - ETA: 0s - loss: 0.6914 - accuracy: 0.3285
Epoch 2: val_accuracy did not improve from 0.31111
22/22 [==============================] - 9s 415ms/s

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

  Lungcancer       1.00      0.04      0.07        27
      Normal       0.37      1.00      0.54        33
Tuberculosis       0.00      0.00      0.00        30

    accuracy                           0.38        90
   macro avg       0.46      0.35      0.20        90
weighted avg       0.44      0.38      0.22        90

Fold #4
Epoch 1/100
22/22 [==============================] - ETA: 0s - loss: 0.6695 - accuracy: 0.3634
Epoch 1: val_accuracy improved from -inf to 0.35556, saving model to  Kmodel16_soft_pre_bi_sgd4.h5 
INFO:tensorflow:Assets written to:  Kmodel16_soft_pre_bi_sgd4.h5 /assets
22/22 [==============================] - 44s 2s/step - loss: 0.6695 - accuracy: 0.3634 - val_loss: 0.6647 - val_accuracy: 0.3556 - lr: 1.0000e-05
Epoch 2/100
22/22 [==============================] - ETA: 0s - loss: 0.6628 - accuracy: 0.3459
Epoch 2: val_accuracy did not improve from 0.35556
22/22 [==============================] - 9s 392ms/s

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

  Lungcancer       0.00      0.00      0.00        30
      Normal       0.35      0.97      0.52        31
Tuberculosis       0.40      0.07      0.12        29

    accuracy                           0.36        90
   macro avg       0.25      0.35      0.21        90
weighted avg       0.25      0.36      0.22        90

Fold #5
Epoch 1/100
22/22 [==============================] - ETA: 0s - loss: 0.6584 - accuracy: 0.3721
Epoch 1: val_accuracy improved from -inf to 0.32222, saving model to  Kmodel16_soft_pre_bi_sgd5.h5 
INFO:tensorflow:Assets written to:  Kmodel16_soft_pre_bi_sgd5.h5 /assets
22/22 [==============================] - 44s 2s/step - loss: 0.6584 - accuracy: 0.3721 - val_loss: 0.6760 - val_accuracy: 0.3222 - lr: 1.0000e-05
Epoch 2/100
22/22 [==============================] - ETA: 0s - loss: 0.6530 - accuracy: 0.3866
Epoch 2: val_accuracy did not improve from 0.32222
22/22 [==============================] - 9s 398ms/s

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#### 16 soft pre cate 


In [ ]:
BS = 16
filepath = "Kmodel16_soft_pre_cate.h5"                  ##

train_img_Incep = []
for ImageName in tqdm(data_train3['ImageName']):              
    image_path = All_3class_dir +'/' + ImageName                  
    img = cv2.imread(image_path)                                          
    img = cv2.resize(img, (299, 299))                                      
    train_img_Incep.append(img)
data_Incep = np.array(train_img_Incep, dtype="float32") / 255.0

labels_Incep = np.array(data_train3[['NameType']] )        
labels_Incep.shape = (len(labels_Incep),) 
le_Incep = LabelEncoder()                                                                     
labels_Incep = le_Incep.fit_transform(labels_Incep)                                         
labels_Incep = to_categorical(labels_Incep, num_classes=3)                   

aug = ImageDataGenerator( rotation_range=15,	width_shift_range=0.1, 	height_shift_range=0.1,
		shear_range=0.15, 	horizontal_flip=True, 	fill_mode="nearest")
                                                                                                           ## 
callback = EarlyStopping( monitor= "val_accuracy",    patience=15,       mode="max" )

# checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1,   save_best_only=True, mode='max')
                                                                                                            ##
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5,   patience=6,     verbose=1, mode='max', min_lr=0.00001)

100%|██████████| 450/450 [00:06<00:00, 72.99it/s] 


In [ ]:
base_model_Incep = tf.keras.applications.InceptionV3(input_shape=(299, 299, 3), include_top=False, weights="imagenet")
base_model_Incep.trainable = False

#Add custom head
average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model_Incep.output)
prediction_layer = tf.keras.layers.Dense(units=3, activation="softmax")(average_pooling_layer)   # unit 3
model_Incep = tf.keras.models.Model(inputs=base_model_Incep.input, outputs=prediction_layer)

model_Incep.compile(optimizer=RMSprop(learning_rate=0.0001), loss="categorical_crossentropy", metrics=["accuracy"])


### K-fold
kf_Incep = KFold(5, shuffle=True, random_state=40) 

oos_y = []
oos_pred = []
fold = 0

for train, test in kf_Incep.split(data_Incep):
    fold+=1   
    print(f"Fold #{fold}")

    filename = f" Kmodel16_soft_pre_cate{fold}.h5 "
    checkpoint = ModelCheckpoint(filename, monitor='val_accuracy', verbose=1,   save_best_only=True, mode='max')
        
    x_train_Incep = data_Incep[train]
    y_train_Incep = labels_Incep[train]
    x_test_Incep = data_Incep[test]
    y_test_Incep = labels_Incep[test]

    H_Incep = model_Incep.fit( x=aug.flow(x_train_Incep, y_train_Incep, batch_size=BS),
	                                            validation_data=(x_test_Incep, y_test_Incep),
	                                            steps_per_epoch=len(x_train_Incep) // BS,
	                                            epochs=100,  callbacks = [callback,checkpoint,reduce_lr])

    pred_Incep = model_Incep.predict(x_test_Incep)
    
    oos_y.append(y_test_Incep)
    oos_pred.append(pred_Incep)  
    print(classification_report(y_test_Incep.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        pred_Incep.argmax(axis=1),  
                                        target_names=le_Incep.classes_))
    
oos_y = np.concatenate(oos_y)
oos_pred = np.concatenate(oos_pred)
print(classification_report(oos_y.argmax(axis=1), oos_pred.argmax(axis=1),  target_names=le_Incep.classes_))

Fold #1
Epoch 1/100
22/22 [==============================] - ETA: 0s - loss: 1.0570 - accuracy: 0.4302
Epoch 1: val_accuracy improved from -inf to 0.35556, saving model to  Kmodel16_soft_pre_cate1.h5 
INFO:tensorflow:Assets written to:  Kmodel16_soft_pre_cate1.h5 /assets
22/22 [==============================] - 52s 2s/step - loss: 1.0570 - accuracy: 0.4302 - val_loss: 1.1335 - val_accuracy: 0.3556 - lr: 1.0000e-04
Epoch 2/100
22/22 [==============================] - ETA: 0s - loss: 1.0495 - accuracy: 0.4855
Epoch 2: val_accuracy did not improve from 0.35556
22/22 [==============================] - 9s 394ms/step - loss: 1.0495 - accuracy: 0.4855 - val_loss: 1.1036 - val_accuracy: 0.3444 - lr: 1.0000e-04
Epoch 3/100
22/22 [==============================] - ETA: 0s - loss: 0.9883 - accuracy: 0.5465
Epoch 3: val_accuracy improved from 0.35556 to 0.42222, saving model to  Kmodel16_soft_pre_cate1.h5 
INFO:tensorflow:Assets written to:  Kmodel16_soft_pre_cate1.h5 /assets
22/22 [==============

####16 soft pre cate adam


In [ ]:
BS = 16                                                       ##
filepath = "Kmodel16_soft_pre_cate_adam.h5"                  ##

train_img_Incep = []
for ImageName in tqdm(data_train3['ImageName']):              
    image_path = All_3class_dir +'/' + ImageName                  
    img = cv2.imread(image_path)                                          
    img = cv2.resize(img, (299, 299))                                      
    train_img_Incep.append(img)
data_Incep = np.array(train_img_Incep, dtype="float32") / 255.0

labels_Incep = np.array(data_train3[['NameType']] )        
labels_Incep.shape = (len(labels_Incep),) 
le_Incep = LabelEncoder()                                                                     
labels_Incep = le_Incep.fit_transform(labels_Incep)                                         
labels_Incep = to_categorical(labels_Incep, num_classes=3)                   

aug = ImageDataGenerator( rotation_range=15,	width_shift_range=0.1, 	height_shift_range=0.1,
		shear_range=0.15, 	horizontal_flip=True, 	fill_mode="nearest")
                                                                                                           ## 
callback = EarlyStopping( monitor= "val_accuracy",    patience=15,       mode="max")

# checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1,   save_best_only=True, mode='max')
                                                                                                            ##
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5,   patience=6,     verbose=1, mode='max', min_lr=0.00001)

100%|██████████| 450/450 [00:06<00:00, 69.72it/s]


In [ ]:
base_model_Incep = tf.keras.applications.InceptionV3(input_shape=(299, 299, 3), include_top=False, weights="imagenet")
base_model_Incep.trainable = False

#Add custom head
average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model_Incep.output)
prediction_layer = tf.keras.layers.Dense(units=3, activation="softmax")(average_pooling_layer)   # unit 3
model_Incep = tf.keras.models.Model(inputs=base_model_Incep.input, outputs=prediction_layer)

model_Incep.compile(optimizer=Adam(learning_rate=0.0001), loss="categorical_crossentropy", metrics=["accuracy"])

### K-fold
kf_Incep = KFold(5, shuffle=True, random_state=40) 

oos_y = []
oos_pred = []
fold = 0

for train, test in kf_Incep.split(data_Incep):
    fold+=1   
    print(f"Fold #{fold}")

    filename = f" Kmodel16_soft_pre_cate_adam{fold}.h5 "
    checkpoint = ModelCheckpoint(filename, monitor='val_accuracy', verbose=1,   save_best_only=True, mode='max')
        
    x_train_Incep = data_Incep[train]
    y_train_Incep = labels_Incep[train]
    x_test_Incep = data_Incep[test]
    y_test_Incep = labels_Incep[test]

    H_Incep = model_Incep.fit( x=aug.flow(x_train_Incep, y_train_Incep, batch_size=BS),
	                                            validation_data=(x_test_Incep, y_test_Incep),
	                                            steps_per_epoch=len(x_train_Incep) // BS,
	                                            epochs=100,  callbacks = [callback,checkpoint,reduce_lr])

    pred_Incep = model_Incep.predict(x_test_Incep)
    
    oos_y.append(y_test_Incep)
    oos_pred.append(pred_Incep)  
    print(classification_report(y_test_Incep.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        pred_Incep.argmax(axis=1),  
                                        target_names=le_Incep.classes_))
    
oos_y = np.concatenate(oos_y)
oos_pred = np.concatenate(oos_pred)
print(classification_report(oos_y.argmax(axis=1), oos_pred.argmax(axis=1),  target_names=le_Incep.classes_))

Fold #1
Epoch 1/100
22/22 [==============================] - ETA: 0s - loss: 1.2154 - accuracy: 0.3576
Epoch 1: val_accuracy improved from -inf to 0.30000, saving model to  Kmodel16_soft_pre_cate_adam1.h5 
INFO:tensorflow:Assets written to:  Kmodel16_soft_pre_cate_adam1.h5 /assets
22/22 [==============================] - 52s 2s/step - loss: 1.2154 - accuracy: 0.3576 - val_loss: 1.1558 - val_accuracy: 0.3000 - lr: 1.0000e-04
Epoch 2/100
22/22 [==============================] - ETA: 0s - loss: 1.0885 - accuracy: 0.3983
Epoch 2: val_accuracy improved from 0.30000 to 0.37778, saving model to  Kmodel16_soft_pre_cate_adam1.h5 
INFO:tensorflow:Assets written to:  Kmodel16_soft_pre_cate_adam1.h5 /assets
22/22 [==============================] - 45s 2s/step - loss: 1.0885 - accuracy: 0.3983 - val_loss: 1.1208 - val_accuracy: 0.3778 - lr: 1.0000e-04
Epoch 3/100
22/22 [==============================] - ETA: 0s - loss: 1.0576 - accuracy: 0.4215
Epoch 3: val_accuracy improved from 0.37778 to 0.42222

#### 16 soft pre cate sgd


In [ ]:
BS = 16                                                       ##
filepath = "Kmodel16_soft_pre_cate_sgd.h5"                  ##

train_img_Incep = []
for ImageName in tqdm(data_train3['ImageName']):              
    image_path = All_3class_dir +'/' + ImageName                  
    img = cv2.imread(image_path)                                          
    img = cv2.resize(img, (299, 299))                                      
    train_img_Incep.append(img)
data_Incep = np.array(train_img_Incep, dtype="float32") / 255.0

labels_Incep = np.array(data_train3[['NameType']] )        
labels_Incep.shape = (len(labels_Incep),) 
le_Incep = LabelEncoder()                                                                     
labels_Incep = le_Incep.fit_transform(labels_Incep)                                         
labels_Incep = to_categorical(labels_Incep, num_classes=3)                   

aug = ImageDataGenerator( rotation_range=15,	width_shift_range=0.1, 	height_shift_range=0.1,
		shear_range=0.15, 	horizontal_flip=True, 	fill_mode="nearest")
                                                                                                           ## 
callback = EarlyStopping( monitor= "val_accuracy",    patience=15,       mode="max" )

# checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1,   save_best_only=True, mode='max')
                                                                                                            ##
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5,   patience=6,     verbose=1, mode='max', min_lr=0.00001)

100%|██████████| 450/450 [04:20<00:00,  1.73it/s]


In [ ]:
base_model_Incep = tf.keras.applications.InceptionV3(input_shape=(299, 299, 3), include_top=False, weights="imagenet")
base_model_Incep.trainable = False

#Add custom head
average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model_Incep.output)
prediction_layer = tf.keras.layers.Dense(units=3, activation="softmax")(average_pooling_layer)   # unit 3
model_Incep = tf.keras.models.Model(inputs=base_model_Incep.input, outputs=prediction_layer)

model_Incep.compile(optimizer=SGD(learning_rate=0.0001), loss="categorical_crossentropy", metrics=["accuracy"])

### K-fold
kf_Incep = KFold(5, shuffle=True, random_state=40) 

oos_y = []
oos_pred = []
fold = 0

for train, test in kf_Incep.split(data_Incep):
    fold+=1   
    print(f"Fold #{fold}")

    filename = f" Kmodel16_soft_pre_cate_sgd{fold}.h5 "
    checkpoint = ModelCheckpoint(filename, monitor='val_accuracy', verbose=1,   save_best_only=True, mode='max')
        
    x_train_Incep = data_Incep[train]
    y_train_Incep = labels_Incep[train]
    x_test_Incep = data_Incep[test]
    y_test_Incep = labels_Incep[test]

    H_Incep = model_Incep.fit( x=aug.flow(x_train_Incep, y_train_Incep, batch_size=BS),
	                                            validation_data=(x_test_Incep, y_test_Incep),
	                                            steps_per_epoch=len(x_train_Incep) // BS,
	                                            epochs=100,  callbacks = [callback,checkpoint,reduce_lr])

    pred_Incep = model_Incep.predict(x_test_Incep)
    
    oos_y.append(y_test_Incep)
    oos_pred.append(pred_Incep)  
    print(classification_report(y_test_Incep.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        pred_Incep.argmax(axis=1),  
                                        target_names=le_Incep.classes_))
    
oos_y = np.concatenate(oos_y)
oos_pred = np.concatenate(oos_pred)
print(classification_report(oos_y.argmax(axis=1), oos_pred.argmax(axis=1),  target_names=le_Incep.classes_))

87924736/87910968 [==============================] - 3s 0us/step
Fold #1
Epoch 1/100
22/22 [==============================] - ETA: 0s - loss: 1.3062 - accuracy: 0.2267
Epoch 1: val_accuracy improved from -inf to 0.24444, saving model to  Kmodel16_soft_pre_cate_sgd1.h5 
INFO:tensorflow:Assets written to:  Kmodel16_soft_pre_cate_sgd1.h5 /assets
22/22 [==============================] - 65s 2s/step - loss: 1.3062 - accuracy: 0.2267 - val_loss: 1.3134 - val_accuracy: 0.2444 - lr: 1.0000e-04
Epoch 2/100
22/22 [==============================] - ETA: 0s - loss: 1.2752 - accuracy: 0.2500
Epoch 2: val_accuracy did not improve from 0.24444
22/22 [==============================] - 9s 399ms/step - loss: 1.2752 - accuracy: 0.2500 - val_loss: 1.2786 - val_accuracy: 0.2444 - lr: 1.0000e-04
Epoch 3/100
22/22 [==============================] - ETA: 0s - loss: 1.2532 - accuracy: 0.2074
Epoch 3: val_accuracy improved from 0.24444 to 0.25556, saving model to  Kmodel16_soft_pre_cate_sgd1.h5 
INFO:tensorflow

### 32  

#### 32  soft pre bi


In [ ]:
BS = 32
filepath = "Kmodel32_soft_pre_bi.h5"                  ##

train_img_Incep = []
for ImageName in tqdm(data_train3['ImageName']):              
    image_path = All_3class_dir +'/' + ImageName                  
    img = cv2.imread(image_path)                                          
    img = cv2.resize(img, (299, 299))                                      
    train_img_Incep.append(img)
data_Incep = np.array(train_img_Incep, dtype="float32") / 255.0

labels_Incep = np.array(data_train3[['NameType']] )        
labels_Incep.shape = (len(labels_Incep),) 
le_Incep = LabelEncoder()                                                                     
labels_Incep = le_Incep.fit_transform(labels_Incep)                                         
labels_Incep = to_categorical(labels_Incep, num_classes=3)                   

aug = ImageDataGenerator( rotation_range=15,	width_shift_range=0.1, 	height_shift_range=0.1,
		shear_range=0.15, 	horizontal_flip=True, 	fill_mode="nearest")
                                                                                                           ## 
callback = EarlyStopping( monitor= "val_accuracy",    patience=15,       mode="max")

# checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1,   save_best_only=True, mode='max')
                                                                                                            ##
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5,   patience=6,     verbose=1, mode='max', min_lr=0.00001)

100%|██████████| 450/450 [04:30<00:00,  1.66it/s]


In [ ]:
base_model_Incep = tf.keras.applications.InceptionV3(input_shape=(299, 299, 3), include_top=False, weights="imagenet")
base_model_Incep.trainable = False

#Add custom head
average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model_Incep.output)
prediction_layer = tf.keras.layers.Dense(units=3, activation="softmax")(average_pooling_layer)   # unit 3
model_Incep = tf.keras.models.Model(inputs=base_model_Incep.input, outputs=prediction_layer)

model_Incep.compile(optimizer=RMSprop(learning_rate=0.0001), loss="binary_crossentropy", metrics=["accuracy"])


### K-fold
kf_Incep = KFold(5, shuffle=True, random_state=40) 

oos_y = []
oos_pred = []
fold = 0

for train, test in kf_Incep.split(data_Incep):
    fold+=1   
    print(f"Fold #{fold}")

    filename = f" Kmodel32_soft_pre_bi{fold}.h5 "
    checkpoint = ModelCheckpoint(filename, monitor='val_accuracy', verbose=1,   save_best_only=True, mode='max')
        
    x_train_Incep = data_Incep[train]
    y_train_Incep = labels_Incep[train]
    x_test_Incep = data_Incep[test]
    y_test_Incep = labels_Incep[test]

    H_Incep = model_Incep.fit( x=aug.flow(x_train_Incep, y_train_Incep, batch_size=BS),
	                                            validation_data=(x_test_Incep, y_test_Incep),
	                                            steps_per_epoch=len(x_train_Incep) // BS,
	                                            epochs=100,  callbacks = [callback,checkpoint,reduce_lr])

    pred_Incep = model_Incep.predict(x_test_Incep)
    
    oos_y.append(y_test_Incep)
    oos_pred.append(pred_Incep)  
    print(classification_report(y_test_Incep.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        pred_Incep.argmax(axis=1),  
                                        target_names=le_Incep.classes_))
    
oos_y = np.concatenate(oos_y)
oos_pred = np.concatenate(oos_pred)
print(classification_report(oos_y.argmax(axis=1), oos_pred.argmax(axis=1),  target_names=le_Incep.classes_))

87924736/87910968 [==============================] - 1s 0us/step
Fold #1
Epoch 1/100
11/11 [==============================] - ETA: 0s - loss: 0.7715 - accuracy: 0.3445
Epoch 1: val_accuracy improved from -inf to 0.31111, saving model to  Kmodel32_soft_pre_bi1.h5 
INFO:tensorflow:Assets written to:  Kmodel32_soft_pre_bi1.h5 /assets
11/11 [==============================] - 62s 5s/step - loss: 0.7715 - accuracy: 0.3445 - val_loss: 0.7008 - val_accuracy: 0.3111 - lr: 1.0000e-04
Epoch 2/100
11/11 [==============================] - ETA: 0s - loss: 0.6746 - accuracy: 0.3409
Epoch 2: val_accuracy improved from 0.31111 to 0.32222, saving model to  Kmodel32_soft_pre_bi1.h5 
INFO:tensorflow:Assets written to:  Kmodel32_soft_pre_bi1.h5 /assets
11/11 [==============================] - 41s 4s/step - loss: 0.6746 - accuracy: 0.3409 - val_loss: 0.6497 - val_accuracy: 0.3222 - lr: 1.0000e-04
Epoch 3/100
11/11 [==============================] - ETA: 0s - loss: 0.6503 - accuracy: 0.3323
Epoch 3: val_accu

#### 32 soft pre bi adam  


In [ ]:
BS = 32
filepath = "Kmodel32_soft_pre_bi_adam.h5"                  ##

train_img_Incep = []
for ImageName in tqdm(data_train3['ImageName']):              
    image_path = All_3class_dir +'/' + ImageName                  
    img = cv2.imread(image_path)                                          
    img = cv2.resize(img, (299, 299))                                      
    train_img_Incep.append(img)
data_Incep = np.array(train_img_Incep, dtype="float32") / 255.0

labels_Incep = np.array(data_train3[['NameType']] )        
labels_Incep.shape = (len(labels_Incep),) 
le_Incep = LabelEncoder()                                                                     
labels_Incep = le_Incep.fit_transform(labels_Incep)                                         
labels_Incep = to_categorical(labels_Incep, num_classes=3)                   

aug = ImageDataGenerator( rotation_range=15,	width_shift_range=0.1, 	height_shift_range=0.1,
		shear_range=0.15, 	horizontal_flip=True, 	fill_mode="nearest")
                                                                                                           ## 
callback = EarlyStopping( monitor= "val_accuracy",    patience=15,       mode="max" )

# checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1,   save_best_only=True, mode='max')
                                                                                                            ##
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5,   patience=6,     verbose=1, mode='max', min_lr=0.00001)

100%|██████████| 450/450 [00:05<00:00, 87.44it/s] 


In [ ]:
base_model_Incep = tf.keras.applications.InceptionV3(input_shape=(299, 299, 3), include_top=False, weights="imagenet")
base_model_Incep.trainable = False

#Add custom head
average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model_Incep.output)
prediction_layer = tf.keras.layers.Dense(units=3, activation="softmax")(average_pooling_layer)   # unit 3
model_Incep = tf.keras.models.Model(inputs=base_model_Incep.input, outputs=prediction_layer)

model_Incep.compile(optimizer=Adam(learning_rate=0.0001), loss="binary_crossentropy", metrics=["accuracy"])


### K-fold
kf_Incep = KFold(5, shuffle=True, random_state=40) 

oos_y = []
oos_pred = []
fold = 0

for train, test in kf_Incep.split(data_Incep):
    fold+=1   
    print(f"Fold #{fold}")

    filename = f" Kmodel32_soft_pre_bi_adam{fold}.h5 "
    checkpoint = ModelCheckpoint(filename, monitor='val_accuracy', verbose=1,   save_best_only=True, mode='max')
        
    x_train_Incep = data_Incep[train]
    y_train_Incep = labels_Incep[train]
    x_test_Incep = data_Incep[test]
    y_test_Incep = labels_Incep[test]

    H_Incep = model_Incep.fit( x=aug.flow(x_train_Incep, y_train_Incep, batch_size=BS),
	                                            validation_data=(x_test_Incep, y_test_Incep),
	                                            steps_per_epoch=len(x_train_Incep) // BS,
	                                            epochs=100,  callbacks = [callback,checkpoint,reduce_lr])

    pred_Incep = model_Incep.predict(x_test_Incep)
    
    oos_y.append(y_test_Incep)
    oos_pred.append(pred_Incep)  
    print(classification_report(y_test_Incep.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        pred_Incep.argmax(axis=1),  
                                        target_names=le_Incep.classes_))
    
oos_y = np.concatenate(oos_y)
oos_pred = np.concatenate(oos_pred)
print(classification_report(oos_y.argmax(axis=1), oos_pred.argmax(axis=1),  target_names=le_Incep.classes_))

Fold #1
Epoch 1/100
11/11 [==============================] - ETA: 0s - loss: 0.7000 - accuracy: 0.3415
Epoch 1: val_accuracy improved from -inf to 0.35556, saving model to  Kmodel32_soft_pre_bi_adam1.h5 
INFO:tensorflow:Assets written to:  Kmodel32_soft_pre_bi_adam1.h5 /assets
11/11 [==============================] - 49s 4s/step - loss: 0.7000 - accuracy: 0.3415 - val_loss: 0.6597 - val_accuracy: 0.3556 - lr: 1.0000e-04
Epoch 2/100
11/11 [==============================] - ETA: 0s - loss: 0.6475 - accuracy: 0.3537
Epoch 2: val_accuracy improved from 0.35556 to 0.37778, saving model to  Kmodel32_soft_pre_bi_adam1.h5 
INFO:tensorflow:Assets written to:  Kmodel32_soft_pre_bi_adam1.h5 /assets
11/11 [==============================] - 42s 4s/step - loss: 0.6475 - accuracy: 0.3537 - val_loss: 0.6337 - val_accuracy: 0.3778 - lr: 1.0000e-04
Epoch 3/100
11/11 [==============================] - ETA: 0s - loss: 0.6421 - accuracy: 0.3537
Epoch 3: val_accuracy improved from 0.37778 to 0.44444, saving

#### 32 soft pre bi sgd  


In [ ]:
BS = 32
filepath = "Kmodel32_soft_pre_bi_sgd.h5"                  ##

train_img_Incep = []
for ImageName in tqdm(data_train3['ImageName']):              
    image_path = All_3class_dir +'/' + ImageName                  
    img = cv2.imread(image_path)                                          
    img = cv2.resize(img, (299, 299))                                      
    train_img_Incep.append(img)
data_Incep = np.array(train_img_Incep, dtype="float32") / 255.0

labels_Incep = np.array(data_train3[['NameType']] )        
labels_Incep.shape = (len(labels_Incep),) 
le_Incep = LabelEncoder()                                                                     
labels_Incep = le_Incep.fit_transform(labels_Incep)                                         
labels_Incep = to_categorical(labels_Incep, num_classes=3)                   

aug = ImageDataGenerator( rotation_range=15,	width_shift_range=0.1, 	height_shift_range=0.1,
		shear_range=0.15, 	horizontal_flip=True, 	fill_mode="nearest")
                                                                                                           ## 
callback = EarlyStopping( monitor= "val_accuracy",    patience=15,       mode="max" )

# checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1,   save_best_only=True, mode='max')
                                                                                                            ##
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5,   patience=6,     verbose=1, mode='max', min_lr=0.00001)

100%|██████████| 450/450 [00:04<00:00, 90.59it/s] 


In [ ]:
base_model_Incep = tf.keras.applications.InceptionV3(input_shape=(299, 299, 3), include_top=False, weights="imagenet")
base_model_Incep.trainable = False

#Add custom head
average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model_Incep.output)
prediction_layer = tf.keras.layers.Dense(units=3, activation="softmax")(average_pooling_layer)   # unit 3
model_Incep = tf.keras.models.Model(inputs=base_model_Incep.input, outputs=prediction_layer)

model_Incep.compile(optimizer=SGD(learning_rate=0.0001), loss="binary_crossentropy", metrics=["accuracy"])


### K-fold
kf_Incep = KFold(5, shuffle=True, random_state=40) 

oos_y = []
oos_pred = []
fold = 0

for train, test in kf_Incep.split(data_Incep):
    fold+=1   
    print(f"Fold #{fold}")

    filename = f" Kmodel32_soft_pre_bi_sgd{fold}.h5 "
    checkpoint = ModelCheckpoint(filename, monitor='val_accuracy', verbose=1,   save_best_only=True, mode='max')
        
    x_train_Incep = data_Incep[train]
    y_train_Incep = labels_Incep[train]
    x_test_Incep = data_Incep[test]
    y_test_Incep = labels_Incep[test]

    H_Incep = model_Incep.fit( x=aug.flow(x_train_Incep, y_train_Incep, batch_size=BS),
	                                            validation_data=(x_test_Incep, y_test_Incep),
	                                            steps_per_epoch=len(x_train_Incep) // BS,
	                                            epochs=100,  callbacks = [callback,checkpoint,reduce_lr])

    pred_Incep = model_Incep.predict(x_test_Incep)
    
    oos_y.append(y_test_Incep)
    oos_pred.append(pred_Incep)  
    print(classification_report(y_test_Incep.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        pred_Incep.argmax(axis=1),  
                                        target_names=le_Incep.classes_))
    
oos_y = np.concatenate(oos_y)
oos_pred = np.concatenate(oos_pred)
print(classification_report(oos_y.argmax(axis=1), oos_pred.argmax(axis=1),  target_names=le_Incep.classes_))

Fold #1
Epoch 1/100
11/11 [==============================] - ETA: 0s - loss: 0.7177 - accuracy: 0.2500
Epoch 1: val_accuracy improved from -inf to 0.20000, saving model to  Kmodel32_soft_pre_bi_sgd1.h5 
INFO:tensorflow:Assets written to:  Kmodel32_soft_pre_bi_sgd1.h5 /assets
11/11 [==============================] - 48s 4s/step - loss: 0.7177 - accuracy: 0.2500 - val_loss: 0.7625 - val_accuracy: 0.2000 - lr: 1.0000e-04
Epoch 2/100
11/11 [==============================] - ETA: 0s - loss: 0.7157 - accuracy: 0.2500
Epoch 2: val_accuracy did not improve from 0.20000
11/11 [==============================] - 8s 722ms/step - loss: 0.7157 - accuracy: 0.2500 - val_loss: 0.7590 - val_accuracy: 0.1889 - lr: 1.0000e-04
Epoch 3/100
11/11 [==============================] - ETA: 0s - loss: 0.7138 - accuracy: 0.2652
Epoch 3: val_accuracy did not improve from 0.20000
11/11 [==============================] - 8s 715ms/step - loss: 0.7138 - accuracy: 0.2652 - val_loss: 0.7557 - val_accuracy: 0.1889 - lr: 1

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

  Lungcancer       0.32      0.68      0.43        34
      Normal       0.06      0.04      0.04        27
Tuberculosis       0.00      0.00      0.00        29

    accuracy                           0.27        90
   macro avg       0.12      0.24      0.16        90
weighted avg       0.14      0.27      0.18        90

Fold #3
Epoch 1/100
11/11 [==============================] - ETA: 0s - loss: 0.6851 - accuracy: 0.2561
Epoch 1: val_accuracy improved from -inf to 0.20000, saving model to  Kmodel32_soft_pre_bi_sgd3.h5 
INFO:tensorflow:Assets written to:  Kmodel32_soft_pre_bi_sgd3.h5 /assets
11/11 [==============================] - 40s 4s/step - loss: 0.6851 - accuracy: 0.2561 - val_loss: 0.7302 - val_accuracy: 0.2000 - lr: 1.0000e-05
Epoch 2/100
11/11 [==============================] - ETA: 0s - loss: 0.6892 - accuracy: 0.2774
Epoch 2: val_accuracy did not improve from 0.20000
11/11 [==============================] - 8s 722ms/s

#### 32 soft pre cate 


In [ ]:
BS = 32
filepath = "Kmodel32_soft_pre_cate.h5"                  ##

train_img_Incep = []
for ImageName in tqdm(data_train3['ImageName']):              
    image_path = All_3class_dir +'/' + ImageName                  
    img = cv2.imread(image_path)                                          
    img = cv2.resize(img, (299, 299))                                      
    train_img_Incep.append(img)
data_Incep = np.array(train_img_Incep, dtype="float32") / 255.0

labels_Incep = np.array(data_train3[['NameType']] )        
labels_Incep.shape = (len(labels_Incep),) 
le_Incep = LabelEncoder()                                                                     
labels_Incep = le_Incep.fit_transform(labels_Incep)                                         
labels_Incep = to_categorical(labels_Incep, num_classes=3)                   

aug = ImageDataGenerator( rotation_range=15,	width_shift_range=0.1, 	height_shift_range=0.1,
		shear_range=0.15, 	horizontal_flip=True, 	fill_mode="nearest")
                                                                                                           ## 
callback = EarlyStopping( monitor= "val_accuracy",    patience=15,       mode="max" )

# checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1,   save_best_only=True, mode='max')
                                                                                                            ##
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5,   patience=6,     verbose=1, mode='max', min_lr=0.00001)

100%|██████████| 450/450 [00:05<00:00, 89.31it/s] 


In [ ]:
base_model_Incep = tf.keras.applications.InceptionV3(input_shape=(299, 299, 3), include_top=False, weights="imagenet")
base_model_Incep.trainable = False

#Add custom head
average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model_Incep.output)
prediction_layer = tf.keras.layers.Dense(units=3, activation="softmax")(average_pooling_layer)   # unit 3
model_Incep = tf.keras.models.Model(inputs=base_model_Incep.input, outputs=prediction_layer)

model_Incep.compile(optimizer=RMSprop(learning_rate=0.0001), loss="categorical_crossentropy", metrics=["accuracy"])


### K-fold
kf_Incep = KFold(5, shuffle=True, random_state=40) 

oos_y = []
oos_pred = []
fold = 0

for train, test in kf_Incep.split(data_Incep):
    fold+=1   
    print(f"Fold #{fold}")

    filename = f" Kmodel32_soft_pre_cate{fold}.h5 "
    checkpoint = ModelCheckpoint(filename, monitor='val_accuracy', verbose=1,   save_best_only=True, mode='max')
        
    x_train_Incep = data_Incep[train]
    y_train_Incep = labels_Incep[train]
    x_test_Incep = data_Incep[test]
    y_test_Incep = labels_Incep[test]

    H_Incep = model_Incep.fit( x=aug.flow(x_train_Incep, y_train_Incep, batch_size=BS),
	                                            validation_data=(x_test_Incep, y_test_Incep),
	                                            steps_per_epoch=len(x_train_Incep) // BS,
	                                            epochs=100,  callbacks = [callback,checkpoint,reduce_lr])

    pred_Incep = model_Incep.predict(x_test_Incep)
    
    oos_y.append(y_test_Incep)
    oos_pred.append(pred_Incep)  
    print(classification_report(y_test_Incep.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        pred_Incep.argmax(axis=1),  
                                        target_names=le_Incep.classes_))
    
oos_y = np.concatenate(oos_y)
oos_pred = np.concatenate(oos_pred)
print(classification_report(oos_y.argmax(axis=1), oos_pred.argmax(axis=1),  target_names=le_Incep.classes_))

Fold #1
Epoch 1/100
11/11 [==============================] - ETA: 0s - loss: 1.1988 - accuracy: 0.3902
Epoch 1: val_accuracy improved from -inf to 0.43333, saving model to  Kmodel32_soft_pre_cate1.h5 
INFO:tensorflow:Assets written to:  Kmodel32_soft_pre_cate1.h5 /assets
11/11 [==============================] - 49s 4s/step - loss: 1.1988 - accuracy: 0.3902 - val_loss: 1.0583 - val_accuracy: 0.4333 - lr: 1.0000e-04
Epoch 2/100
11/11 [==============================] - ETA: 0s - loss: 1.0949 - accuracy: 0.3780
Epoch 2: val_accuracy improved from 0.43333 to 0.46667, saving model to  Kmodel32_soft_pre_cate1.h5 
INFO:tensorflow:Assets written to:  Kmodel32_soft_pre_cate1.h5 /assets
11/11 [==============================] - 40s 4s/step - loss: 1.0949 - accuracy: 0.3780 - val_loss: 1.0335 - val_accuracy: 0.4667 - lr: 1.0000e-04
Epoch 3/100
11/11 [==============================] - ETA: 0s - loss: 1.0339 - accuracy: 0.5030
Epoch 3: val_accuracy improved from 0.46667 to 0.47778, saving model to  K

#### 32 soft pre cate adam


In [ ]:
BS = 32
filepath = "Kmodel32_soft_pre_cate_adam.h5"                  ##

train_img_Incep = []
for ImageName in tqdm(data_train3['ImageName']):              
    image_path = All_3class_dir +'/' + ImageName                  
    img = cv2.imread(image_path)                                          
    img = cv2.resize(img, (299, 299))                                      
    train_img_Incep.append(img)
data_Incep = np.array(train_img_Incep, dtype="float32") / 255.0

labels_Incep = np.array(data_train3[['NameType']] )        
labels_Incep.shape = (len(labels_Incep),) 
le_Incep = LabelEncoder()                                                                     
labels_Incep = le_Incep.fit_transform(labels_Incep)                                         
labels_Incep = to_categorical(labels_Incep, num_classes=3)                   

aug = ImageDataGenerator( rotation_range=15,	width_shift_range=0.1, 	height_shift_range=0.1,
		shear_range=0.15, 	horizontal_flip=True, 	fill_mode="nearest")
                                                                                                           ## 
callback = EarlyStopping( monitor= "val_accuracy",    patience=15,       mode="max" )

# checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1,   save_best_only=True, mode='max')
                                                                                                            ##
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5,   patience=6,     verbose=1, mode='max', min_lr=0.00001)

100%|██████████| 450/450 [00:05<00:00, 82.33it/s] 


In [ ]:
base_model_Incep = tf.keras.applications.InceptionV3(input_shape=(299, 299, 3), include_top=False, weights="imagenet")
base_model_Incep.trainable = False

#Add custom head
average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model_Incep.output)
prediction_layer = tf.keras.layers.Dense(units=3, activation="softmax")(average_pooling_layer)   # unit 3
model_Incep = tf.keras.models.Model(inputs=base_model_Incep.input, outputs=prediction_layer)

model_Incep.compile(optimizer=Adam(learning_rate=0.0001), loss="categorical_crossentropy", metrics=["accuracy"])

### K-fold
kf_Incep = KFold(5, shuffle=True, random_state=40) 

oos_y = []
oos_pred = []
fold = 0

for train, test in kf_Incep.split(data_Incep):
    fold+=1   
    print(f"Fold #{fold}")

    filename = f" Kmodel32_soft_pre_cate_adam{fold}.h5 "
    checkpoint = ModelCheckpoint(filename, monitor='val_accuracy', verbose=1,   save_best_only=True, mode='max')
        
    x_train_Incep = data_Incep[train]
    y_train_Incep = labels_Incep[train]
    x_test_Incep = data_Incep[test]
    y_test_Incep = labels_Incep[test]

    H_Incep = model_Incep.fit( x=aug.flow(x_train_Incep, y_train_Incep, batch_size=BS),
	                                            validation_data=(x_test_Incep, y_test_Incep),
	                                            steps_per_epoch=len(x_train_Incep) // BS,
	                                            epochs=100,  callbacks = [callback,checkpoint,reduce_lr])

    pred_Incep = model_Incep.predict(x_test_Incep)
    
    oos_y.append(y_test_Incep)
    oos_pred.append(pred_Incep)  
    print(classification_report(y_test_Incep.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        pred_Incep.argmax(axis=1),  
                                        target_names=le_Incep.classes_))
    
oos_y = np.concatenate(oos_y)
oos_pred = np.concatenate(oos_pred)
print(classification_report(oos_y.argmax(axis=1), oos_pred.argmax(axis=1),  target_names=le_Incep.classes_))

Fold #1
Epoch 1/100
11/11 [==============================] - ETA: 0s - loss: 1.1857 - accuracy: 0.3445
Epoch 1: val_accuracy improved from -inf to 0.28889, saving model to  Kmodel32_soft_pre_cate_adam1.h5 
INFO:tensorflow:Assets written to:  Kmodel32_soft_pre_cate_adam1.h5 /assets
11/11 [==============================] - 62s 5s/step - loss: 1.1857 - accuracy: 0.3445 - val_loss: 1.1792 - val_accuracy: 0.2889 - lr: 1.0000e-04
Epoch 2/100
11/11 [==============================] - ETA: 0s - loss: 1.1223 - accuracy: 0.3049
Epoch 2: val_accuracy improved from 0.28889 to 0.36667, saving model to  Kmodel32_soft_pre_cate_adam1.h5 
INFO:tensorflow:Assets written to:  Kmodel32_soft_pre_cate_adam1.h5 /assets
11/11 [==============================] - 42s 4s/step - loss: 1.1223 - accuracy: 0.3049 - val_loss: 1.1638 - val_accuracy: 0.3667 - lr: 1.0000e-04
Epoch 3/100
11/11 [==============================] - ETA: 0s - loss: 1.0898 - accuracy: 0.3659
Epoch 3: val_accuracy improved from 0.36667 to 0.38889

#### 32 soft pre cate sgd


In [ ]:
BS = 32
filepath = "Kmodel32_soft_pre_cate_sgd.h5"                  ##

train_img_Incep = []
for ImageName in tqdm(data_train3['ImageName']):              
    image_path = All_3class_dir +'/' + ImageName                  
    img = cv2.imread(image_path)                                          
    img = cv2.resize(img, (299, 299))                                      
    train_img_Incep.append(img)
data_Incep = np.array(train_img_Incep, dtype="float32") / 255.0

labels_Incep = np.array(data_train3[['NameType']] )        
labels_Incep.shape = (len(labels_Incep),) 
le_Incep = LabelEncoder()                                                                     
labels_Incep = le_Incep.fit_transform(labels_Incep)                                         
labels_Incep = to_categorical(labels_Incep, num_classes=3)                   

aug = ImageDataGenerator( rotation_range=15,	width_shift_range=0.1, 	height_shift_range=0.1,
		shear_range=0.15, 	horizontal_flip=True, 	fill_mode="nearest")
                                                                                                           ## 
callback = EarlyStopping( monitor= "val_accuracy",    patience=15,    mode="max" )    #, baseline=1.0

# checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1,   save_best_only=True, mode='max')
                                                                                                            ##
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5,   patience=6,     verbose=1, mode='max', min_lr=0.00001)

100%|██████████| 450/450 [00:05<00:00, 83.69it/s] 


In [ ]:
base_model_Incep = tf.keras.applications.InceptionV3(input_shape=(299, 299, 3), include_top=False, weights="imagenet")
base_model_Incep.trainable = False

#Add custom head
average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model_Incep.output)
prediction_layer = tf.keras.layers.Dense(units=3, activation="softmax")(average_pooling_layer)   # unit 3
model_Incep = tf.keras.models.Model(inputs=base_model_Incep.input, outputs=prediction_layer)

model_Incep.compile(optimizer=SGD(learning_rate=0.0001), loss="categorical_crossentropy", metrics=["accuracy"])

### K-fold
kf_Incep = KFold(5, shuffle=True, random_state=40) 

oos_y = []
oos_pred = []
fold = 0

for train, test in kf_Incep.split(data_Incep):
    fold+=1   
    print(f"Fold #{fold}")

    filename = f" Kmodel32_soft_pre_cate_sgd{fold}.h5 "
    checkpoint = ModelCheckpoint(filename, monitor='val_accuracy', verbose=1,   save_best_only=True, mode='max')
        
    x_train_Incep = data_Incep[train]
    y_train_Incep = labels_Incep[train]
    x_test_Incep = data_Incep[test]
    y_test_Incep = labels_Incep[test]

    H_Incep = model_Incep.fit( x=aug.flow(x_train_Incep, y_train_Incep, batch_size=BS),
	                                            validation_data=(x_test_Incep, y_test_Incep),
	                                            steps_per_epoch=len(x_train_Incep) // BS,
	                                            epochs=100,  callbacks = [callback,checkpoint,reduce_lr])

    pred_Incep = model_Incep.predict(x_test_Incep)
    
    oos_y.append(y_test_Incep)
    oos_pred.append(pred_Incep)  
    print(classification_report(y_test_Incep.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        pred_Incep.argmax(axis=1),  
                                        target_names=le_Incep.classes_))
    
oos_y = np.concatenate(oos_y)
oos_pred = np.concatenate(oos_pred)
print(classification_report(oos_y.argmax(axis=1), oos_pred.argmax(axis=1),  target_names=le_Incep.classes_))

Fold #1
Epoch 1/100
11/11 [==============================] - ETA: 0s - loss: 1.2854 - accuracy: 0.3171
Epoch 1: val_accuracy improved from -inf to 0.35556, saving model to  Kmodel32_soft_pre_cate_sgd1.h5 
INFO:tensorflow:Assets written to:  Kmodel32_soft_pre_cate_sgd1.h5 /assets
11/11 [==============================] - 48s 4s/step - loss: 1.2854 - accuracy: 0.3171 - val_loss: 1.3078 - val_accuracy: 0.3556 - lr: 1.0000e-04
Epoch 2/100
11/11 [==============================] - ETA: 0s - loss: 1.2767 - accuracy: 0.3171
Epoch 2: val_accuracy did not improve from 0.35556
11/11 [==============================] - 8s 729ms/step - loss: 1.2767 - accuracy: 0.3171 - val_loss: 1.2715 - val_accuracy: 0.3556 - lr: 1.0000e-04
Epoch 3/100
11/11 [==============================] - ETA: 0s - loss: 1.2400 - accuracy: 0.3293
Epoch 3: val_accuracy improved from 0.35556 to 0.36667, saving model to  Kmodel32_soft_pre_cate_sgd1.h5 
INFO:tensorflow:Assets written to:  Kmodel32_soft_pre_cate_sgd1.h5 /assets
11/11

# K-fold 0-100
- 311 layer
-   filepath มี .h5 ทุกอัน
- bi cate RMSprop Adam SGD

### 1 ครบ


#### 1  soft pre bi


In [ ]:
BS = 1
filepath = "Kmodel1_soft_pre_bi.h5"                  ##

train_img_Incep = []
for ImageName in tqdm(data_train3['ImageName']):              
    image_path = All_3class_dir +'/' + ImageName                  
    img = cv2.imread(image_path)                                          
    img = cv2.resize(img, (299, 299))                                      
    train_img_Incep.append(img)
data_Incep = np.array(train_img_Incep, dtype="float32") / 255.0

labels_Incep = np.array(data_train3[['NameType']] )        
labels_Incep.shape = (len(labels_Incep),) 
le_Incep = LabelEncoder()                                                                     
labels_Incep = le_Incep.fit_transform(labels_Incep)                                         
labels_Incep = to_categorical(labels_Incep, num_classes=3)                   

aug = ImageDataGenerator( rotation_range=15,	width_shift_range=0.1, 	height_shift_range=0.1,
		shear_range=0.15, 	horizontal_flip=True, 	fill_mode="nearest")
                                                                                                           ## 
callback = EarlyStopping( monitor= "val_accuracy",    patience=15,       mode="max")

# checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1,   save_best_only=True, mode='max')
                                                                                                            ##
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5,   patience=6,     verbose=1, mode='max', min_lr=0.00001)

100%|██████████| 450/450 [01:49<00:00,  4.11it/s]


In [ ]:
base_model_Incep = tf.keras.applications.InceptionV3(input_shape=(299, 299, 3), include_top=False, weights="imagenet")
len(base_model_Incep.layers)

base_model_Incep.trainable = True
for layer in base_model_Incep.layers[:100]:    #fix w & bias in layer 0-100
    layer.trainable = False


#Add custom head
average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model_Incep.output)
prediction_layer = tf.keras.layers.Dense(units=3, activation="softmax")(average_pooling_layer)   # unit 3
model_Incep = tf.keras.models.Model(inputs=base_model_Incep.input, outputs=prediction_layer)

model_Incep.compile(optimizer=RMSprop(learning_rate=0.0001), loss="binary_crossentropy", metrics=["accuracy"])


### K-fold
kf_Incep = KFold(5, shuffle=True, random_state=40) 

oos_y = []
oos_pred = []
fold = 0

for train, test in kf_Incep.split(data_Incep):
    fold+=1   
    print(f"Fold #{fold}")

    filename = f" Kmodel1_soft_pre_bi{fold}.h5 "
    checkpoint = ModelCheckpoint(filename, monitor='val_accuracy', verbose=1,   save_best_only=True, mode='max')
        
    x_train_Incep = data_Incep[train]
    y_train_Incep = labels_Incep[train]
    x_test_Incep = data_Incep[test]
    y_test_Incep = labels_Incep[test]

    H_Incep = model_Incep.fit( x=aug.flow(x_train_Incep, y_train_Incep, batch_size=BS),
	                                            validation_data=(x_test_Incep, y_test_Incep),
	                                            steps_per_epoch=len(x_train_Incep) // BS,
	                                            epochs=100,  callbacks = [callback,checkpoint,reduce_lr])

    pred_Incep = model_Incep.predict(x_test_Incep)
    
    oos_y.append(y_test_Incep)
    oos_pred.append(pred_Incep)  
    print(classification_report(y_test_Incep.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        pred_Incep.argmax(axis=1),  
                                        target_names=le_Incep.classes_))
    
oos_y = np.concatenate(oos_y)
oos_pred = np.concatenate(oos_pred)
print(classification_report(oos_y.argmax(axis=1), oos_pred.argmax(axis=1),  target_names=le_Incep.classes_))

87924736/87910968 [==============================] - 3s 0us/step
Fold #1
Epoch 1/100
360/360 [==============================] - ETA: 0s - loss: 0.6104 - accuracy: 0.4528
Epoch 1: val_accuracy improved from -inf to 0.30000, saving model to  Kmodel1_soft_pre_bi1.h5 
INFO:tensorflow:Assets written to:  Kmodel1_soft_pre_bi1.h5 /assets
360/360 [==============================] - 122s 214ms/step - loss: 0.6104 - accuracy: 0.4528 - val_loss: 0.7491 - val_accuracy: 0.3000 - lr: 1.0000e-04
Epoch 2/100
360/360 [==============================] - ETA: 0s - loss: 0.4744 - accuracy: 0.5972
Epoch 2: val_accuracy improved from 0.30000 to 0.38889, saving model to  Kmodel1_soft_pre_bi1.h5 
INFO:tensorflow:Assets written to:  Kmodel1_soft_pre_bi1.h5 /assets
360/360 [==============================] - 68s 189ms/step - loss: 0.4744 - accuracy: 0.5972 - val_loss: 0.8039 - val_accuracy: 0.3889 - lr: 1.0000e-04
Epoch 3/100
360/360 [==============================] - ETA: 0s - loss: 0.4354 - accuracy: 0.6556
Epoc

#### 1 soft pre bi adam  


In [ ]:
BS = 1
filepath = "Kmodel1_soft_pre_bi_adam.h5"                  ##

train_img_Incep = []
for ImageName in tqdm(data_train3['ImageName']):              
    image_path = All_3class_dir +'/' + ImageName                  
    img = cv2.imread(image_path)                                          
    img = cv2.resize(img, (299, 299))                                      
    train_img_Incep.append(img)
data_Incep = np.array(train_img_Incep, dtype="float32") / 255.0

labels_Incep = np.array(data_train3[['NameType']] )        
labels_Incep.shape = (len(labels_Incep),) 
le_Incep = LabelEncoder()                                                                     
labels_Incep = le_Incep.fit_transform(labels_Incep)                                         
labels_Incep = to_categorical(labels_Incep, num_classes=3)                   

aug = ImageDataGenerator( rotation_range=15,	width_shift_range=0.1, 	height_shift_range=0.1,
		shear_range=0.15, 	horizontal_flip=True, 	fill_mode="nearest")
                                                                                                           ## 
callback = EarlyStopping( monitor= "val_accuracy",    patience=15,       mode="max")

# checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1,   save_best_only=True, mode='max')
                                                                                                            ##
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5,   patience=6,     verbose=1, mode='max', min_lr=0.00001)

100%|██████████| 450/450 [02:09<00:00,  3.49it/s]


In [ ]:
base_model_Incep = tf.keras.applications.InceptionV3(input_shape=(299, 299, 3), include_top=False, weights="imagenet")
len(base_model_Incep.layers)

base_model_Incep.trainable = True
for layer in base_model_Incep.layers[:100]:    #fix w & bias in layer 0-100
    layer.trainable = False


#Add custom head
average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model_Incep.output)
prediction_layer = tf.keras.layers.Dense(units=3, activation="softmax")(average_pooling_layer)   # unit 3
model_Incep = tf.keras.models.Model(inputs=base_model_Incep.input, outputs=prediction_layer)

model_Incep.compile(optimizer=Adam(learning_rate=0.0001), loss="binary_crossentropy", metrics=["accuracy"])


### K-fold
kf_Incep = KFold(5, shuffle=True, random_state=40) 

oos_y = []
oos_pred = []
fold = 0

for train, test in kf_Incep.split(data_Incep):
    fold+=1   
    print(f"Fold #{fold}")

    filename = f" Kmodel1_soft_pre_bi_adam{fold}.h5 "
    checkpoint = ModelCheckpoint(filename, monitor='val_accuracy', verbose=1,   save_best_only=True, mode='max')
        
    x_train_Incep = data_Incep[train]
    y_train_Incep = labels_Incep[train]
    x_test_Incep = data_Incep[test]
    y_test_Incep = labels_Incep[test]

    H_Incep = model_Incep.fit( x=aug.flow(x_train_Incep, y_train_Incep, batch_size=BS),
	                                            validation_data=(x_test_Incep, y_test_Incep),
	                                            steps_per_epoch=len(x_train_Incep) // BS,
	                                            epochs=100,  callbacks = [callback,checkpoint,reduce_lr])

    pred_Incep = model_Incep.predict(x_test_Incep)
    
    oos_y.append(y_test_Incep)
    oos_pred.append(pred_Incep)  
    print(classification_report(y_test_Incep.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        pred_Incep.argmax(axis=1),  
                                        target_names=le_Incep.classes_))
    
oos_y = np.concatenate(oos_y)
oos_pred = np.concatenate(oos_pred)
print(classification_report(oos_y.argmax(axis=1), oos_pred.argmax(axis=1),  target_names=le_Incep.classes_))

87924736/87910968 [==============================] - 1s 0us/step
Fold #1
Epoch 1/100
360/360 [==============================] - ETA: 0s - loss: 0.6118 - accuracy: 0.4361
Epoch 1: val_accuracy improved from -inf to 0.53333, saving model to  Kmodel1_soft_pre_bi_adam1.h5 
INFO:tensorflow:Assets written to:  Kmodel1_soft_pre_bi_adam1.h5 /assets
360/360 [==============================] - 73s 149ms/step - loss: 0.6118 - accuracy: 0.4361 - val_loss: 0.6954 - val_accuracy: 0.5333 - lr: 1.0000e-04
Epoch 2/100
360/360 [==============================] - ETA: 0s - loss: 0.4879 - accuracy: 0.5694
Epoch 2: val_accuracy improved from 0.53333 to 0.63333, saving model to  Kmodel1_soft_pre_bi_adam1.h5 
INFO:tensorflow:Assets written to:  Kmodel1_soft_pre_bi_adam1.h5 /assets
360/360 [==============================] - 47s 131ms/step - loss: 0.4879 - accuracy: 0.5694 - val_loss: 0.6227 - val_accuracy: 0.6333 - lr: 1.0000e-04
Epoch 3/100
360/360 [==============================] - ETA: 0s - loss: 0.4199 - ac

#### 1 soft pre bi sgd  


In [ ]:
BS = 1
filepath = "Kmodel1_soft_pre_bi_sgd.h5"                  ##

train_img_Incep = []
for ImageName in tqdm(data_train3['ImageName']):              
    image_path = All_3class_dir +'/' + ImageName                  
    img = cv2.imread(image_path)                                          
    img = cv2.resize(img, (299, 299))                                      
    train_img_Incep.append(img)
data_Incep = np.array(train_img_Incep, dtype="float32") / 255.0

labels_Incep = np.array(data_train3[['NameType']] )        
labels_Incep.shape = (len(labels_Incep),) 
le_Incep = LabelEncoder()                                                                     
labels_Incep = le_Incep.fit_transform(labels_Incep)                                         
labels_Incep = to_categorical(labels_Incep, num_classes=3)                   

aug = ImageDataGenerator( rotation_range=15,	width_shift_range=0.1, 	height_shift_range=0.1,
		shear_range=0.15, 	horizontal_flip=True, 	fill_mode="nearest")
                                                                                                           ## 
callback = EarlyStopping( monitor= "val_accuracy",    patience=15,       mode="max")

# checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1,   save_best_only=True, mode='max')
                                                                                                            ##
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5,   patience=6,     verbose=1, mode='max', min_lr=0.00001)

100%|██████████| 450/450 [00:13<00:00, 32.90it/s]


In [ ]:
base_model_Incep = tf.keras.applications.InceptionV3(input_shape=(299, 299, 3), include_top=False, weights="imagenet")
len(base_model_Incep.layers)

base_model_Incep.trainable = True
for layer in base_model_Incep.layers[:100]:    #fix w & bias in layer 0-100
    layer.trainable = False


#Add custom head
average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model_Incep.output)
prediction_layer = tf.keras.layers.Dense(units=3, activation="softmax")(average_pooling_layer)   # unit 3
model_Incep = tf.keras.models.Model(inputs=base_model_Incep.input, outputs=prediction_layer)

model_Incep.compile(optimizer=SGD(learning_rate=0.0001), loss="binary_crossentropy", metrics=["accuracy"])


### K-fold
kf_Incep = KFold(5, shuffle=True, random_state=40) 

oos_y = []
oos_pred = []
fold = 0

for train, test in kf_Incep.split(data_Incep):
    fold+=1   
    print(f"Fold #{fold}")

    filename = f" Kmodel1_soft_pre_bi_sgd{fold}.h5 "
    checkpoint = ModelCheckpoint(filename, monitor='val_accuracy', verbose=1,   save_best_only=True, mode='max')
        
    x_train_Incep = data_Incep[train]
    y_train_Incep = labels_Incep[train]
    x_test_Incep = data_Incep[test]
    y_test_Incep = labels_Incep[test]

    H_Incep = model_Incep.fit( x=aug.flow(x_train_Incep, y_train_Incep, batch_size=BS),
	                                            validation_data=(x_test_Incep, y_test_Incep),
	                                            steps_per_epoch=len(x_train_Incep) // BS,
	                                            epochs=100,  callbacks = [callback,checkpoint,reduce_lr])

    pred_Incep = model_Incep.predict(x_test_Incep)
    
    oos_y.append(y_test_Incep)
    oos_pred.append(pred_Incep)  
    print(classification_report(y_test_Incep.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        pred_Incep.argmax(axis=1),  
                                        target_names=le_Incep.classes_))
    
oos_y = np.concatenate(oos_y)
oos_pred = np.concatenate(oos_pred)
print(classification_report(oos_y.argmax(axis=1), oos_pred.argmax(axis=1),  target_names=le_Incep.classes_))

Fold #1
Epoch 1/100
360/360 [==============================] - ETA: 0s - loss: 0.6972 - accuracy: 0.3389
Epoch 1: val_accuracy improved from -inf to 0.27778, saving model to  Kmodel1_soft_pre_bi_sgd1.h5 
INFO:tensorflow:Assets written to:  Kmodel1_soft_pre_bi_sgd1.h5 /assets
360/360 [==============================] - 83s 188ms/step - loss: 0.6972 - accuracy: 0.3389 - val_loss: 0.6729 - val_accuracy: 0.2778 - lr: 1.0000e-04
Epoch 2/100
360/360 [==============================] - ETA: 0s - loss: 0.6573 - accuracy: 0.3306
Epoch 2: val_accuracy improved from 0.27778 to 0.31111, saving model to  Kmodel1_soft_pre_bi_sgd1.h5 
INFO:tensorflow:Assets written to:  Kmodel1_soft_pre_bi_sgd1.h5 /assets
360/360 [==============================] - 62s 173ms/step - loss: 0.6573 - accuracy: 0.3306 - val_loss: 0.6523 - val_accuracy: 0.3111 - lr: 1.0000e-04
Epoch 3/100
360/360 [==============================] - ETA: 0s - loss: 0.6430 - accuracy: 0.3417
Epoch 3: val_accuracy improved from 0.31111 to 0.33333

#### 1 soft pre cate 


In [ ]:
BS = 1
filepath = "Kmodel1_soft_pre_cate.h5"                  ##

train_img_Incep = []
for ImageName in tqdm(data_train3['ImageName']):              
    image_path = All_3class_dir +'/' + ImageName                  
    img = cv2.imread(image_path)                                          
    img = cv2.resize(img, (299, 299))                                      
    train_img_Incep.append(img)
data_Incep = np.array(train_img_Incep, dtype="float32") / 255.0

labels_Incep = np.array(data_train3[['NameType']] )        
labels_Incep.shape = (len(labels_Incep),) 
le_Incep = LabelEncoder()                                                                     
labels_Incep = le_Incep.fit_transform(labels_Incep)                                         
labels_Incep = to_categorical(labels_Incep, num_classes=3)                   

aug = ImageDataGenerator( rotation_range=15,	width_shift_range=0.1, 	height_shift_range=0.1,
		shear_range=0.15, 	horizontal_flip=True, 	fill_mode="nearest")
                                                                                                           ## 
callback = EarlyStopping( monitor= "val_accuracy",    patience=15,       mode="max")

# checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1,   save_best_only=True, mode='max')
                                                                                                            ##
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5,   patience=6,     verbose=1, mode='max', min_lr=0.00001)

100%|██████████| 450/450 [04:05<00:00,  1.83it/s]


In [ ]:
base_model_Incep = tf.keras.applications.InceptionV3(input_shape=(299, 299, 3), include_top=False, weights="imagenet")
len(base_model_Incep.layers)

base_model_Incep.trainable = True
for layer in base_model_Incep.layers[:100]:    #fix w & bias in layer 0-100
    layer.trainable = False


#Add custom head
average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model_Incep.output)
prediction_layer = tf.keras.layers.Dense(units=3, activation="softmax")(average_pooling_layer)   # unit 3
model_Incep = tf.keras.models.Model(inputs=base_model_Incep.input, outputs=prediction_layer)

model_Incep.compile(optimizer=RMSprop(learning_rate=0.0001), loss="categorical_crossentropy", metrics=["accuracy"])


### K-fold
kf_Incep = KFold(5, shuffle=True, random_state=40) 

oos_y = []
oos_pred = []
fold = 0

for train, test in kf_Incep.split(data_Incep):
    fold+=1   
    print(f"Fold #{fold}")

    filename = f" Kmodel1_soft_pre_cate{fold}.h5 "
    checkpoint = ModelCheckpoint(filename, monitor='val_accuracy', verbose=1,   save_best_only=True, mode='max')
        
    x_train_Incep = data_Incep[train]
    y_train_Incep = labels_Incep[train]
    x_test_Incep = data_Incep[test]
    y_test_Incep = labels_Incep[test]

    H_Incep = model_Incep.fit( x=aug.flow(x_train_Incep, y_train_Incep, batch_size=BS),
	                                            validation_data=(x_test_Incep, y_test_Incep),
	                                            steps_per_epoch=len(x_train_Incep) // BS,
	                                            epochs=100,  callbacks = [callback,checkpoint,reduce_lr])

    pred_Incep = model_Incep.predict(x_test_Incep)
    
    oos_y.append(y_test_Incep)
    oos_pred.append(pred_Incep)  
    print(classification_report(y_test_Incep.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        pred_Incep.argmax(axis=1),  
                                        target_names=le_Incep.classes_))
    
oos_y = np.concatenate(oos_y)
oos_pred = np.concatenate(oos_pred)
print(classification_report(oos_y.argmax(axis=1), oos_pred.argmax(axis=1),  target_names=le_Incep.classes_))

87924736/87910968 [==============================] - 2s 0us/step
Fold #1
Epoch 1/100
360/360 [==============================] - ETA: 0s - loss: 1.0959 - accuracy: 0.3611
Epoch 1: val_accuracy improved from -inf to 0.45556, saving model to  Kmodel1_soft_pre_cate1.h5 
INFO:tensorflow:Assets written to:  Kmodel1_soft_pre_cate1.h5 /assets
360/360 [==============================] - 94s 202ms/step - loss: 1.0959 - accuracy: 0.3611 - val_loss: 1.0793 - val_accuracy: 0.4556 - lr: 1.0000e-04
Epoch 2/100
360/360 [==============================] - ETA: 0s - loss: 0.8068 - accuracy: 0.5889
Epoch 2: val_accuracy improved from 0.45556 to 0.65556, saving model to  Kmodel1_soft_pre_cate1.h5 
INFO:tensorflow:Assets written to:  Kmodel1_soft_pre_cate1.h5 /assets
360/360 [==============================] - 65s 182ms/step - loss: 0.8068 - accuracy: 0.5889 - val_loss: 0.9109 - val_accuracy: 0.6556 - lr: 1.0000e-04
Epoch 3/100
360/360 [==============================] - ETA: 0s - loss: 0.7226 - accuracy: 0.66

#### 1 soft pre cate adam


In [ ]:
BS = 1
filepath = "Kmodel1_soft_pre_cate_adam.h5"                  ##

train_img_Incep = []
for ImageName in tqdm(data_train3['ImageName']):              
    image_path = All_3class_dir +'/' + ImageName                  
    img = cv2.imread(image_path)                                          
    img = cv2.resize(img, (299, 299))                                      
    train_img_Incep.append(img)
data_Incep = np.array(train_img_Incep, dtype="float32") / 255.0

labels_Incep = np.array(data_train3[['NameType']] )        
labels_Incep.shape = (len(labels_Incep),) 
le_Incep = LabelEncoder()                                                                     
labels_Incep = le_Incep.fit_transform(labels_Incep)                                         
labels_Incep = to_categorical(labels_Incep, num_classes=3)                   

aug = ImageDataGenerator( rotation_range=15,	width_shift_range=0.1, 	height_shift_range=0.1,
		shear_range=0.15, 	horizontal_flip=True, 	fill_mode="nearest")
                                                                                                           ## 
callback = EarlyStopping( monitor= "val_accuracy",    patience=15,       mode="max")

# checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1,   save_best_only=True, mode='max')
                                                                                                            ##
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5,   patience=6,     verbose=1, mode='max', min_lr=0.00001)

In [ ]:
base_model_Incep = tf.keras.applications.InceptionV3(input_shape=(299, 299, 3), include_top=False, weights="imagenet")
len(base_model_Incep.layers)

base_model_Incep.trainable = True
for layer in base_model_Incep.layers[:100]:    #fix w & bias in layer 0-100
    layer.trainable = False


#Add custom head
average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model_Incep.output)
prediction_layer = tf.keras.layers.Dense(units=3, activation="softmax")(average_pooling_layer)   # unit 3
model_Incep = tf.keras.models.Model(inputs=base_model_Incep.input, outputs=prediction_layer)

model_Incep.compile(optimizer=Adam(learning_rate=0.0001), loss="categorical_crossentropy", metrics=["accuracy"])

### K-fold
kf_Incep = KFold(5, shuffle=True, random_state=40) 

oos_y = []
oos_pred = []
fold = 0

for train, test in kf_Incep.split(data_Incep):
    fold+=1   
    print(f"Fold #{fold}")

    filename = f" Kmodel1_soft_pre_cate_adam{fold}.h5 "
    checkpoint = ModelCheckpoint(filename, monitor='val_accuracy', verbose=1,   save_best_only=True, mode='max')
        
    x_train_Incep = data_Incep[train]
    y_train_Incep = labels_Incep[train]
    x_test_Incep = data_Incep[test]
    y_test_Incep = labels_Incep[test]

    H_Incep = model_Incep.fit( x=aug.flow(x_train_Incep, y_train_Incep, batch_size=BS),
	                                            validation_data=(x_test_Incep, y_test_Incep),
	                                            steps_per_epoch=len(x_train_Incep) // BS,
	                                            epochs=100,  callbacks = [callback,checkpoint,reduce_lr])

    pred_Incep = model_Incep.predict(x_test_Incep)
    
    oos_y.append(y_test_Incep)
    oos_pred.append(pred_Incep)  
    print(classification_report(y_test_Incep.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        pred_Incep.argmax(axis=1),  
                                        target_names=le_Incep.classes_))
    
oos_y = np.concatenate(oos_y)
oos_pred = np.concatenate(oos_pred)
print(classification_report(oos_y.argmax(axis=1), oos_pred.argmax(axis=1),  target_names=le_Incep.classes_))

Fold #1
Epoch 1/100
360/360 [==============================] - ETA: 0s - loss: 1.0928 - accuracy: 0.3972
Epoch 1: val_accuracy improved from -inf to 0.55556, saving model to  Kmodel1_soft_pre_cate_adam1.h5 
INFO:tensorflow:Assets written to:  Kmodel1_soft_pre_cate_adam1.h5 /assets
360/360 [==============================] - 72s 177ms/step - loss: 1.0928 - accuracy: 0.3972 - val_loss: 0.8553 - val_accuracy: 0.5556 - lr: 1.0000e-04
Epoch 2/100
360/360 [==============================] - ETA: 0s - loss: 0.8239 - accuracy: 0.5694
Epoch 2: val_accuracy did not improve from 0.55556
360/360 [==============================] - 29s 79ms/step - loss: 0.8239 - accuracy: 0.5694 - val_loss: 1.7056 - val_accuracy: 0.5333 - lr: 1.0000e-04
Epoch 3/100
360/360 [==============================] - ETA: 0s - loss: 0.7614 - accuracy: 0.5861
Epoch 3: val_accuracy improved from 0.55556 to 0.64444, saving model to  Kmodel1_soft_pre_cate_adam1.h5 
INFO:tensorflow:Assets written to:  Kmodel1_soft_pre_cate_adam1.h5 

#### 1 soft pre cate sgd


In [ ]:
BS = 1
filepath = "Kmodel1_soft_pre_cate_sgd.h5"                  ##

train_img_Incep = []
for ImageName in tqdm(data_train3['ImageName']):              
    image_path = All_3class_dir +'/' + ImageName                  
    img = cv2.imread(image_path)                                          
    img = cv2.resize(img, (299, 299))                                      
    train_img_Incep.append(img)
data_Incep = np.array(train_img_Incep, dtype="float32") / 255.0

labels_Incep = np.array(data_train3[['NameType']] )        
labels_Incep.shape = (len(labels_Incep),) 
le_Incep = LabelEncoder()                                                                     
labels_Incep = le_Incep.fit_transform(labels_Incep)                                         
labels_Incep = to_categorical(labels_Incep, num_classes=3)                   

aug = ImageDataGenerator( rotation_range=15,	width_shift_range=0.1, 	height_shift_range=0.1,
		shear_range=0.15, 	horizontal_flip=True, 	fill_mode="nearest")
                                                                                                           ## 
callback = EarlyStopping( monitor= "val_accuracy",    patience=15,       mode="max")

# checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1,   save_best_only=True, mode='max')
                                                                                                            ##
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5,   patience=6,     verbose=1, mode='max', min_lr=0.00001)

In [ ]:
base_model_Incep = tf.keras.applications.InceptionV3(input_shape=(299, 299, 3), include_top=False, weights="imagenet")
len(base_model_Incep.layers)

base_model_Incep.trainable = True
for layer in base_model_Incep.layers[:100]:    #fix w & bias in layer 0-100
    layer.trainable = False


#Add custom head
average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model_Incep.output)
prediction_layer = tf.keras.layers.Dense(units=3, activation="softmax")(average_pooling_layer)   # unit 3
model_Incep = tf.keras.models.Model(inputs=base_model_Incep.input, outputs=prediction_layer)

model_Incep.compile(optimizer=SGD(learning_rate=0.0001), loss="categorical_crossentropy", metrics=["accuracy"])

### K-fold
kf_Incep = KFold(5, shuffle=True, random_state=40) 

oos_y = []
oos_pred = []
fold = 0

for train, test in kf_Incep.split(data_Incep):
    fold+=1   
    print(f"Fold #{fold}")

    filename = f" Kmodel1_soft_pre_cate_sgd{fold}.h5 "
    checkpoint = ModelCheckpoint(filename, monitor='val_accuracy', verbose=1,   save_best_only=True, mode='max')
        
    x_train_Incep = data_Incep[train]
    y_train_Incep = labels_Incep[train]
    x_test_Incep = data_Incep[test]
    y_test_Incep = labels_Incep[test]

    H_Incep = model_Incep.fit( x=aug.flow(x_train_Incep, y_train_Incep, batch_size=BS),
	                                            validation_data=(x_test_Incep, y_test_Incep),
	                                            steps_per_epoch=len(x_train_Incep) // BS,
	                                            epochs=100,  callbacks = [callback,checkpoint,reduce_lr])

    pred_Incep = model_Incep.predict(x_test_Incep)
    
    oos_y.append(y_test_Incep)
    oos_pred.append(pred_Incep)  
    print(classification_report(y_test_Incep.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        pred_Incep.argmax(axis=1),  
                                        target_names=le_Incep.classes_))
    
oos_y = np.concatenate(oos_y)
oos_pred = np.concatenate(oos_pred)
print(classification_report(oos_y.argmax(axis=1), oos_pred.argmax(axis=1),  target_names=le_Incep.classes_))

Fold #1
Epoch 1/100
360/360 [==============================] - ETA: 0s - loss: 1.1100 - accuracy: 0.2861
Epoch 1: val_accuracy improved from -inf to 0.31111, saving model to  Kmodel1_soft_pre_cate_sgd1.h5 
INFO:tensorflow:Assets written to:  Kmodel1_soft_pre_cate_sgd1.h5 /assets
360/360 [==============================] - 66s 168ms/step - loss: 1.1100 - accuracy: 0.2861 - val_loss: 1.1377 - val_accuracy: 0.3111 - lr: 1.0000e-04
Epoch 2/100
360/360 [==============================] - ETA: 0s - loss: 1.1011 - accuracy: 0.3528
Epoch 2: val_accuracy did not improve from 0.31111
360/360 [==============================] - 27s 74ms/step - loss: 1.1011 - accuracy: 0.3528 - val_loss: 1.1321 - val_accuracy: 0.2778 - lr: 1.0000e-04
Epoch 3/100
360/360 [==============================] - ETA: 0s - loss: 1.1011 - accuracy: 0.3472
Epoch 3: val_accuracy improved from 0.31111 to 0.32222, saving model to  Kmodel1_soft_pre_cate_sgd1.h5 
INFO:tensorflow:Assets written to:  Kmodel1_soft_pre_cate_sgd1.h5 /ass

### 8 ครบ


#### 8  soft pre bi


In [ ]:
BS = 8
filepath = "Kmodel8_soft_pre_bi.h5"                  ##

train_img_Incep = []
for ImageName in tqdm(data_train3['ImageName']):              
    image_path = All_3class_dir +'/' + ImageName                  
    img = cv2.imread(image_path)                                          
    img = cv2.resize(img, (299, 299))                                      
    train_img_Incep.append(img)
data_Incep = np.array(train_img_Incep, dtype="float32") / 255.0

labels_Incep = np.array(data_train3[['NameType']] )        
labels_Incep.shape = (len(labels_Incep),) 
le_Incep = LabelEncoder()                                                                     
labels_Incep = le_Incep.fit_transform(labels_Incep)                                         
labels_Incep = to_categorical(labels_Incep, num_classes=3)                   

aug = ImageDataGenerator( rotation_range=15,	width_shift_range=0.1, 	height_shift_range=0.1,
		shear_range=0.15, 	horizontal_flip=True, 	fill_mode="nearest")
                                                                                                           ## 
callback = EarlyStopping( monitor= "val_accuracy",    patience=15,       mode="max")

# checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1,   save_best_only=True, mode='max')
                                                                                                            ##
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5,   patience=6,     verbose=1, mode='max', min_lr=0.00001)

100%|██████████| 450/450 [00:09<00:00, 49.89it/s]


In [ ]:
base_model_Incep = tf.keras.applications.InceptionV3(input_shape=(299, 299, 3), include_top=False, weights="imagenet")
len(base_model_Incep.layers)

base_model_Incep.trainable = True
for layer in base_model_Incep.layers[:100]:    #fix w & bias in layer 0-100
    layer.trainable = False


#Add custom head
average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model_Incep.output)
prediction_layer = tf.keras.layers.Dense(units=3, activation="softmax")(average_pooling_layer)   # unit 3
model_Incep = tf.keras.models.Model(inputs=base_model_Incep.input, outputs=prediction_layer)

model_Incep.compile(optimizer=RMSprop(learning_rate=0.0001), loss="binary_crossentropy", metrics=["accuracy"])


### K-fold
kf_Incep = KFold(5, shuffle=True, random_state=40) 

oos_y = []
oos_pred = []
fold = 0

for train, test in kf_Incep.split(data_Incep):
    fold+=1   
    print(f"Fold #{fold}")

    filename = f" Kmodel8_soft_pre_bi{fold}.h5 "
    checkpoint = ModelCheckpoint(filename, monitor='val_accuracy', verbose=1,   save_best_only=True, mode='max')
        
    x_train_Incep = data_Incep[train]
    y_train_Incep = labels_Incep[train]
    x_test_Incep = data_Incep[test]
    y_test_Incep = labels_Incep[test]

    H_Incep = model_Incep.fit( x=aug.flow(x_train_Incep, y_train_Incep, batch_size=BS),
	                                            validation_data=(x_test_Incep, y_test_Incep),
	                                            steps_per_epoch=len(x_train_Incep) // BS,
	                                            epochs=100,  callbacks = [callback,checkpoint,reduce_lr])

    pred_Incep = model_Incep.predict(x_test_Incep)
    
    oos_y.append(y_test_Incep)
    oos_pred.append(pred_Incep)  
    print(classification_report(y_test_Incep.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        pred_Incep.argmax(axis=1),  
                                        target_names=le_Incep.classes_))
    
oos_y = np.concatenate(oos_y)
oos_pred = np.concatenate(oos_pred)
print(classification_report(oos_y.argmax(axis=1), oos_pred.argmax(axis=1),  target_names=le_Incep.classes_))

Fold #1
Epoch 1/100
45/45 [==============================] - ETA: 0s - loss: 0.4859 - accuracy: 0.6361
Epoch 1: val_accuracy improved from -inf to 0.55556, saving model to  Kmodel8_soft_pre_bi1.h5 
INFO:tensorflow:Assets written to:  Kmodel8_soft_pre_bi1.h5 /assets
45/45 [==============================] - 99s 2s/step - loss: 0.4859 - accuracy: 0.6361 - val_loss: 0.5577 - val_accuracy: 0.5556 - lr: 1.0000e-04
Epoch 2/100
45/45 [==============================] - ETA: 0s - loss: 0.3325 - accuracy: 0.7972
Epoch 2: val_accuracy did not improve from 0.55556
45/45 [==============================] - 13s 284ms/step - loss: 0.3325 - accuracy: 0.7972 - val_loss: 0.6807 - val_accuracy: 0.5444 - lr: 1.0000e-04
Epoch 3/100
45/45 [==============================] - ETA: 0s - loss: 0.2386 - accuracy: 0.8639
Epoch 3: val_accuracy improved from 0.55556 to 0.63333, saving model to  Kmodel8_soft_pre_bi1.h5 
INFO:tensorflow:Assets written to:  Kmodel8_soft_pre_bi1.h5 /assets
45/45 [=========================

In [ ]:
H_Incep.history.keys()

dict_keys(['loss', 'accuracy', 'val_loss', 'val_accuracy', 'lr'])

In [ ]:
lr = H_Incep.history['lr']

plt.figure(figsize=(20,15))

plt.subplot(2, 2, 1)
plt.plot(lr, label = " Learning rate")

#### 8 soft pre bi adam  


In [ ]:
BS = 8
filepath = "Kmodel8_soft_pre_bi_adam.h5"                  ##

train_img_Incep = []
for ImageName in tqdm(data_train3['ImageName']):              
    image_path = All_3class_dir +'/' + ImageName                  
    img = cv2.imread(image_path)                                          
    img = cv2.resize(img, (299, 299))                                      
    train_img_Incep.append(img)
data_Incep = np.array(train_img_Incep, dtype="float32") / 255.0

labels_Incep = np.array(data_train3[['NameType']] )        
labels_Incep.shape = (len(labels_Incep),) 
le_Incep = LabelEncoder()                                                                     
labels_Incep = le_Incep.fit_transform(labels_Incep)                                         
labels_Incep = to_categorical(labels_Incep, num_classes=3)                   

aug = ImageDataGenerator( rotation_range=15,	width_shift_range=0.1, 	height_shift_range=0.1,
		shear_range=0.15, 	horizontal_flip=True, 	fill_mode="nearest")
                                                                                                           ## 
callback = EarlyStopping( monitor= "val_accuracy",    patience=15,       mode="max")

# checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1,   save_best_only=True, mode='max')
                                                                                                            ##
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5,   patience=6,     verbose=1, mode='max', min_lr=0.00001)

100%|██████████| 450/450 [00:05<00:00, 87.26it/s] 


In [ ]:
base_model_Incep = tf.keras.applications.InceptionV3(input_shape=(299, 299, 3), include_top=False, weights="imagenet")
len(base_model_Incep.layers)

base_model_Incep.trainable = True
for layer in base_model_Incep.layers[:100]:    #fix w & bias in layer 0-100
    layer.trainable = False


#Add custom head
average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model_Incep.output)
prediction_layer = tf.keras.layers.Dense(units=3, activation="softmax")(average_pooling_layer)   # unit 3
model_Incep = tf.keras.models.Model(inputs=base_model_Incep.input, outputs=prediction_layer)

model_Incep.compile(optimizer=Adam(learning_rate=0.0001), loss="binary_crossentropy", metrics=["accuracy"])


### K-fold
kf_Incep = KFold(5, shuffle=True, random_state=40) 

oos_y = []
oos_pred = []
fold = 0

for train, test in kf_Incep.split(data_Incep):
    fold+=1   
    print(f"Fold #{fold}")

    filename = f" Kmodel8_soft_pre_bi_adam{fold}.h5 "
    checkpoint = ModelCheckpoint(filename, monitor='val_accuracy', verbose=1,   save_best_only=True, mode='max')
        
    x_train_Incep = data_Incep[train]
    y_train_Incep = labels_Incep[train]
    x_test_Incep = data_Incep[test]
    y_test_Incep = labels_Incep[test]

    H_Incep = model_Incep.fit( x=aug.flow(x_train_Incep, y_train_Incep, batch_size=BS),
	                                            validation_data=(x_test_Incep, y_test_Incep),
	                                            steps_per_epoch=len(x_train_Incep) // BS,
	                                            epochs=100,  callbacks = [callback,checkpoint,reduce_lr])

    pred_Incep = model_Incep.predict(x_test_Incep)
    
    oos_y.append(y_test_Incep)
    oos_pred.append(pred_Incep)  
    print(classification_report(y_test_Incep.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        pred_Incep.argmax(axis=1),  
                                        target_names=le_Incep.classes_))
    
oos_y = np.concatenate(oos_y)
oos_pred = np.concatenate(oos_pred)
print(classification_report(oos_y.argmax(axis=1), oos_pred.argmax(axis=1),  target_names=le_Incep.classes_))

Fold #1
Epoch 1/100
45/45 [==============================] - ETA: 0s - loss: 0.5132 - accuracy: 0.5972
Epoch 1: val_accuracy improved from -inf to 0.52222, saving model to  Kmodel8_soft_pre_bi_adam1.h5 
INFO:tensorflow:Assets written to:  Kmodel8_soft_pre_bi_adam1.h5 /assets
45/45 [==============================] - 78s 2s/step - loss: 0.5132 - accuracy: 0.5972 - val_loss: 0.6663 - val_accuracy: 0.5222 - lr: 1.0000e-04
Epoch 2/100
45/45 [==============================] - ETA: 0s - loss: 0.3175 - accuracy: 0.8083
Epoch 2: val_accuracy improved from 0.52222 to 0.72222, saving model to  Kmodel8_soft_pre_bi_adam1.h5 
INFO:tensorflow:Assets written to:  Kmodel8_soft_pre_bi_adam1.h5 /assets
45/45 [==============================] - 49s 1s/step - loss: 0.3175 - accuracy: 0.8083 - val_loss: 0.3642 - val_accuracy: 0.7222 - lr: 1.0000e-04
Epoch 3/100
45/45 [==============================] - ETA: 0s - loss: 0.2747 - accuracy: 0.8417
Epoch 3: val_accuracy improved from 0.72222 to 0.74444, saving mod

#### 8 soft pre bi sgd  


In [ ]:
BS = 8
filepath = "Kmodel8_soft_pre_bi_sgd.h5"                  ##

train_img_Incep = []
for ImageName in tqdm(data_train3['ImageName']):              
    image_path = All_3class_dir +'/' + ImageName                  
    img = cv2.imread(image_path)                                          
    img = cv2.resize(img, (299, 299))                                      
    train_img_Incep.append(img)
data_Incep = np.array(train_img_Incep, dtype="float32") / 255.0

labels_Incep = np.array(data_train3[['NameType']] )        
labels_Incep.shape = (len(labels_Incep),) 
le_Incep = LabelEncoder()                                                                     
labels_Incep = le_Incep.fit_transform(labels_Incep)                                         
labels_Incep = to_categorical(labels_Incep, num_classes=3)                   

aug = ImageDataGenerator( rotation_range=15,	width_shift_range=0.1, 	height_shift_range=0.1,
		shear_range=0.15, 	horizontal_flip=True, 	fill_mode="nearest")
                                                                                                           ## 
callback = EarlyStopping( monitor= "val_accuracy",    patience=15,       mode="max")

# checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1,   save_best_only=True, mode='max')
                                                                                                            ##
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5,   patience=6,     verbose=1, mode='max', min_lr=0.00001)

100%|██████████| 450/450 [00:05<00:00, 86.61it/s] 


In [ ]:
base_model_Incep = tf.keras.applications.InceptionV3(input_shape=(299, 299, 3), include_top=False, weights="imagenet")
len(base_model_Incep.layers)

base_model_Incep.trainable = True
for layer in base_model_Incep.layers[:100]:    #fix w & bias in layer 0-100
    layer.trainable = False


#Add custom head
average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model_Incep.output)
prediction_layer = tf.keras.layers.Dense(units=3, activation="softmax")(average_pooling_layer)   # unit 3
model_Incep = tf.keras.models.Model(inputs=base_model_Incep.input, outputs=prediction_layer)

model_Incep.compile(optimizer=SGD(learning_rate=0.0001), loss="binary_crossentropy", metrics=["accuracy"])


### K-fold
kf_Incep = KFold(5, shuffle=True, random_state=40) 

oos_y = []
oos_pred = []
fold = 0

for train, test in kf_Incep.split(data_Incep):
    fold+=1   
    print(f"Fold #{fold}")

    filename = f" Kmodel8_soft_pre_bi_sgd{fold}.h5 "
    checkpoint = ModelCheckpoint(filename, monitor='val_accuracy', verbose=1,   save_best_only=True, mode='max')
        
    x_train_Incep = data_Incep[train]
    y_train_Incep = labels_Incep[train]
    x_test_Incep = data_Incep[test]
    y_test_Incep = labels_Incep[test]

    H_Incep = model_Incep.fit( x=aug.flow(x_train_Incep, y_train_Incep, batch_size=BS),
	                                            validation_data=(x_test_Incep, y_test_Incep),
	                                            steps_per_epoch=len(x_train_Incep) // BS,
	                                            epochs=100,  callbacks = [callback,checkpoint,reduce_lr])

    pred_Incep = model_Incep.predict(x_test_Incep)
    
    oos_y.append(y_test_Incep)
    oos_pred.append(pred_Incep)  
    print(classification_report(y_test_Incep.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        pred_Incep.argmax(axis=1),  
                                        target_names=le_Incep.classes_))
    
oos_y = np.concatenate(oos_y)
oos_pred = np.concatenate(oos_pred)
print(classification_report(oos_y.argmax(axis=1), oos_pred.argmax(axis=1),  target_names=le_Incep.classes_))

Fold #1
Epoch 1/100
45/45 [==============================] - ETA: 0s - loss: 0.7086 - accuracy: 0.3083
Epoch 1: val_accuracy improved from -inf to 0.34444, saving model to  Kmodel8_soft_pre_bi_sgd1.h5 
INFO:tensorflow:Assets written to:  Kmodel8_soft_pre_bi_sgd1.h5 /assets
45/45 [==============================] - 93s 1s/step - loss: 0.7086 - accuracy: 0.3083 - val_loss: 0.7040 - val_accuracy: 0.3444 - lr: 1.0000e-04
Epoch 2/100
45/45 [==============================] - ETA: 0s - loss: 0.6998 - accuracy: 0.3500
Epoch 2: val_accuracy did not improve from 0.34444
45/45 [==============================] - 12s 256ms/step - loss: 0.6998 - accuracy: 0.3500 - val_loss: 0.7079 - val_accuracy: 0.3333 - lr: 1.0000e-04
Epoch 3/100
45/45 [==============================] - ETA: 0s - loss: 0.6975 - accuracy: 0.2889
Epoch 3: val_accuracy improved from 0.34444 to 0.37778, saving model to  Kmodel8_soft_pre_bi_sgd1.h5 
INFO:tensorflow:Assets written to:  Kmodel8_soft_pre_bi_sgd1.h5 /assets
45/45 [=========

#### 8 soft pre cate 


In [ ]:
BS = 8
filepath = "Kmodel8_soft_pre_cate.h5"                  ##

train_img_Incep = []
for ImageName in tqdm(data_train3['ImageName']):              
    image_path = All_3class_dir +'/' + ImageName                  
    img = cv2.imread(image_path)                                          
    img = cv2.resize(img, (299, 299))                                      
    train_img_Incep.append(img)
data_Incep = np.array(train_img_Incep, dtype="float32") / 255.0

labels_Incep = np.array(data_train3[['NameType']] )        
labels_Incep.shape = (len(labels_Incep),) 
le_Incep = LabelEncoder()                                                                     
labels_Incep = le_Incep.fit_transform(labels_Incep)                                         
labels_Incep = to_categorical(labels_Incep, num_classes=3)                   

aug = ImageDataGenerator( rotation_range=15,	width_shift_range=0.1, 	height_shift_range=0.1,
		shear_range=0.15, 	horizontal_flip=True, 	fill_mode="nearest")
                                                                                                           ## 
callback = EarlyStopping( monitor= "val_accuracy",    patience=15,       mode="max")

# checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1,   save_best_only=True, mode='max')
                                                                                                            ##
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5,   patience=6,     verbose=1, mode='max', min_lr=0.00001)

100%|██████████| 450/450 [00:05<00:00, 87.44it/s] 


In [ ]:
base_model_Incep = tf.keras.applications.InceptionV3(input_shape=(299, 299, 3), include_top=False, weights="imagenet")
len(base_model_Incep.layers)

base_model_Incep.trainable = True
for layer in base_model_Incep.layers[:100]:    #fix w & bias in layer 0-100
    layer.trainable = False


#Add custom head
average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model_Incep.output)
prediction_layer = tf.keras.layers.Dense(units=3, activation="softmax")(average_pooling_layer)   # unit 3
model_Incep = tf.keras.models.Model(inputs=base_model_Incep.input, outputs=prediction_layer)

model_Incep.compile(optimizer=RMSprop(learning_rate=0.0001), loss="categorical_crossentropy", metrics=["accuracy"])


### K-fold
kf_Incep = KFold(5, shuffle=True, random_state=40) 

oos_y = []
oos_pred = []
fold = 0

for train, test in kf_Incep.split(data_Incep):
    fold+=1   
    print(f"Fold #{fold}")

    filename = f" Kmodel8_soft_pre_cate{fold}.h5 "
    checkpoint = ModelCheckpoint(filename, monitor='val_accuracy', verbose=1,   save_best_only=True, mode='max')
        
    x_train_Incep = data_Incep[train]
    y_train_Incep = labels_Incep[train]
    x_test_Incep = data_Incep[test]
    y_test_Incep = labels_Incep[test]

    H_Incep = model_Incep.fit( x=aug.flow(x_train_Incep, y_train_Incep, batch_size=BS),
	                                            validation_data=(x_test_Incep, y_test_Incep),
	                                            steps_per_epoch=len(x_train_Incep) // BS,
	                                            epochs=100,  callbacks = [callback,checkpoint,reduce_lr])

    pred_Incep = model_Incep.predict(x_test_Incep)
    
    oos_y.append(y_test_Incep)
    oos_pred.append(pred_Incep)  
    print(classification_report(y_test_Incep.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        pred_Incep.argmax(axis=1),  
                                        target_names=le_Incep.classes_))
    
oos_y = np.concatenate(oos_y)
oos_pred = np.concatenate(oos_pred)
print(classification_report(oos_y.argmax(axis=1), oos_pred.argmax(axis=1),  target_names=le_Incep.classes_))

Fold #1
Epoch 1/100
45/45 [==============================] - ETA: 0s - loss: 0.7615 - accuracy: 0.6528
Epoch 1: val_accuracy improved from -inf to 0.60000, saving model to  Kmodel8_soft_pre_cate1.h5 
INFO:tensorflow:Assets written to:  Kmodel8_soft_pre_cate1.h5 /assets
45/45 [==============================] - 59s 1s/step - loss: 0.7615 - accuracy: 0.6528 - val_loss: 0.8505 - val_accuracy: 0.6000 - lr: 1.0000e-04
Epoch 2/100
45/45 [==============================] - ETA: 0s - loss: 0.5673 - accuracy: 0.7750
Epoch 2: val_accuracy improved from 0.60000 to 0.71111, saving model to  Kmodel8_soft_pre_cate1.h5 
INFO:tensorflow:Assets written to:  Kmodel8_soft_pre_cate1.h5 /assets
45/45 [==============================] - 47s 1s/step - loss: 0.5673 - accuracy: 0.7750 - val_loss: 0.7341 - val_accuracy: 0.7111 - lr: 1.0000e-04
Epoch 3/100
45/45 [==============================] - ETA: 0s - loss: 0.5094 - accuracy: 0.8000
Epoch 3: val_accuracy did not improve from 0.71111
45/45 [====================

#### 8 soft pre cate adam


In [ ]:
BS = 8
filepath = "Kmodel8_soft_pre_cate_adam.h5"                  ##

train_img_Incep = []
for ImageName in tqdm(data_train3['ImageName']):              
    image_path = All_3class_dir +'/' + ImageName                  
    img = cv2.imread(image_path)                                          
    img = cv2.resize(img, (299, 299))                                      
    train_img_Incep.append(img)
data_Incep = np.array(train_img_Incep, dtype="float32") / 255.0

labels_Incep = np.array(data_train3[['NameType']] )        
labels_Incep.shape = (len(labels_Incep),) 
le_Incep = LabelEncoder()                                                                     
labels_Incep = le_Incep.fit_transform(labels_Incep)                                         
labels_Incep = to_categorical(labels_Incep, num_classes=3)                   

aug = ImageDataGenerator( rotation_range=15,	width_shift_range=0.1, 	height_shift_range=0.1,
		shear_range=0.15, 	horizontal_flip=True, 	fill_mode="nearest")
                                                                                                           ## 
callback = EarlyStopping( monitor= "val_accuracy",    patience=15,       mode="max")

# checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1,   save_best_only=True, mode='max')
                                                                                                            ##
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5,   patience=6,     verbose=1, mode='max', min_lr=0.00001)

100%|██████████| 450/450 [02:37<00:00,  2.86it/s]


In [ ]:
base_model_Incep = tf.keras.applications.InceptionV3(input_shape=(299, 299, 3), include_top=False, weights="imagenet")
len(base_model_Incep.layers)

base_model_Incep.trainable = True
for layer in base_model_Incep.layers[:100]:    #fix w & bias in layer 0-100
    layer.trainable = False


#Add custom head
average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model_Incep.output)
prediction_layer = tf.keras.layers.Dense(units=3, activation="softmax")(average_pooling_layer)   # unit 3
model_Incep = tf.keras.models.Model(inputs=base_model_Incep.input, outputs=prediction_layer)

model_Incep.compile(optimizer=Adam(learning_rate=0.0001), loss="categorical_crossentropy", metrics=["accuracy"])

### K-fold
kf_Incep = KFold(5, shuffle=True, random_state=40) 

oos_y = []
oos_pred = []
fold = 0

for train, test in kf_Incep.split(data_Incep):
    fold+=1   
    print(f"Fold #{fold}")

    filename = f" Kmodel8_soft_pre_cate_adam{fold}.h5 "
    checkpoint = ModelCheckpoint(filename, monitor='val_accuracy', verbose=1,   save_best_only=True, mode='max')
        
    x_train_Incep = data_Incep[train]
    y_train_Incep = labels_Incep[train]
    x_test_Incep = data_Incep[test]
    y_test_Incep = labels_Incep[test]

    H_Incep = model_Incep.fit( x=aug.flow(x_train_Incep, y_train_Incep, batch_size=BS),
	                                            validation_data=(x_test_Incep, y_test_Incep),
	                                            steps_per_epoch=len(x_train_Incep) // BS,
	                                            epochs=100,  callbacks = [callback,checkpoint,reduce_lr])

    pred_Incep = model_Incep.predict(x_test_Incep)
    
    oos_y.append(y_test_Incep)
    oos_pred.append(pred_Incep)  
    print(classification_report(y_test_Incep.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        pred_Incep.argmax(axis=1),  
                                        target_names=le_Incep.classes_))
    
oos_y = np.concatenate(oos_y)
oos_pred = np.concatenate(oos_pred)
print(classification_report(oos_y.argmax(axis=1), oos_pred.argmax(axis=1),  target_names=le_Incep.classes_))

87924736/87910968 [==============================] - 1s 0us/step
Fold #1
Epoch 1/100
45/45 [==============================] - ETA: 0s - loss: 0.7665 - accuracy: 0.6139
Epoch 1: val_accuracy improved from -inf to 0.56667, saving model to  Kmodel8_soft_pre_cate_adam1.h5 
INFO:tensorflow:Assets written to:  Kmodel8_soft_pre_cate_adam1.h5 /assets
45/45 [==============================] - 240s 5s/step - loss: 0.7665 - accuracy: 0.6139 - val_loss: 1.0940 - val_accuracy: 0.5667 - lr: 1.0000e-04
Epoch 2/100
45/45 [==============================] - ETA: 0s - loss: 0.5185 - accuracy: 0.8056
Epoch 2: val_accuracy improved from 0.56667 to 0.78889, saving model to  Kmodel8_soft_pre_cate_adam1.h5 
INFO:tensorflow:Assets written to:  Kmodel8_soft_pre_cate_adam1.h5 /assets
45/45 [==============================] - 222s 5s/step - loss: 0.5185 - accuracy: 0.8056 - val_loss: 0.5704 - val_accuracy: 0.7889 - lr: 1.0000e-04
Epoch 3/100
45/45 [==============================] - ETA: 0s - loss: 0.4244 - accuracy

#### 8 soft pre cate sgd


In [ ]:
BS = 8
filepath = "Kmodel8_soft_pre_cate_sgd.h5"                  ##

train_img_Incep = []
for ImageName in tqdm(data_train3['ImageName']):              
    image_path = All_3class_dir +'/' + ImageName                  
    img = cv2.imread(image_path)                                          
    img = cv2.resize(img, (299, 299))                                      
    train_img_Incep.append(img)
data_Incep = np.array(train_img_Incep, dtype="float32") / 255.0

labels_Incep = np.array(data_train3[['NameType']] )        
labels_Incep.shape = (len(labels_Incep),) 
le_Incep = LabelEncoder()                                                                     
labels_Incep = le_Incep.fit_transform(labels_Incep)                                         
labels_Incep = to_categorical(labels_Incep, num_classes=3)                   

aug = ImageDataGenerator( rotation_range=15,	width_shift_range=0.1, 	height_shift_range=0.1,
		shear_range=0.15, 	horizontal_flip=True, 	fill_mode="nearest")
                                                                                                           ## 
callback = EarlyStopping( monitor= "val_accuracy",    patience=15,       mode="max")

# checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1,   save_best_only=True, mode='max')
                                                                                                            ##
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5,   patience=6,     verbose=1, mode='max', min_lr=0.00001)

100%|██████████| 450/450 [00:15<00:00, 28.30it/s]


In [ ]:
base_model_Incep = tf.keras.applications.InceptionV3(input_shape=(299, 299, 3), include_top=False, weights="imagenet")
len(base_model_Incep.layers)

base_model_Incep.trainable = True
for layer in base_model_Incep.layers[:100]:    #fix w & bias in layer 0-100
    layer.trainable = False


#Add custom head
average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model_Incep.output)
prediction_layer = tf.keras.layers.Dense(units=3, activation="softmax")(average_pooling_layer)   # unit 3
model_Incep = tf.keras.models.Model(inputs=base_model_Incep.input, outputs=prediction_layer)

model_Incep.compile(optimizer=SGD(learning_rate=0.0001), loss="categorical_crossentropy", metrics=["accuracy"])

### K-fold
kf_Incep = KFold(5, shuffle=True, random_state=40) 

oos_y = []
oos_pred = []
fold = 0

for train, test in kf_Incep.split(data_Incep):
    fold+=1   
    print(f"Fold #{fold}")

    filename = f" Kmodel8_soft_pre_cate_sgd{fold}.h5 "
    checkpoint = ModelCheckpoint(filename, monitor='val_accuracy', verbose=1,   save_best_only=True, mode='max')
        
    x_train_Incep = data_Incep[train]
    y_train_Incep = labels_Incep[train]
    x_test_Incep = data_Incep[test]
    y_test_Incep = labels_Incep[test]

    H_Incep = model_Incep.fit( x=aug.flow(x_train_Incep, y_train_Incep, batch_size=BS),
	                                            validation_data=(x_test_Incep, y_test_Incep),
	                                            steps_per_epoch=len(x_train_Incep) // BS,
	                                            epochs=100,  callbacks = [callback,checkpoint,reduce_lr])

    pred_Incep = model_Incep.predict(x_test_Incep)
    
    oos_y.append(y_test_Incep)
    oos_pred.append(pred_Incep)  
    print(classification_report(y_test_Incep.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        pred_Incep.argmax(axis=1),  
                                        target_names=le_Incep.classes_))
    
oos_y = np.concatenate(oos_y)
oos_pred = np.concatenate(oos_pred)
print(classification_report(oos_y.argmax(axis=1), oos_pred.argmax(axis=1),  target_names=le_Incep.classes_))

Fold #1
Epoch 1/100
45/45 [==============================] - ETA: 0s - loss: 1.1543 - accuracy: 0.2917
Epoch 1: val_accuracy improved from -inf to 0.32222, saving model to  Kmodel8_soft_pre_cate_sgd1.h5 
INFO:tensorflow:Assets written to:  Kmodel8_soft_pre_cate_sgd1.h5 /assets
45/45 [==============================] - 82s 2s/step - loss: 1.1543 - accuracy: 0.2917 - val_loss: 1.1798 - val_accuracy: 0.3222 - lr: 1.0000e-04
Epoch 2/100
45/45 [==============================] - ETA: 0s - loss: 1.1319 - accuracy: 0.3028
Epoch 2: val_accuracy improved from 0.32222 to 0.35556, saving model to  Kmodel8_soft_pre_cate_sgd1.h5 
INFO:tensorflow:Assets written to:  Kmodel8_soft_pre_cate_sgd1.h5 /assets
45/45 [==============================] - 54s 1s/step - loss: 1.1319 - accuracy: 0.3028 - val_loss: 1.1130 - val_accuracy: 0.3556 - lr: 1.0000e-04
Epoch 3/100
45/45 [==============================] - ETA: 0s - loss: 1.1151 - accuracy: 0.3333
Epoch 3: val_accuracy improved from 0.35556 to 0.37778, saving

### 16 ครบ


#### 16  soft pre bi


In [ ]:
BS = 16
filepath = "Kmodel16_soft_pre_bi.h5"                  ##

train_img_Incep = []
for ImageName in tqdm(data_train3['ImageName']):              
    image_path = All_3class_dir +'/' + ImageName                  
    img = cv2.imread(image_path)                                          
    img = cv2.resize(img, (299, 299))                                      
    train_img_Incep.append(img)
data_Incep = np.array(train_img_Incep, dtype="float32") / 255.0

labels_Incep = np.array(data_train3[['NameType']] )        
labels_Incep.shape = (len(labels_Incep),) 
le_Incep = LabelEncoder()                                                                     
labels_Incep = le_Incep.fit_transform(labels_Incep)                                         
labels_Incep = to_categorical(labels_Incep, num_classes=3)                   

aug = ImageDataGenerator( rotation_range=15,	width_shift_range=0.1, 	height_shift_range=0.1,
		shear_range=0.15, 	horizontal_flip=True, 	fill_mode="nearest")
                                                                                                           ## 
callback = EarlyStopping( monitor= "val_accuracy",    patience=15,       mode="max")

# checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1,   save_best_only=True, mode='max')
                                                                                                            ##
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5,   patience=6,     verbose=1, mode='max', min_lr=0.00001)

100%|██████████| 450/450 [03:49<00:00,  1.96it/s]


In [ ]:
base_model_Incep = tf.keras.applications.InceptionV3(input_shape=(299, 299, 3), include_top=False, weights="imagenet")
len(base_model_Incep.layers)

base_model_Incep.trainable = True
for layer in base_model_Incep.layers[:100]:    #fix w & bias in layer 0-100
    layer.trainable = False


#Add custom head
average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model_Incep.output)
prediction_layer = tf.keras.layers.Dense(units=3, activation="softmax")(average_pooling_layer)   # unit 3
model_Incep = tf.keras.models.Model(inputs=base_model_Incep.input, outputs=prediction_layer)

model_Incep.compile(optimizer=RMSprop(learning_rate=0.0001), loss="binary_crossentropy", metrics=["accuracy"])


### K-fold
kf_Incep = KFold(5, shuffle=True, random_state=40) 

oos_y = []
oos_pred = []
fold = 0

for train, test in kf_Incep.split(data_Incep):
    fold+=1   
    print(f"Fold #{fold}")

    filename = f" Kmodel16_soft_pre_bi{fold}.h5 "
    checkpoint = ModelCheckpoint(filename, monitor='val_accuracy', verbose=1,   save_best_only=True, mode='max')
        
    x_train_Incep = data_Incep[train]
    y_train_Incep = labels_Incep[train]
    x_test_Incep = data_Incep[test]
    y_test_Incep = labels_Incep[test]

    H_Incep = model_Incep.fit( x=aug.flow(x_train_Incep, y_train_Incep, batch_size=BS),
	                                            validation_data=(x_test_Incep, y_test_Incep),
	                                            steps_per_epoch=len(x_train_Incep) // BS,
	                                            epochs=100,  callbacks = [callback,checkpoint,reduce_lr])

    pred_Incep = model_Incep.predict(x_test_Incep)
    
    oos_y.append(y_test_Incep)
    oos_pred.append(pred_Incep)  
    print(classification_report(y_test_Incep.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        pred_Incep.argmax(axis=1),  
                                        target_names=le_Incep.classes_))
    
oos_y = np.concatenate(oos_y)
oos_pred = np.concatenate(oos_pred)
print(classification_report(oos_y.argmax(axis=1), oos_pred.argmax(axis=1),  target_names=le_Incep.classes_))

87924736/87910968 [==============================] - 1s 0us/step
Fold #1
Epoch 1/100
22/22 [==============================] - ETA: 0s - loss: 0.4811 - accuracy: 0.6628
Epoch 1: val_accuracy improved from -inf to 0.34444, saving model to  Kmodel16_soft_pre_bi1.h5 
INFO:tensorflow:Assets written to:  Kmodel16_soft_pre_bi1.h5 /assets
22/22 [==============================] - 227s 10s/step - loss: 0.4811 - accuracy: 0.6628 - val_loss: 1.0341 - val_accuracy: 0.3444 - lr: 1.0000e-04
Epoch 2/100
22/22 [==============================] - ETA: 0s - loss: 0.3386 - accuracy: 0.8081
Epoch 2: val_accuracy improved from 0.34444 to 0.47778, saving model to  Kmodel16_soft_pre_bi1.h5 
INFO:tensorflow:Assets written to:  Kmodel16_soft_pre_bi1.h5 /assets
22/22 [==============================] - 216s 10s/step - loss: 0.3386 - accuracy: 0.8081 - val_loss: 0.9141 - val_accuracy: 0.4778 - lr: 1.0000e-04
Epoch 3/100
22/22 [==============================] - ETA: 0s - loss: 0.2409 - accuracy: 0.8488
Epoch 3: val_

#### 16 soft pre bi adam  


In [ ]:
BS = 16
filepath = "Kmodel16_soft_pre_bi_adam.h5"                  ##

train_img_Incep = []
for ImageName in tqdm(data_train3['ImageName']):              
    image_path = All_3class_dir +'/' + ImageName                  
    img = cv2.imread(image_path)                                          
    img = cv2.resize(img, (299, 299))                                      
    train_img_Incep.append(img)
data_Incep = np.array(train_img_Incep, dtype="float32") / 255.0

labels_Incep = np.array(data_train3[['NameType']] )        
labels_Incep.shape = (len(labels_Incep),) 
le_Incep = LabelEncoder()                                                                     
labels_Incep = le_Incep.fit_transform(labels_Incep)                                         
labels_Incep = to_categorical(labels_Incep, num_classes=3)                   

aug = ImageDataGenerator( rotation_range=15,	width_shift_range=0.1, 	height_shift_range=0.1,
		shear_range=0.15, 	horizontal_flip=True, 	fill_mode="nearest")
                                                                                                           ## 
callback = EarlyStopping( monitor= "val_accuracy",    patience=15,       mode="max")

# checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1,   save_best_only=True, mode='max')
                                                                                                            ##
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5,   patience=6,     verbose=1, mode='max', min_lr=0.00001)

100%|██████████| 450/450 [02:11<00:00,  3.43it/s]


In [ ]:
base_model_Incep = tf.keras.applications.InceptionV3(input_shape=(299, 299, 3), include_top=False, weights="imagenet")
len(base_model_Incep.layers)

base_model_Incep.trainable = True
for layer in base_model_Incep.layers[:100]:    #fix w & bias in layer 0-100
    layer.trainable = False


#Add custom head
average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model_Incep.output)
prediction_layer = tf.keras.layers.Dense(units=3, activation="softmax")(average_pooling_layer)   # unit 3
model_Incep = tf.keras.models.Model(inputs=base_model_Incep.input, outputs=prediction_layer)

model_Incep.compile(optimizer=Adam(learning_rate=0.0001), loss="binary_crossentropy", metrics=["accuracy"])


### K-fold
kf_Incep = KFold(5, shuffle=True, random_state=40) 

oos_y = []
oos_pred = []
fold = 0

for train, test in kf_Incep.split(data_Incep):
    fold+=1   
    print(f"Fold #{fold}")

    filename = f" Kmodel16_soft_pre_bi_adam{fold}.h5 "
    checkpoint = ModelCheckpoint(filename, monitor='val_accuracy', verbose=1,   save_best_only=True, mode='max')
        
    x_train_Incep = data_Incep[train]
    y_train_Incep = labels_Incep[train]
    x_test_Incep = data_Incep[test]
    y_test_Incep = labels_Incep[test]

    H_Incep = model_Incep.fit( x=aug.flow(x_train_Incep, y_train_Incep, batch_size=BS),
	                                            validation_data=(x_test_Incep, y_test_Incep),
	                                            steps_per_epoch=len(x_train_Incep) // BS,
	                                            epochs=100,  callbacks = [callback,checkpoint,reduce_lr])

    pred_Incep = model_Incep.predict(x_test_Incep)
    
    oos_y.append(y_test_Incep)
    oos_pred.append(pred_Incep)  
    print(classification_report(y_test_Incep.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        pred_Incep.argmax(axis=1),  
                                        target_names=le_Incep.classes_))
    
oos_y = np.concatenate(oos_y)
oos_pred = np.concatenate(oos_pred)
print(classification_report(oos_y.argmax(axis=1), oos_pred.argmax(axis=1),  target_names=le_Incep.classes_))

Fold #1
Epoch 1/100
22/22 [==============================] - ETA: 0s - loss: 0.5259 - accuracy: 0.6335
Epoch 1: val_accuracy improved from -inf to 0.40000, saving model to  Kmodel16_soft_pre_bi_adam1.h5 
INFO:tensorflow:Assets written to:  Kmodel16_soft_pre_bi_adam1.h5 /assets
22/22 [==============================] - 57s 2s/step - loss: 0.5259 - accuracy: 0.6335 - val_loss: 0.9410 - val_accuracy: 0.4000 - lr: 1.0000e-04
Epoch 2/100
22/22 [==============================] - ETA: 0s - loss: 0.3359 - accuracy: 0.7791
Epoch 2: val_accuracy improved from 0.40000 to 0.47778, saving model to  Kmodel16_soft_pre_bi_adam1.h5 
INFO:tensorflow:Assets written to:  Kmodel16_soft_pre_bi_adam1.h5 /assets
22/22 [==============================] - 49s 2s/step - loss: 0.3359 - accuracy: 0.7791 - val_loss: 1.1569 - val_accuracy: 0.4778 - lr: 1.0000e-04
Epoch 3/100
22/22 [==============================] - ETA: 0s - loss: 0.2637 - accuracy: 0.8401
Epoch 3: val_accuracy improved from 0.47778 to 0.63333, saving

#### 16 soft pre bi sgd  


In [ ]:
BS = 16
filepath = "Kmodel16_soft_pre_bi_sgd.h5"                  ##

train_img_Incep = []
for ImageName in tqdm(data_train3['ImageName']):              
    image_path = All_3class_dir +'/' + ImageName                  
    img = cv2.imread(image_path)                                          
    img = cv2.resize(img, (299, 299))                                      
    train_img_Incep.append(img)
data_Incep = np.array(train_img_Incep, dtype="float32") / 255.0

labels_Incep = np.array(data_train3[['NameType']] )        
labels_Incep.shape = (len(labels_Incep),) 
le_Incep = LabelEncoder()                                                                     
labels_Incep = le_Incep.fit_transform(labels_Incep)                                         
labels_Incep = to_categorical(labels_Incep, num_classes=3)                   

aug = ImageDataGenerator( rotation_range=15,	width_shift_range=0.1, 	height_shift_range=0.1,
		shear_range=0.15, 	horizontal_flip=True, 	fill_mode="nearest")
                                                                                                           ## 
callback = EarlyStopping( monitor= "val_accuracy",    patience=15,       mode="max")

# checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1,   save_best_only=True, mode='max')
                                                                                                            ##
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5,   patience=6,     verbose=1, mode='max', min_lr=0.00001)

100%|██████████| 450/450 [01:50<00:00,  4.08it/s]


In [ ]:
base_model_Incep = tf.keras.applications.InceptionV3(input_shape=(299, 299, 3), include_top=False, weights="imagenet")
len(base_model_Incep.layers)

base_model_Incep.trainable = True
for layer in base_model_Incep.layers[:100]:    #fix w & bias in layer 0-100
    layer.trainable = False


#Add custom head
average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model_Incep.output)
prediction_layer = tf.keras.layers.Dense(units=3, activation="softmax")(average_pooling_layer)   # unit 3
model_Incep = tf.keras.models.Model(inputs=base_model_Incep.input, outputs=prediction_layer)

model_Incep.compile(optimizer=SGD(learning_rate=0.0001), loss="binary_crossentropy", metrics=["accuracy"])


### K-fold
kf_Incep = KFold(5, shuffle=True, random_state=40) 

oos_y = []
oos_pred = []
fold = 0

for train, test in kf_Incep.split(data_Incep):
    fold+=1   
    print(f"Fold #{fold}")

    filename = f" Kmodel16_soft_pre_bi_sgd{fold}.h5 "
    checkpoint = ModelCheckpoint(filename, monitor='val_accuracy', verbose=1,   save_best_only=True, mode='max')
        
    x_train_Incep = data_Incep[train]
    y_train_Incep = labels_Incep[train]
    x_test_Incep = data_Incep[test]
    y_test_Incep = labels_Incep[test]

    H_Incep = model_Incep.fit( x=aug.flow(x_train_Incep, y_train_Incep, batch_size=BS),
	                                            validation_data=(x_test_Incep, y_test_Incep),
	                                            steps_per_epoch=len(x_train_Incep) // BS,
	                                            epochs=100,  callbacks = [callback,checkpoint,reduce_lr])

    pred_Incep = model_Incep.predict(x_test_Incep)
    
    oos_y.append(y_test_Incep)
    oos_pred.append(pred_Incep)  
    print(classification_report(y_test_Incep.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        pred_Incep.argmax(axis=1),  
                                        target_names=le_Incep.classes_))
    
oos_y = np.concatenate(oos_y)
oos_pred = np.concatenate(oos_pred)
print(classification_report(oos_y.argmax(axis=1), oos_pred.argmax(axis=1),  target_names=le_Incep.classes_))

87924736/87910968 [==============================] - 1s 0us/step
Fold #1
Epoch 1/100
22/22 [==============================] - ETA: 0s - loss: 0.7646 - accuracy: 0.3634
Epoch 1: val_accuracy improved from -inf to 0.34444, saving model to  Kmodel16_soft_pre_bi_sgd1.h5 
INFO:tensorflow:Assets written to:  Kmodel16_soft_pre_bi_sgd1.h5 /assets
22/22 [==============================] - 72s 3s/step - loss: 0.7646 - accuracy: 0.3634 - val_loss: 0.7279 - val_accuracy: 0.3444 - lr: 1.0000e-04
Epoch 2/100
22/22 [==============================] - ETA: 0s - loss: 0.7555 - accuracy: 0.3372
Epoch 2: val_accuracy did not improve from 0.34444
22/22 [==============================] - 11s 478ms/step - loss: 0.7555 - accuracy: 0.3372 - val_loss: 0.7234 - val_accuracy: 0.3000 - lr: 1.0000e-04
Epoch 3/100
22/22 [==============================] - ETA: 0s - loss: 0.7471 - accuracy: 0.3576
Epoch 3: val_accuracy did not improve from 0.34444
22/22 [==============================] - 11s 471ms/step - loss: 0.7471 -

#### 16 soft pre cate 


In [ ]:
BS = 16
filepath = "Kmodel16_soft_pre_cate.h5"                  ##

train_img_Incep = []
for ImageName in tqdm(data_train3['ImageName']):              
    image_path = All_3class_dir +'/' + ImageName                  
    img = cv2.imread(image_path)                                          
    img = cv2.resize(img, (299, 299))                                      
    train_img_Incep.append(img)
data_Incep = np.array(train_img_Incep, dtype="float32") / 255.0

labels_Incep = np.array(data_train3[['NameType']] )        
labels_Incep.shape = (len(labels_Incep),) 
le_Incep = LabelEncoder()                                                                     
labels_Incep = le_Incep.fit_transform(labels_Incep)                                         
labels_Incep = to_categorical(labels_Incep, num_classes=3)                   

aug = ImageDataGenerator( rotation_range=15,	width_shift_range=0.1, 	height_shift_range=0.1,
		shear_range=0.15, 	horizontal_flip=True, 	fill_mode="nearest")
                                                                                                           ## 
callback = EarlyStopping( monitor= "val_accuracy",    patience=15,       mode="max" )

# checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1,   save_best_only=True, mode='max')
                                                                                                            ##
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5,   patience=6,     verbose=1, mode='max', min_lr=0.00001)

100%|██████████| 450/450 [01:48<00:00,  4.16it/s]


In [ ]:
base_model_Incep = tf.keras.applications.InceptionV3(input_shape=(299, 299, 3), include_top=False, weights="imagenet")
len(base_model_Incep.layers)

base_model_Incep.trainable = True
for layer in base_model_Incep.layers[:100]:    #fix w & bias in layer 0-100
    layer.trainable = False


#Add custom head
average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model_Incep.output)
prediction_layer = tf.keras.layers.Dense(units=3, activation="softmax")(average_pooling_layer)   # unit 3
model_Incep = tf.keras.models.Model(inputs=base_model_Incep.input, outputs=prediction_layer)

model_Incep.compile(optimizer=RMSprop(learning_rate=0.0001), loss="categorical_crossentropy", metrics=["accuracy"])


### K-fold
kf_Incep = KFold(5, shuffle=True, random_state=40) 

oos_y = []
oos_pred = []
fold = 0

for train, test in kf_Incep.split(data_Incep):
    fold+=1   
    print(f"Fold #{fold}")

    filename = f" Kmodel16_soft_pre_cate{fold}.h5 "
    checkpoint = ModelCheckpoint(filename, monitor='val_accuracy', verbose=1,   save_best_only=True, mode='max')
        
    x_train_Incep = data_Incep[train]
    y_train_Incep = labels_Incep[train]
    x_test_Incep = data_Incep[test]
    y_test_Incep = labels_Incep[test]

    H_Incep = model_Incep.fit( x=aug.flow(x_train_Incep, y_train_Incep, batch_size=BS),
	                                            validation_data=(x_test_Incep, y_test_Incep),
	                                            steps_per_epoch=len(x_train_Incep) // BS,
	                                            epochs=100,  callbacks = [callback,checkpoint,reduce_lr])

    pred_Incep = model_Incep.predict(x_test_Incep)
    
    oos_y.append(y_test_Incep)
    oos_pred.append(pred_Incep)  
    print(classification_report(y_test_Incep.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        pred_Incep.argmax(axis=1),  
                                        target_names=le_Incep.classes_))
    
oos_y = np.concatenate(oos_y)
oos_pred = np.concatenate(oos_pred)
print(classification_report(oos_y.argmax(axis=1), oos_pred.argmax(axis=1),  target_names=le_Incep.classes_))

87924736/87910968 [==============================] - 3s 0us/step
Fold #1
Epoch 1/100
22/22 [==============================] - ETA: 0s - loss: 0.7561 - accuracy: 0.6657
Epoch 1: val_accuracy improved from -inf to 0.43333, saving model to  Kmodel16_soft_pre_cate1.h5 
INFO:tensorflow:Assets written to:  Kmodel16_soft_pre_cate1.h5 /assets
22/22 [==============================] - 106s 3s/step - loss: 0.7561 - accuracy: 0.6657 - val_loss: 1.4739 - val_accuracy: 0.4333 - lr: 1.0000e-04
Epoch 2/100
22/22 [==============================] - ETA: 0s - loss: 0.4257 - accuracy: 0.8401
Epoch 2: val_accuracy improved from 0.43333 to 0.46667, saving model to  Kmodel16_soft_pre_cate1.h5 
INFO:tensorflow:Assets written to:  Kmodel16_soft_pre_cate1.h5 /assets
22/22 [==============================] - 49s 2s/step - loss: 0.4257 - accuracy: 0.8401 - val_loss: 1.7103 - val_accuracy: 0.4667 - lr: 1.0000e-04
Epoch 3/100
22/22 [==============================] - ETA: 0s - loss: 0.3984 - accuracy: 0.8488
Epoch 3:

####16 soft pre cate adam


In [ ]:
BS = 16                                                       ##
filepath = "Kmodel16_soft_pre_cate_adam.h5"                  ##

train_img_Incep = []
for ImageName in tqdm(data_train3['ImageName']):              
    image_path = All_3class_dir +'/' + ImageName                  
    img = cv2.imread(image_path)                                          
    img = cv2.resize(img, (299, 299))                                      
    train_img_Incep.append(img)
data_Incep = np.array(train_img_Incep, dtype="float32") / 255.0

labels_Incep = np.array(data_train3[['NameType']] )        
labels_Incep.shape = (len(labels_Incep),) 
le_Incep = LabelEncoder()                                                                     
labels_Incep = le_Incep.fit_transform(labels_Incep)                                         
labels_Incep = to_categorical(labels_Incep, num_classes=3)                   

aug = ImageDataGenerator( rotation_range=15,	width_shift_range=0.1, 	height_shift_range=0.1,
		shear_range=0.15, 	horizontal_flip=True, 	fill_mode="nearest")
                                                                                                           ## 
callback = EarlyStopping( monitor= "val_accuracy",    patience=15,       mode="max")

# checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1,   save_best_only=True, mode='max')
                                                                                                            ##
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5,   patience=6,     verbose=1, mode='max', min_lr=0.00001)

100%|██████████| 450/450 [00:11<00:00, 38.62it/s]


In [ ]:
base_model_Incep = tf.keras.applications.InceptionV3(input_shape=(299, 299, 3), include_top=False, weights="imagenet")
len(base_model_Incep.layers)

base_model_Incep.trainable = True
for layer in base_model_Incep.layers[:100]:    #fix w & bias in layer 0-100
    layer.trainable = False


#Add custom head
average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model_Incep.output)
prediction_layer = tf.keras.layers.Dense(units=3, activation="softmax")(average_pooling_layer)   # unit 3
model_Incep = tf.keras.models.Model(inputs=base_model_Incep.input, outputs=prediction_layer)

model_Incep.compile(optimizer=Adam(learning_rate=0.0001), loss="categorical_crossentropy", metrics=["accuracy"])

### K-fold
kf_Incep = KFold(5, shuffle=True, random_state=40) 

oos_y = []
oos_pred = []
fold = 0

for train, test in kf_Incep.split(data_Incep):
    fold+=1   
    print(f"Fold #{fold}")

    filename = f" Kmodel16_soft_pre_cate_adam{fold}.h5 "
    checkpoint = ModelCheckpoint(filename, monitor='val_accuracy', verbose=1,   save_best_only=True, mode='max')
        
    x_train_Incep = data_Incep[train]
    y_train_Incep = labels_Incep[train]
    x_test_Incep = data_Incep[test]
    y_test_Incep = labels_Incep[test]

    H_Incep = model_Incep.fit( x=aug.flow(x_train_Incep, y_train_Incep, batch_size=BS),
	                                            validation_data=(x_test_Incep, y_test_Incep),
	                                            steps_per_epoch=len(x_train_Incep) // BS,
	                                            epochs=100,  callbacks = [callback,checkpoint,reduce_lr])

    pred_Incep = model_Incep.predict(x_test_Incep)
    
    oos_y.append(y_test_Incep)
    oos_pred.append(pred_Incep)  
    print(classification_report(y_test_Incep.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        pred_Incep.argmax(axis=1),  
                                        target_names=le_Incep.classes_))
    
oos_y = np.concatenate(oos_y)
oos_pred = np.concatenate(oos_pred)
print(classification_report(oos_y.argmax(axis=1), oos_pred.argmax(axis=1),  target_names=le_Incep.classes_))

Fold #1
Epoch 1/100
22/22 [==============================] - ETA: 0s - loss: 0.7811 - accuracy: 0.6512
Epoch 1: val_accuracy improved from -inf to 0.41111, saving model to  Kmodel16_soft_pre_cate_adam1.h5 
INFO:tensorflow:Assets written to:  Kmodel16_soft_pre_cate_adam1.h5 /assets
22/22 [==============================] - 71s 3s/step - loss: 0.7811 - accuracy: 0.6512 - val_loss: 1.4843 - val_accuracy: 0.4111 - lr: 1.0000e-04
Epoch 2/100
22/22 [==============================] - ETA: 0s - loss: 0.4689 - accuracy: 0.8227
Epoch 2: val_accuracy improved from 0.41111 to 0.60000, saving model to  Kmodel16_soft_pre_cate_adam1.h5 
INFO:tensorflow:Assets written to:  Kmodel16_soft_pre_cate_adam1.h5 /assets
22/22 [==============================] - 50s 2s/step - loss: 0.4689 - accuracy: 0.8227 - val_loss: 0.9592 - val_accuracy: 0.6000 - lr: 1.0000e-04
Epoch 3/100
22/22 [==============================] - ETA: 0s - loss: 0.3591 - accuracy: 0.8605
Epoch 3: val_accuracy improved from 0.60000 to 0.61111

#### 16 soft pre cate sgd


In [ ]:
BS = 16                                                       ##
filepath = "Kmodel16_soft_pre_cate_sgd.h5"                  ##

train_img_Incep = []
for ImageName in tqdm(data_train3['ImageName']):              
    image_path = All_3class_dir +'/' + ImageName                  
    img = cv2.imread(image_path)                                          
    img = cv2.resize(img, (299, 299))                                      
    train_img_Incep.append(img)
data_Incep = np.array(train_img_Incep, dtype="float32") / 255.0

labels_Incep = np.array(data_train3[['NameType']] )        
labels_Incep.shape = (len(labels_Incep),) 
le_Incep = LabelEncoder()                                                                     
labels_Incep = le_Incep.fit_transform(labels_Incep)                                         
labels_Incep = to_categorical(labels_Incep, num_classes=3)                   

aug = ImageDataGenerator( rotation_range=15,	width_shift_range=0.1, 	height_shift_range=0.1,
		shear_range=0.15, 	horizontal_flip=True, 	fill_mode="nearest")
                                                                                                           ## 
callback = EarlyStopping( monitor= "val_accuracy",    patience=15,       mode="max" )

# checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1,   save_best_only=True, mode='max')
                                                                                                            ##
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5,   patience=6,     verbose=1, mode='max', min_lr=0.00001)

100%|██████████| 450/450 [00:11<00:00, 38.52it/s]


In [ ]:
base_model_Incep = tf.keras.applications.InceptionV3(input_shape=(299, 299, 3), include_top=False, weights="imagenet")
len(base_model_Incep.layers)

base_model_Incep.trainable = True
for layer in base_model_Incep.layers[:100]:    #fix w & bias in layer 0-100
    layer.trainable = False


#Add custom head
average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model_Incep.output)
prediction_layer = tf.keras.layers.Dense(units=3, activation="softmax")(average_pooling_layer)   # unit 3
model_Incep = tf.keras.models.Model(inputs=base_model_Incep.input, outputs=prediction_layer)

model_Incep.compile(optimizer=SGD(learning_rate=0.0001), loss="categorical_crossentropy", metrics=["accuracy"])

### K-fold
kf_Incep = KFold(5, shuffle=True, random_state=40) 

oos_y = []
oos_pred = []
fold = 0

for train, test in kf_Incep.split(data_Incep):
    fold+=1   
    print(f"Fold #{fold}")

    filename = f" Kmodel16_soft_pre_cate_sgd{fold}.h5 "
    checkpoint = ModelCheckpoint(filename, monitor='val_accuracy', verbose=1,   save_best_only=True, mode='max')
        
    x_train_Incep = data_Incep[train]
    y_train_Incep = labels_Incep[train]
    x_test_Incep = data_Incep[test]
    y_test_Incep = labels_Incep[test]

    H_Incep = model_Incep.fit( x=aug.flow(x_train_Incep, y_train_Incep, batch_size=BS),
	                                            validation_data=(x_test_Incep, y_test_Incep),
	                                            steps_per_epoch=len(x_train_Incep) // BS,
	                                            epochs=100,  callbacks = [callback,checkpoint,reduce_lr])

    pred_Incep = model_Incep.predict(x_test_Incep)
    
    oos_y.append(y_test_Incep)
    oos_pred.append(pred_Incep)  
    print(classification_report(y_test_Incep.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        pred_Incep.argmax(axis=1),  
                                        target_names=le_Incep.classes_))
    
oos_y = np.concatenate(oos_y)
oos_pred = np.concatenate(oos_pred)
print(classification_report(oos_y.argmax(axis=1), oos_pred.argmax(axis=1),  target_names=le_Incep.classes_))

Fold #1
Epoch 1/100
22/22 [==============================] - ETA: 0s - loss: 1.1706 - accuracy: 0.3140
Epoch 1: val_accuracy improved from -inf to 0.35556, saving model to  Kmodel16_soft_pre_cate_sgd1.h5 
INFO:tensorflow:Assets written to:  Kmodel16_soft_pre_cate_sgd1.h5 /assets
22/22 [==============================] - 57s 2s/step - loss: 1.1706 - accuracy: 0.3140 - val_loss: 1.1708 - val_accuracy: 0.3556 - lr: 1.0000e-04
Epoch 2/100
22/22 [==============================] - ETA: 0s - loss: 1.1396 - accuracy: 0.3663
Epoch 2: val_accuracy did not improve from 0.35556
22/22 [==============================] - 11s 486ms/step - loss: 1.1396 - accuracy: 0.3663 - val_loss: 1.1458 - val_accuracy: 0.2889 - lr: 1.0000e-04
Epoch 3/100
22/22 [==============================] - ETA: 0s - loss: 1.1336 - accuracy: 0.3285
Epoch 3: val_accuracy did not improve from 0.35556
22/22 [==============================] - 11s 484ms/step - loss: 1.1336 - accuracy: 0.3285 - val_loss: 1.1415 - val_accuracy: 0.3444 -

### 32  ครบ

#### 32  soft pre bi


In [ ]:
BS = 32
filepath = "Kmodel32_soft_pre_bi.h5"                  ##

train_img_Incep = []
for ImageName in tqdm(data_train3['ImageName']):              
    image_path = All_3class_dir +'/' + ImageName                  
    img = cv2.imread(image_path)                                          
    img = cv2.resize(img, (299, 299))                                      
    train_img_Incep.append(img)
data_Incep = np.array(train_img_Incep, dtype="float32") / 255.0

labels_Incep = np.array(data_train3[['NameType']] )        
labels_Incep.shape = (len(labels_Incep),) 
le_Incep = LabelEncoder()                                                                     
labels_Incep = le_Incep.fit_transform(labels_Incep)                                         
labels_Incep = to_categorical(labels_Incep, num_classes=3)                   

aug = ImageDataGenerator( rotation_range=15,	width_shift_range=0.1, 	height_shift_range=0.1,
		shear_range=0.15, 	horizontal_flip=True, 	fill_mode="nearest")
                                                                                                           ## 
callback = EarlyStopping( monitor= "val_accuracy",    patience=15,       mode="max")

# checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1,   save_best_only=True, mode='max')
                                                                                                            ##
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5,   patience=6,     verbose=1, mode='max', min_lr=0.00001)

100%|██████████| 450/450 [01:48<00:00,  4.13it/s]


In [ ]:
base_model_Incep = tf.keras.applications.InceptionV3(input_shape=(299, 299, 3), include_top=False, weights="imagenet")
len(base_model_Incep.layers)

base_model_Incep.trainable = True
for layer in base_model_Incep.layers[:100]:    #fix w & bias in layer 0-100
    layer.trainable = False


#Add custom head
average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model_Incep.output)
prediction_layer = tf.keras.layers.Dense(units=3, activation="softmax")(average_pooling_layer)   # unit 3
model_Incep = tf.keras.models.Model(inputs=base_model_Incep.input, outputs=prediction_layer)

model_Incep.compile(optimizer=RMSprop(learning_rate=0.0001), loss="binary_crossentropy", metrics=["accuracy"])


### K-fold
kf_Incep = KFold(5, shuffle=True, random_state=40) 

oos_y = []
oos_pred = []
fold = 0

for train, test in kf_Incep.split(data_Incep):
    fold+=1   
    print(f"Fold #{fold}")

    filename = f" Kmodel32_soft_pre_bi{fold}.h5 "
    checkpoint = ModelCheckpoint(filename, monitor='val_accuracy', verbose=1,   save_best_only=True, mode='max')
        
    x_train_Incep = data_Incep[train]
    y_train_Incep = labels_Incep[train]
    x_test_Incep = data_Incep[test]
    y_test_Incep = labels_Incep[test]

    H_Incep = model_Incep.fit( x=aug.flow(x_train_Incep, y_train_Incep, batch_size=BS),
	                                            validation_data=(x_test_Incep, y_test_Incep),
	                                            steps_per_epoch=len(x_train_Incep) // BS,
	                                            epochs=100,  callbacks = [callback,checkpoint,reduce_lr])

    pred_Incep = model_Incep.predict(x_test_Incep)
    
    oos_y.append(y_test_Incep)
    oos_pred.append(pred_Incep)  
    print(classification_report(y_test_Incep.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        pred_Incep.argmax(axis=1),  
                                        target_names=le_Incep.classes_))
    
oos_y = np.concatenate(oos_y)
oos_pred = np.concatenate(oos_pred)
print(classification_report(oos_y.argmax(axis=1), oos_pred.argmax(axis=1),  target_names=le_Incep.classes_))

87924736/87910968 [==============================] - 1s 0us/step
Fold #1
Epoch 1/100
11/11 [==============================] - ETA: 0s - loss: 0.5182 - accuracy: 0.6067
Epoch 1: val_accuracy improved from -inf to 0.34444, saving model to  Kmodel32_soft_pre_bi1.h5 
INFO:tensorflow:Assets written to:  Kmodel32_soft_pre_bi1.h5 /assets
11/11 [==============================] - 79s 5s/step - loss: 0.5182 - accuracy: 0.6067 - val_loss: 1.2654 - val_accuracy: 0.3444 - lr: 1.0000e-04
Epoch 2/100
11/11 [==============================] - ETA: 0s - loss: 0.3303 - accuracy: 0.7744
Epoch 2: val_accuracy did not improve from 0.34444
11/11 [==============================] - 10s 874ms/step - loss: 0.3303 - accuracy: 0.7744 - val_loss: 1.4151 - val_accuracy: 0.3444 - lr: 1.0000e-04
Epoch 3/100
11/11 [==============================] - ETA: 0s - loss: 0.2257 - accuracy: 0.8659
Epoch 3: val_accuracy did not improve from 0.34444
11/11 [==============================] - 10s 964ms/step - loss: 0.2257 - accurac

#### 32 soft pre bi adam  


In [ ]:
BS = 32
filepath = "Kmodel32_soft_pre_bi_adam.h5"                  ##

train_img_Incep = []
for ImageName in tqdm(data_train3['ImageName']):              
    image_path = All_3class_dir +'/' + ImageName                  
    img = cv2.imread(image_path)                                          
    img = cv2.resize(img, (299, 299))                                      
    train_img_Incep.append(img)
data_Incep = np.array(train_img_Incep, dtype="float32") / 255.0

labels_Incep = np.array(data_train3[['NameType']] )        
labels_Incep.shape = (len(labels_Incep),) 
le_Incep = LabelEncoder()                                                                     
labels_Incep = le_Incep.fit_transform(labels_Incep)                                         
labels_Incep = to_categorical(labels_Incep, num_classes=3)                   

aug = ImageDataGenerator( rotation_range=15,	width_shift_range=0.1, 	height_shift_range=0.1,
		shear_range=0.15, 	horizontal_flip=True, 	fill_mode="nearest")
                                                                                                           ## 
callback = EarlyStopping( monitor= "val_accuracy",    patience=15,       mode="max" )

# checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1,   save_best_only=True, mode='max')
                                                                                                            ##
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5,   patience=6,     verbose=1, mode='max', min_lr=0.00001)

100%|██████████| 450/450 [00:05<00:00, 82.73it/s] 


In [ ]:
base_model_Incep = tf.keras.applications.InceptionV3(input_shape=(299, 299, 3), include_top=False, weights="imagenet")
len(base_model_Incep.layers)

base_model_Incep.trainable = True
for layer in base_model_Incep.layers[:100]:    #fix w & bias in layer 0-100
    layer.trainable = False


#Add custom head
average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model_Incep.output)
prediction_layer = tf.keras.layers.Dense(units=3, activation="softmax")(average_pooling_layer)   # unit 3
model_Incep = tf.keras.models.Model(inputs=base_model_Incep.input, outputs=prediction_layer)

model_Incep.compile(optimizer=Adam(learning_rate=0.0001), loss="binary_crossentropy", metrics=["accuracy"])


### K-fold
kf_Incep = KFold(5, shuffle=True, random_state=40) 

oos_y = []
oos_pred = []
fold = 0

for train, test in kf_Incep.split(data_Incep):
    fold+=1   
    print(f"Fold #{fold}")

    filename = f" Kmodel32_soft_pre_bi_adam{fold}.h5 "
    checkpoint = ModelCheckpoint(filename, monitor='val_accuracy', verbose=1,   save_best_only=True, mode='max')
        
    x_train_Incep = data_Incep[train]
    y_train_Incep = labels_Incep[train]
    x_test_Incep = data_Incep[test]
    y_test_Incep = labels_Incep[test]

    H_Incep = model_Incep.fit( x=aug.flow(x_train_Incep, y_train_Incep, batch_size=BS),
	                                            validation_data=(x_test_Incep, y_test_Incep),
	                                            steps_per_epoch=len(x_train_Incep) // BS,
	                                            epochs=100,  callbacks = [callback,checkpoint,reduce_lr])

    pred_Incep = model_Incep.predict(x_test_Incep)
    
    oos_y.append(y_test_Incep)
    oos_pred.append(pred_Incep)  
    print(classification_report(y_test_Incep.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        pred_Incep.argmax(axis=1),  
                                        target_names=le_Incep.classes_))
    
oos_y = np.concatenate(oos_y)
oos_pred = np.concatenate(oos_pred)
print(classification_report(oos_y.argmax(axis=1), oos_pred.argmax(axis=1),  target_names=le_Incep.classes_))

Fold #1
Epoch 1/100
11/11 [==============================] - ETA: 0s - loss: 0.5595 - accuracy: 0.5945
Epoch 1: val_accuracy improved from -inf to 0.36667, saving model to  Kmodel32_soft_pre_bi_adam1.h5 
INFO:tensorflow:Assets written to:  Kmodel32_soft_pre_bi_adam1.h5 /assets
11/11 [==============================] - 68s 5s/step - loss: 0.5595 - accuracy: 0.5945 - val_loss: 0.8587 - val_accuracy: 0.3667 - lr: 1.0000e-04
Epoch 2/100
11/11 [==============================] - ETA: 0s - loss: 0.3488 - accuracy: 0.7713
Epoch 2: val_accuracy did not improve from 0.36667
11/11 [==============================] - 10s 919ms/step - loss: 0.3488 - accuracy: 0.7713 - val_loss: 1.1159 - val_accuracy: 0.3444 - lr: 1.0000e-04
Epoch 3/100
11/11 [==============================] - ETA: 0s - loss: 0.2659 - accuracy: 0.8628
Epoch 3: val_accuracy did not improve from 0.36667
11/11 [==============================] - 10s 911ms/step - loss: 0.2659 - accuracy: 0.8628 - val_loss: 1.2206 - val_accuracy: 0.3556 - l

#### 32 soft pre bi sgd  


In [ ]:
BS = 32
filepath = "Kmodel32_soft_pre_bi_sgd.h5"                  ##

train_img_Incep = []
for ImageName in tqdm(data_train3['ImageName']):              
    image_path = All_3class_dir +'/' + ImageName                  
    img = cv2.imread(image_path)                                          
    img = cv2.resize(img, (299, 299))                                      
    train_img_Incep.append(img)
data_Incep = np.array(train_img_Incep, dtype="float32") / 255.0

labels_Incep = np.array(data_train3[['NameType']] )        
labels_Incep.shape = (len(labels_Incep),) 
le_Incep = LabelEncoder()                                                                     
labels_Incep = le_Incep.fit_transform(labels_Incep)                                         
labels_Incep = to_categorical(labels_Incep, num_classes=3)                   

aug = ImageDataGenerator( rotation_range=15,	width_shift_range=0.1, 	height_shift_range=0.1,
		shear_range=0.15, 	horizontal_flip=True, 	fill_mode="nearest")
                                                                                                           ## 
callback = EarlyStopping( monitor= "val_accuracy",    patience=15,       mode="max" )

# checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1,   save_best_only=True, mode='max')
                                                                                                            ##
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5,   patience=6,     verbose=1, mode='max', min_lr=0.00001)

100%|██████████| 450/450 [00:05<00:00, 88.34it/s] 


In [ ]:
base_model_Incep = tf.keras.applications.InceptionV3(input_shape=(299, 299, 3), include_top=False, weights="imagenet")
len(base_model_Incep.layers)

base_model_Incep.trainable = True
for layer in base_model_Incep.layers[:100]:    #fix w & bias in layer 0-100
    layer.trainable = False


#Add custom head
average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model_Incep.output)
prediction_layer = tf.keras.layers.Dense(units=3, activation="softmax")(average_pooling_layer)   # unit 3
model_Incep = tf.keras.models.Model(inputs=base_model_Incep.input, outputs=prediction_layer)

model_Incep.compile(optimizer=SGD(learning_rate=0.0001), loss="binary_crossentropy", metrics=["accuracy"])


### K-fold
kf_Incep = KFold(5, shuffle=True, random_state=40) 

oos_y = []
oos_pred = []
fold = 0

for train, test in kf_Incep.split(data_Incep):
    fold+=1   
    print(f"Fold #{fold}")

    filename = f" Kmodel32_soft_pre_bi_sgd{fold}.h5 "
    checkpoint = ModelCheckpoint(filename, monitor='val_accuracy', verbose=1,   save_best_only=True, mode='max')
        
    x_train_Incep = data_Incep[train]
    y_train_Incep = labels_Incep[train]
    x_test_Incep = data_Incep[test]
    y_test_Incep = labels_Incep[test]

    H_Incep = model_Incep.fit( x=aug.flow(x_train_Incep, y_train_Incep, batch_size=BS),
	                                            validation_data=(x_test_Incep, y_test_Incep),
	                                            steps_per_epoch=len(x_train_Incep) // BS,
	                                            epochs=100,  callbacks = [callback,checkpoint,reduce_lr])

    pred_Incep = model_Incep.predict(x_test_Incep)
    
    oos_y.append(y_test_Incep)
    oos_pred.append(pred_Incep)  
    print(classification_report(y_test_Incep.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        pred_Incep.argmax(axis=1),  
                                        target_names=le_Incep.classes_))
    
oos_y = np.concatenate(oos_y)
oos_pred = np.concatenate(oos_pred)
print(classification_report(oos_y.argmax(axis=1), oos_pred.argmax(axis=1),  target_names=le_Incep.classes_))

Fold #1
Epoch 1/100
11/11 [==============================] - ETA: 0s - loss: 0.7136 - accuracy: 0.3232
Epoch 1: val_accuracy improved from -inf to 0.28889, saving model to  Kmodel32_soft_pre_bi_sgd1.h5 
INFO:tensorflow:Assets written to:  Kmodel32_soft_pre_bi_sgd1.h5 /assets
11/11 [==============================] - 53s 4s/step - loss: 0.7136 - accuracy: 0.3232 - val_loss: 0.7855 - val_accuracy: 0.2889 - lr: 1.0000e-04
Epoch 2/100
11/11 [==============================] - ETA: 0s - loss: 0.7109 - accuracy: 0.3720
Epoch 2: val_accuracy did not improve from 0.28889
11/11 [==============================] - 10s 884ms/step - loss: 0.7109 - accuracy: 0.3720 - val_loss: 0.7706 - val_accuracy: 0.2889 - lr: 1.0000e-04
Epoch 3/100
11/11 [==============================] - ETA: 0s - loss: 0.7120 - accuracy: 0.3018
Epoch 3: val_accuracy improved from 0.28889 to 0.30000, saving model to  Kmodel32_soft_pre_bi_sgd1.h5 
INFO:tensorflow:Assets written to:  Kmodel32_soft_pre_bi_sgd1.h5 /assets
11/11 [=====

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

  Lungcancer       0.17      0.04      0.06        28
      Normal       0.00      0.00      0.00        31
Tuberculosis       0.32      0.87      0.47        31

    accuracy                           0.31        90
   macro avg       0.16      0.30      0.18        90
weighted avg       0.16      0.31      0.18        90

Fold #2
Epoch 1/100
11/11 [==============================] - ETA: 0s - loss: 0.6881 - accuracy: 0.3628
Epoch 1: val_accuracy improved from -inf to 0.36667, saving model to  Kmodel32_soft_pre_bi_sgd2.h5 
INFO:tensorflow:Assets written to:  Kmodel32_soft_pre_bi_sgd2.h5 /assets
11/11 [==============================] - 45s 4s/step - loss: 0.6881 - accuracy: 0.3628 - val_loss: 0.6807 - val_accuracy: 0.3667 - lr: 2.5000e-05
Epoch 2/100
11/11 [==============================] - ETA: 0s - loss: 0.6839 - accuracy: 0.3598
Epoch 2: val_accuracy improved from 0.36667 to 0.38889, saving model to  Kmodel32_soft_pre_bi_sgd2.h5 

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

  Lungcancer       0.46      0.32      0.38        34
      Normal       0.00      0.00      0.00        27
Tuberculosis       0.35      0.79      0.48        29

    accuracy                           0.38        90
   macro avg       0.27      0.37      0.29        90
weighted avg       0.29      0.38      0.30        90

Fold #3
Epoch 1/100
11/11 [==============================] - ETA: 0s - loss: 0.6828 - accuracy: 0.3659
Epoch 1: val_accuracy improved from -inf to 0.36667, saving model to  Kmodel32_soft_pre_bi_sgd3.h5 
INFO:tensorflow:Assets written to:  Kmodel32_soft_pre_bi_sgd3.h5 /assets
11/11 [==============================] - 45s 4s/step - loss: 0.6828 - accuracy: 0.3659 - val_loss: 0.6837 - val_accuracy: 0.3667 - lr: 1.0000e-05
Epoch 2/100
11/11 [==============================] - ETA: 0s - loss: 0.6858 - accuracy: 0.3659
Epoch 2: val_accuracy did not improve from 0.36667
11/11 [==============================] - 10s 856ms/

#### 32 soft pre cate 


In [ ]:
BS = 32
filepath = "Kmodel32_soft_pre_cate.h5"                  ##

train_img_Incep = []
for ImageName in tqdm(data_train3['ImageName']):              
    image_path = All_3class_dir +'/' + ImageName                  
    img = cv2.imread(image_path)                                          
    img = cv2.resize(img, (299, 299))                                      
    train_img_Incep.append(img)
data_Incep = np.array(train_img_Incep, dtype="float32") / 255.0

labels_Incep = np.array(data_train3[['NameType']] )        
labels_Incep.shape = (len(labels_Incep),) 
le_Incep = LabelEncoder()                                                                     
labels_Incep = le_Incep.fit_transform(labels_Incep)                                         
labels_Incep = to_categorical(labels_Incep, num_classes=3)                   

aug = ImageDataGenerator( rotation_range=15,	width_shift_range=0.1, 	height_shift_range=0.1,
		shear_range=0.15, 	horizontal_flip=True, 	fill_mode="nearest")
                                                                                                           ## 
callback = EarlyStopping( monitor= "val_accuracy",    patience=15,       mode="max" )

# checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1,   save_best_only=True, mode='max')
                                                                                                            ##
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5,   patience=6,     verbose=1, mode='max', min_lr=0.00001)

100%|██████████| 450/450 [00:05<00:00, 84.27it/s] 


In [ ]:
base_model_Incep = tf.keras.applications.InceptionV3(input_shape=(299, 299, 3), include_top=False, weights="imagenet")
len(base_model_Incep.layers)

base_model_Incep.trainable = True
for layer in base_model_Incep.layers[:100]:    #fix w & bias in layer 0-100
    layer.trainable = False


#Add custom head
average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model_Incep.output)
prediction_layer = tf.keras.layers.Dense(units=3, activation="softmax")(average_pooling_layer)   # unit 3
model_Incep = tf.keras.models.Model(inputs=base_model_Incep.input, outputs=prediction_layer)

model_Incep.compile(optimizer=RMSprop(learning_rate=0.0001), loss="categorical_crossentropy", metrics=["accuracy"])


### K-fold
kf_Incep = KFold(5, shuffle=True, random_state=40) 

oos_y = []
oos_pred = []
fold = 0

for train, test in kf_Incep.split(data_Incep):
    fold+=1   
    print(f"Fold #{fold}")

    filename = f" Kmodel32_soft_pre_cate{fold}.h5 "
    checkpoint = ModelCheckpoint(filename, monitor='val_accuracy', verbose=1,   save_best_only=True, mode='max')
        
    x_train_Incep = data_Incep[train]
    y_train_Incep = labels_Incep[train]
    x_test_Incep = data_Incep[test]
    y_test_Incep = labels_Incep[test]

    H_Incep = model_Incep.fit( x=aug.flow(x_train_Incep, y_train_Incep, batch_size=BS),
	                                            validation_data=(x_test_Incep, y_test_Incep),
	                                            steps_per_epoch=len(x_train_Incep) // BS,
	                                            epochs=100,  callbacks = [callback,checkpoint,reduce_lr])

    pred_Incep = model_Incep.predict(x_test_Incep)
    
    oos_y.append(y_test_Incep)
    oos_pred.append(pred_Incep)  
    print(classification_report(y_test_Incep.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        pred_Incep.argmax(axis=1),  
                                        target_names=le_Incep.classes_))
    
oos_y = np.concatenate(oos_y)
oos_pred = np.concatenate(oos_pred)
print(classification_report(oos_y.argmax(axis=1), oos_pred.argmax(axis=1),  target_names=le_Incep.classes_))

Fold #1
Epoch 1/100
11/11 [==============================] - ETA: 0s - loss: 0.7960 - accuracy: 0.6402
Epoch 1: val_accuracy improved from -inf to 0.51111, saving model to  Kmodel32_soft_pre_cate1.h5 
INFO:tensorflow:Assets written to:  Kmodel32_soft_pre_cate1.h5 /assets
11/11 [==============================] - 75s 5s/step - loss: 0.7960 - accuracy: 0.6402 - val_loss: 0.9990 - val_accuracy: 0.5111 - lr: 1.0000e-04
Epoch 2/100
11/11 [==============================] - ETA: 0s - loss: 0.5359 - accuracy: 0.7805
Epoch 2: val_accuracy improved from 0.51111 to 0.64444, saving model to  Kmodel32_soft_pre_cate1.h5 
INFO:tensorflow:Assets written to:  Kmodel32_soft_pre_cate1.h5 /assets
11/11 [==============================] - 46s 4s/step - loss: 0.5359 - accuracy: 0.7805 - val_loss: 0.7850 - val_accuracy: 0.6444 - lr: 1.0000e-04
Epoch 3/100
11/11 [==============================] - ETA: 0s - loss: 0.3368 - accuracy: 0.8872
Epoch 3: val_accuracy improved from 0.64444 to 0.71111, saving model to  K

#### 32 soft pre cate adam


In [ ]:
BS = 32
filepath = "Kmodel32_soft_pre_cate_adam.h5"                  ##

train_img_Incep = []
for ImageName in tqdm(data_train3['ImageName']):              
    image_path = All_3class_dir +'/' + ImageName                  
    img = cv2.imread(image_path)                                          
    img = cv2.resize(img, (299, 299))                                      
    train_img_Incep.append(img)
data_Incep = np.array(train_img_Incep, dtype="float32") / 255.0

labels_Incep = np.array(data_train3[['NameType']] )        
labels_Incep.shape = (len(labels_Incep),) 
le_Incep = LabelEncoder()                                                                     
labels_Incep = le_Incep.fit_transform(labels_Incep)                                         
labels_Incep = to_categorical(labels_Incep, num_classes=3)                   

aug = ImageDataGenerator( rotation_range=15,	width_shift_range=0.1, 	height_shift_range=0.1,
		shear_range=0.15, 	horizontal_flip=True, 	fill_mode="nearest")
                                                                                                           ## 
callback = EarlyStopping( monitor= "val_accuracy",    patience=15,       mode="max" )

# checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1,   save_best_only=True, mode='max')
                                                                                                            ##
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5,   patience=6,     verbose=1, mode='max', min_lr=0.00001)

100%|██████████| 450/450 [00:05<00:00, 79.81it/s] 


In [ ]:
base_model_Incep = tf.keras.applications.InceptionV3(input_shape=(299, 299, 3), include_top=False, weights="imagenet")
len(base_model_Incep.layers)

base_model_Incep.trainable = True
for layer in base_model_Incep.layers[:100]:    #fix w & bias in layer 0-100
    layer.trainable = False


#Add custom head
average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model_Incep.output)
prediction_layer = tf.keras.layers.Dense(units=3, activation="softmax")(average_pooling_layer)   # unit 3
model_Incep = tf.keras.models.Model(inputs=base_model_Incep.input, outputs=prediction_layer)

model_Incep.compile(optimizer=Adam(learning_rate=0.0001), loss="categorical_crossentropy", metrics=["accuracy"])

### K-fold
kf_Incep = KFold(5, shuffle=True, random_state=40) 

oos_y = []
oos_pred = []
fold = 0

for train, test in kf_Incep.split(data_Incep):
    fold+=1   
    print(f"Fold #{fold}")

    filename = f" Kmodel32_soft_pre_cate_adam{fold}.h5 "
    checkpoint = ModelCheckpoint(filename, monitor='val_accuracy', verbose=1,   save_best_only=True, mode='max')
        
    x_train_Incep = data_Incep[train]
    y_train_Incep = labels_Incep[train]
    x_test_Incep = data_Incep[test]
    y_test_Incep = labels_Incep[test]

    H_Incep = model_Incep.fit( x=aug.flow(x_train_Incep, y_train_Incep, batch_size=BS),
	                                            validation_data=(x_test_Incep, y_test_Incep),
	                                            steps_per_epoch=len(x_train_Incep) // BS,
	                                            epochs=100,  callbacks = [callback,checkpoint,reduce_lr])

    pred_Incep = model_Incep.predict(x_test_Incep)
    
    oos_y.append(y_test_Incep)
    oos_pred.append(pred_Incep)  
    print(classification_report(y_test_Incep.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        pred_Incep.argmax(axis=1),  
                                        target_names=le_Incep.classes_))
    
oos_y = np.concatenate(oos_y)
oos_pred = np.concatenate(oos_pred)
print(classification_report(oos_y.argmax(axis=1), oos_pred.argmax(axis=1),  target_names=le_Incep.classes_))

Fold #1
Epoch 1/100
11/11 [==============================] - ETA: 0s - loss: 0.8509 - accuracy: 0.5762
Epoch 1: val_accuracy improved from -inf to 0.37778, saving model to  Kmodel32_soft_pre_cate_adam1.h5 
INFO:tensorflow:Assets written to:  Kmodel32_soft_pre_cate_adam1.h5 /assets
11/11 [==============================] - 72s 6s/step - loss: 0.8509 - accuracy: 0.5762 - val_loss: 1.9174 - val_accuracy: 0.3778 - lr: 1.0000e-04
Epoch 2/100
11/11 [==============================] - ETA: 0s - loss: 0.5210 - accuracy: 0.7988
Epoch 2: val_accuracy improved from 0.37778 to 0.38889, saving model to  Kmodel32_soft_pre_cate_adam1.h5 
INFO:tensorflow:Assets written to:  Kmodel32_soft_pre_cate_adam1.h5 /assets
11/11 [==============================] - 47s 5s/step - loss: 0.5210 - accuracy: 0.7988 - val_loss: 1.7730 - val_accuracy: 0.3889 - lr: 1.0000e-04
Epoch 3/100
11/11 [==============================] - ETA: 0s - loss: 0.3777 - accuracy: 0.8210
Epoch 3: val_accuracy improved from 0.38889 to 0.43333

#### 32 soft pre cate sgd


In [ ]:
BS = 32
filepath = "Kmodel32_soft_pre_cate_sgd.h5"                  ##

train_img_Incep = []
for ImageName in tqdm(data_train3['ImageName']):              
    image_path = All_3class_dir +'/' + ImageName                  
    img = cv2.imread(image_path)                                          
    img = cv2.resize(img, (299, 299))                                      
    train_img_Incep.append(img)
data_Incep = np.array(train_img_Incep, dtype="float32") / 255.0

labels_Incep = np.array(data_train3[['NameType']] )        
labels_Incep.shape = (len(labels_Incep),) 
le_Incep = LabelEncoder()                                                                     
labels_Incep = le_Incep.fit_transform(labels_Incep)                                         
labels_Incep = to_categorical(labels_Incep, num_classes=3)                   

aug = ImageDataGenerator( rotation_range=15,	width_shift_range=0.1, 	height_shift_range=0.1,
		shear_range=0.15, 	horizontal_flip=True, 	fill_mode="nearest")
                                                                                                           ## 
callback = EarlyStopping( monitor= "val_accuracy",    patience=15,    mode="max" )    #, baseline=1.0

# checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1,   save_best_only=True, mode='max')
                                                                                                            ##
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5,   patience=6,     verbose=1, mode='max', min_lr=0.00001)

100%|██████████| 450/450 [00:08<00:00, 53.32it/s] 


In [ ]:
base_model_Incep = tf.keras.applications.InceptionV3(input_shape=(299, 299, 3), include_top=False, weights="imagenet")
len(base_model_Incep.layers)

base_model_Incep.trainable = True
for layer in base_model_Incep.layers[:100]:    #fix w & bias in layer 0-100
    layer.trainable = False


#Add custom head
average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model_Incep.output)
prediction_layer = tf.keras.layers.Dense(units=3, activation="softmax")(average_pooling_layer)   # unit 3
model_Incep = tf.keras.models.Model(inputs=base_model_Incep.input, outputs=prediction_layer)

model_Incep.compile(optimizer=SGD(learning_rate=0.0001), loss="categorical_crossentropy", metrics=["accuracy"])

### K-fold
kf_Incep = KFold(5, shuffle=True, random_state=40) 

oos_y = []
oos_pred = []
fold = 0

for train, test in kf_Incep.split(data_Incep):
    fold+=1   
    print(f"Fold #{fold}")

    filename = f" Kmodel32_soft_pre_cate_sgd{fold}.h5 "
    checkpoint = ModelCheckpoint(filename, monitor='val_accuracy', verbose=1,   save_best_only=True, mode='max')
        
    x_train_Incep = data_Incep[train]
    y_train_Incep = labels_Incep[train]
    x_test_Incep = data_Incep[test]
    y_test_Incep = labels_Incep[test]

    H_Incep = model_Incep.fit( x=aug.flow(x_train_Incep, y_train_Incep, batch_size=BS),
	                                            validation_data=(x_test_Incep, y_test_Incep),
	                                            steps_per_epoch=len(x_train_Incep) // BS,
	                                            epochs=100,  callbacks = [callback,checkpoint,reduce_lr])

    pred_Incep = model_Incep.predict(x_test_Incep)
    
    oos_y.append(y_test_Incep)
    oos_pred.append(pred_Incep)  
    print(classification_report(y_test_Incep.argmax(axis=1),	                #แสดง report ค่า acc, recall, ...
                                        pred_Incep.argmax(axis=1),  
                                        target_names=le_Incep.classes_))
    
oos_y = np.concatenate(oos_y)
oos_pred = np.concatenate(oos_pred)
print(classification_report(oos_y.argmax(axis=1), oos_pred.argmax(axis=1),  target_names=le_Incep.classes_))

Fold #1
Epoch 1/100
11/11 [==============================] - ETA: 0s - loss: 1.1341 - accuracy: 0.3293
Epoch 1: val_accuracy improved from -inf to 0.37778, saving model to  Kmodel32_soft_pre_cate_sgd1.h5 
INFO:tensorflow:Assets written to:  Kmodel32_soft_pre_cate_sgd1.h5 /assets
11/11 [==============================] - 54s 4s/step - loss: 1.1341 - accuracy: 0.3293 - val_loss: 1.1104 - val_accuracy: 0.3778 - lr: 1.0000e-04
Epoch 2/100
11/11 [==============================] - ETA: 0s - loss: 1.1493 - accuracy: 0.3323
Epoch 2: val_accuracy improved from 0.37778 to 0.42222, saving model to  Kmodel32_soft_pre_cate_sgd1.h5 
INFO:tensorflow:Assets written to:  Kmodel32_soft_pre_cate_sgd1.h5 /assets
11/11 [==============================] - 43s 4s/step - loss: 1.1493 - accuracy: 0.3323 - val_loss: 1.1064 - val_accuracy: 0.4222 - lr: 1.0000e-04
Epoch 3/100
11/11 [==============================] - ETA: 0s - loss: 1.1307 - accuracy: 0.3445
Epoch 3: val_accuracy did not improve from 0.42222
11/11 [

###### import

In [ ]:
import os
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
import cv2
import numpy as np
import pandas as pd
from torchsummary import summary
from skimage.io import imread, imsave
from tqdm import tqdm

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix

In [ ]:
# set the matplotlib backend so figures can be saved in the background
import matplotlib
matplotlib.use("Agg")

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from tensorflow.keras.optimizers import SGD, Adam, RMSprop
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model
from imutils import paths
import argparse

from sklearn.model_selection import KFold
from sklearn import metrics
from scipy.stats import zscore


import matplotlib.image as mpimg
from matplotlib.pyplot import savefig

import itertools
from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels

In [ ]:
import pickle
import joblib

######Load data // KKUmail

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data_dir = '/content/drive/My Drive/Senior Project/CXR Image3class/Image'

train 3 class

In [ ]:
Normal151_dir = os.path.join(data_dir,'Normal151')
TB150_dir = os.path.join(data_dir,'TB150')
CA150_dir = os.path.join(data_dir,'CA150')

All_3class_dir = os.path.join(data_dir,'All_3class')

######Load data // Gmail

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data_dir = '/content/drive/My Drive/Senior Project/CXR Image3class'

train 3 class

In [ ]:
Normal151_dir = os.path.join(data_dir,'Normal151')
TB150_dir = os.path.join(data_dir,'TB150')
CA150_dir = os.path.join(data_dir,'CA150')

All_3class_dir = os.path.join(data_dir,'All_3class')